In [23]:
import ipywidgets as widgets
from IPython.display import clear_output, display, Markdown, Latex, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Layout, Button, Box, FloatProgress
from ipyfilechooser import FileChooser

import zipfile
import pickle
import pandas

import ktrain

import re


In [24]:
fl = None

In [25]:
style = "<style> \
        div.abstract \
            { \
                background-color: #FEFFF0;border-color: #F0F3BD; \
                border-left: 5px solid #f7f1e7;\
                padding: 0.5em; \
            } \
         </style>"
#
#        .tooltip { \
#          position: relative; \
#          display: inline-block; \
#          border-bottom: 1px dotted black; \
#          z-index: 1 \
#        } \
#        .tooltip .tooltiptext { \
#          visibility: hidden; \
#          width: 120px; \
#          background-color: black; \
#          color: #fff; \
#          text-align: center; \
#          padding: 5px 0; \
#          border-radius: 6px; \
#          position: absolute; \
#          z-index: 3; \
#        } \
#        .tooltip:hover .tooltiptext { \
#          visibility: visible; \
#        } \
       
HTML(style)

In [26]:
RESEARCH_ASPECTS = ["Background", "Purpose", "Method", "Finding_Contribution", "Other"]
COLORS = ['red','blue','gray','green','brown']

In [27]:
class FileLoader:
    def __init__(self):
        self.uids = {}
        self.docs = {}
        self.topic_models = {}
        self.available_classifications = []
        self.available_aspects = []
        self.papers_dict = {}
        
    def abstract_dict_to_string(self, abstract_dict, highlighted_aspect):
        whole_text_dict = {}
        whole_text = ''
        for aspect in RESEARCH_ASPECTS:
            for sentence in abstract_dict[aspect]:
                if(highlighted_aspect is not None):
                    if(aspect == highlighted_aspect):
                        whole_text_dict[sentence[0]] = '<strong>{}</strong>'.format(sentence[1])
                    else:
                        whole_text_dict[sentence[0]] = sentence[1]
                else:
                    whole_text_dict[sentence[0]] = sentence[1]

        for key in sorted(whole_text_dict):
            whole_text += whole_text_dict[key] + ' '
        
        return whole_text
        
    def load_topic_model(self, model, params, vectorizer):
        """
        Load saved TopicModel object
        Args:
            fname(str): base filename for all saved files
        """

        tm = ktrain.text.get_topic_model(n_topics=params['n_topics'],
                             n_features = params['n_features'],
                             verbose = params['verbose'])
        tm.model = model
        tm.vectorizer = vectorizer
        
        return tm
        
    
    def load_files(self,path):        
        with zipfile.ZipFile(path) as zf:
            print(zf.namelist())
            #LOAD PAPERS_DICT
            with zf.open(zf.namelist()[0]) as papers_dict:
                self.papers_dict = pickle.load( papers_dict )
        
            #LOAD UID_DICT
            
                
            for i in range(1, len(zf.namelist())):
                full_path = zf.namelist()[i]
                
                slash_indices = [i for i, ltr in enumerate(full_path) if ltr == '/']
                if(len(slash_indices)>2):
                    if full_path[slash_indices[0]+1:slash_indices[1]] not in self.available_classifications:
                        self.available_classifications.append(full_path[slash_indices[0]+1:slash_indices[1]])

                    if full_path[slash_indices[1]+1:slash_indices[2]] not in self.available_aspects:
                        self.available_aspects.append(full_path[slash_indices[1]+1:slash_indices[2]])
                
            for classification in self.available_classifications:
                self.topic_models[classification] = {}
                self.docs[classification] = {}
                
                with zf.open('built_files/'+classification+'/uids.p') as uids:
                    self.uids[classification] = pickle.load( uids )
                    print(self.uids[classification])
                
                for aspect in self.available_aspects:
                    files_path = 'built_files/'+classification+'/'+aspect+'/'
                    
                    with zf.open(files_path+'docs.p') as docs:
                        self.docs[classification][aspect] = pickle.load( docs )
                    
                    with zf.open(files_path+aspect+'.tm_model') as model_zip, zf.open(files_path+aspect+'.tm_params') as params_zip, zf.open(files_path+aspect+'.tm_vect') as vectorizer_zip:
                        self.topic_models[classification][aspect] = self.load_topic_model(pickle.load(model_zip), pickle.load(params_zip),pickle.load(vectorizer_zip))
                        self.topic_models[classification][aspect].build(self.docs[classification][aspect],threshold=0)
                        
                        #self.docs[classification][aspect] = self.topic_models[classification][aspect].filter(self.docs[classification][aspect])
                        #self.topic_models[classification][aspect].visualize_documents(texts=self.docs[classification][aspect])
            #s.find('/')

In [28]:
fl = FileLoader()  

In [29]:
box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='100%')

In [30]:
def selected_file():
    fl_box.children[1].disabled = False
    
    
title = widgets.Label(value='Upload the zip file')

#fc = FileChooser('./')

fl_box = widgets.Box(
    layout = box_layout,
    children=[
        FileChooser('./'),
        widgets.Button(
            description='LOAD BUILT FILES',
            disabled=True,
            button_style='', 
            icon='gear'),
        widgets.FloatProgress(disabled=True, min=0, max=100) 
    ]
)

def pressed_build_files(btn):
    fl_box.children[2].disabled = False
    path = fl_box.children[0].selected_path+'/'+fl_box.children[0].selected_filename
    fl.load_files(path)


fl_box.children[0].register_callback(selected_file)
fl_box.children[1].on_click(pressed_build_files)

### Load ZIP file (CARREGAR ARQUIVO ANTES DAS CELULAS SEGUINTES) ### 

In [31]:
display(fl_box)

Box(children=(FileChooser(path='.', filename='', show_hidden='False'), Button(description='LOAD BUILT FILES', …

['built_files/docs_dict.p', 'built_files/all_papers/uids.p', 'built_files/all_papers/Background/Background.tm_model', 'built_files/all_papers/Background/Background.tm_params', 'built_files/all_papers/Background/Background.tm_vect', 'built_files/all_papers/Background/docs.p', 'built_files/all_papers/Finding_Contribution/docs.p', 'built_files/all_papers/Finding_Contribution/Finding_Contribution.tm_model', 'built_files/all_papers/Finding_Contribution/Finding_Contribution.tm_params', 'built_files/all_papers/Finding_Contribution/Finding_Contribution.tm_vect', 'built_files/all_papers/Method/docs.p', 'built_files/all_papers/Method/Method.tm_model', 'built_files/all_papers/Method/Method.tm_params', 'built_files/all_papers/Method/Method.tm_vect', 'built_files/all_papers/Purpose/docs.p', 'built_files/all_papers/Purpose/Purpose.tm_model', 'built_files/all_papers/Purpose/Purpose.tm_params', 'built_files/all_papers/Purpose/Purpose.tm_vect', 'built_files/all_papers/Whole/docs.p', 'built_files/all_pa

['uk8rfroj', 'ivwn4nhl', '74k12s1z', 'p5bahejq', 'm5h19hy6', '6tso9our', '8g70j0qw', 'hoybp1h6', '9em5tjya', 'j97ugs3y', '4uiyj6lg', 'l63z3btv', 'u473zap9', '4mj7n0ax', '4r0t3q7j', 'meebb4ic', 'dqour5jr', 'zkicn27x', 'r6awepkt', 'tfh1dev6', 'v4t6ca0c', '68sbqdi3', 'jx3alip6', 'boqygpow', 'mar8zt2t', '89fol3pq', 'z6pwfshq', '0y70ae8o', 'l5fxswfz', 'vf85qt8j', 'vpih1wvs', 'uglsx7se', 'f1q7fsax', 'i1q3n101', 'cy4pn193', 'nwl95ysz', '8lud7tqx', 'q9d88fxx', 'udr1z9wf', 'ayi928ct', 'sga5is3q', '4feo08nh', 'n8vvqb20', 'h4cu63cm', 'ajqbljai', '7e8dwcur', '0jxgefdc', 'br33p9xd', 'gm88jqvl', '7u4b8if7', 'm5u3cnvh', 'n8xfmcov', '7acoduj3', '8g6julo6', 'ndscdqcb', 'hmpmm9k8', 'v3qx51f8', 'egzztatj', 'ym2s6fd4', 'zph6r4il', 'ha110a72', 'p2u809yz', 'y55ymsjg', '4msw4cdb', 'agnbus8b', 'wu8avsp6', '8uhjvw7x', 'cxfsf1ic', 'g3ovti22', 'fkzfzml6', 'eqo633am', '1gnoo0us', 'fce2kdzw', '89n52jny', 't5rdljyk', 'yahdp3td', 'iips79vo', 'psfi3hsv', 'm6eal7lu', '7aop9iay', 'ise9c444', '53elkc79', 'nro623aa', 'ke

In [33]:
pandas.set_option('display.max_colwidth', 1000)
def return_topics_df(classification_bool, aspect):
    classification = ''
    if(classification_bool):
        classification='covid_papers'
    else:
        classification='all_papers'
        
    #fl.topic_models[classification][aspect].print_topics(show_counts=True)\
    detailed_topics = {}
    topic_model = fl.topic_models[classification][aspect]
    topic_model._check_build()
    topic_counts = sorted([ (k, topic_model.topics[k], len(v)) for k,v in topic_model.topic_dict.items()], 
                            key=lambda kv:kv[-1], reverse=True)
    for (idx, topic, count) in topic_counts:
        detailed_topics[idx] =  {"Topics words":topic, "Number of papers":count}
    
    return pandas.DataFrame.from_dict(detailed_topics, orient='index')

df = pandas.DataFrame()


box1 = widgets.Box(
    layout = box_layout,
    children=[
        widgets.Label(value='Get topics for:'),
        widgets.Dropdown(
            options=fl.available_aspects,
            layout={'width': 'max-content'},
            description='Aspect:'
        ),
        widgets.Checkbox(
            value=True,
            description='Covid-19 papers',
            disabled=False,
            indent=False,
            layout={'width': 'max-content'}
        ),
        widgets.Button(
            description='Search'
        ),
        
    ]
)

def btn_eventhandler(obj):
    clear_output(wait=True)
    display(box1)
    display(return_topics_df(box1.children[2].value, box1.children[1].value))
    
box1.children[3].on_click(btn_eventhandler)



#display(box)

In [82]:
def get_num_docs_topics(classification_bool, aspect, topic_id):
    classification = ''
    if(classification_bool):
        classification='covid_papers'
    else:
        classification='all_papers'
        
    topic_model = fl.topic_models[classification][aspect]
    return len(topic_model.get_docs(topic_ids=[topic_id]))

def get_selectable_topics(classification_bool, aspect, filter_words):
    classification = ''
    if(classification_bool):
        classification='covid_papers'
    else:
        classification='all_papers'
        
    selectable_topics = []
    topic_model = fl.topic_models[classification][aspect]
    topics = topic_model.get_topics(n_words=10, as_string=True)
    
    topic_model._check_build()
    topic_counts = sorted([ (k, topic_model.topics[k], len(v)) for k,v in topic_model.topic_dict.items()], 
                            key=lambda kv:kv[-1])
    print(topic_counts)
    for (idx, topic, count) in topic_counts:
        if(any(topic_word in filter_words.split() for topic_word in topic.split()) or filter_words == ''):
            selectable_topics.append((topic, idx))
         
    return selectable_topics

def return_docs_in_topic(classification_bool, aspect, topic_id, n):
    classification = ''
    if(classification_bool):
        classification='covid_papers'
    else:
        classification='all_papers'
        
    docs_in_topic = {}
    topic_model = fl.topic_models[classification][aspect]
    docs = topic_model.get_docs(topic_ids=[topic_id], rank=True)
    for i in range(0,n):
        try:
            doc_uid = fl.uids[classification][docs[i][1]]
            doc_title = fl.papers_dict[fl.uids[classification][docs[i][1]]]['title']
            doc_abstract = fl.abstract_dict_to_string(fl.papers_dict[fl.uids[classification][docs[i][1]]]['abstract'], aspect)
            docs_in_topic[doc_uid] ={'title':doc_title,'abstract': doc_abstract, 'score':docs[i][2]}
        except Exception as e:
            return docs_in_topic
    
    
    return docs_in_topic
    #for doc in docs:
        #docs_in_topic[doc[]]

pandas.set_option('display.max_colwidth', 1000)


items_layout = Layout( width='auto')

#pandas.set_option('display.max_colwidth', 40)

#topics_dropdown = widgets.Dropdown(
#    options=[('Whole abstract', topics_all_50_whole), ('Abstract Findings', topics_findings_50_whole), ('Abstract Purpose', topics_findings_50_whole)],
#    value=topics_findings_50_whole,
#    description='Get topics for:',
#)
df = pandas.DataFrame()


box2 = widgets.Box(
    layout = box_layout,
    children=[
        widgets.Label(value='Get topics for:', layout={'width': 'max-content'}),
        widgets.Dropdown(
            options=fl.available_aspects,
            description='Aspect:'
        ),
        widgets.Dropdown(
            options=[],
            description='Topic:'
        ),
        widgets.Textarea(
            value='',
            placeholder='Query Sentence',
            description='Filter topic:',
            disabled=False
        ),
        widgets.Checkbox(
            value=True,
            description='Covid-19 papers',
            disabled=False,
            indent=False,
            layout={'width': 'max-content'}
        ),
        widgets.IntSlider(
            value=7,
            min=0,
            max=10,
            step=1,
            description='Number of documents:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d',
            style={'description_width': 'initial'}
        ),
        
        widgets.Button(
            description='Search',
            disabled=True,
        ),
        
    ]
)

def select_eventhandler(obj):
    box2.children[6].disabled = True
    box2.children[2].options = get_selectable_topics(box2.children[4].value, box2.children[1].value, box2.children[3].value)
    box2.children[5].max = get_num_docs_topics(box2.children[4].value, box2.children[1].value, box2.children[2].value)
    box2.children[6].disabled = False
    
def checkbox_eventhandler(obj):
    box2.children[6].disabled = True
    box2.children[2].options = get_selectable_topics(box2.children[4].value, box2.children[1].value, box2.children[3].value)
    box2.children[5].max = get_num_docs_topics(box2.children[4].value, box2.children[1].value, box2.children[2].value)
    box2.children[6].disabled = False
    
def search_btn_eventhandler(btn):
    clear_output(wait=True)
    display(box2)
    
    docs_in_topic = return_docs_in_topic(box2.children[4].value, box2.children[1].value, box2.children[2].value, box2.children[5].value)
    display(Markdown(pandas.DataFrame.from_dict(docs_in_topic, orient='index').to_markdown()))

    
def topic_eventhandler(btn):
    box2.children[5].max = get_num_docs_topics(box2.children[4].value, box2.children[1].value, box2.children[2].value)
    
def filter_eventhandler(text_field):
    box2.children[2].options = get_selectable_topics(box2.children[4].value, box2.children[1].value, box2.children[3].value)
    
select_eventhandler(box2.children[1])
box2.children[1].observe(select_eventhandler, names=['value'])
box2.children[4].observe(checkbox_eventhandler, names=['value'])
box2.children[2].observe(topic_eventhandler, names=['value'])
box2.children[3].observe(filter_eventhandler, names=['value'])
box2.children[6].on_click(search_btn_eventhandler)


#display(box2)

[(35, 'science response models lymphocyte telehealth pandemic correlation deterministic emergency profiles', 1), (30, 'possibility spaces confined spring festival authors china province hubei ill', 1), (24, 'males optimistic emphasis smoking females rates iran fatalities death likelihood', 1), (20, 'school shanghai decided group closure transmission significantly march closures ncp', 1), (19, 'characteristic valuable parallel sign sample dataset provided antiviral societal long-term', 1), (28, 'inhibits fibrosis renal barriers students subsequent fluids rise plan men', 1), (33, 'structures genome architecture variations simulations evolutionary translated mode sars-cov-2 primary', 1), (34, 'reactions tuberculosis beneficial respiratory tweets coronavirus syndrome acute bacillus animal', 1), (42, 'called union way coronavirus cross-sectional early virus caused health disease', 1), (8, 'mass saudi arabia phases spatial iran humanitarian engagement gatherings relations', 2), (7, 'learning

In [59]:
fl.papers_dict['04cuk2cn']

{'title': 'Recombinant Tula hantavirus shows reduced fitness but is able to survive in the presence of a parental virus: analysis of consecutive passages in a cell culture',
 'abstract': {'Background': [],
  'Purpose': [],
  'Method': [],
  'Finding_Contribution': [(0,
    'Tula hantavirus carrying recombinant S RNA segment (recTULV) grew in a cell culture to the same titers as the original cell adapted variant but presented no real match to the parental virus.'),
   (1,
    'Our data showed that the lower competitiveness of recTULV could not be increased by pre-passaging in the cell culture.'),
   (2,
    'Nevertheless, the recombinant virus was able to survive in the presence of the parental virus during five consecutive passages.'),
   (3,
    'The observed survival time seems to be sufficient for transmission of newly formed recombinant hantaviruses in nature.')],
  'Other': [],
  'Whole': []},
 'is_covid': False,
 'ents': ['Tula hantavirus',
  'recombinant S RNA segment',
  'recTU

In [60]:
def get_query_results(classification_bool, aspect, query_text):
    classification = ''
    if(classification_bool):
        classification='covid_papers'
    else:
        classification='all_papers'
        
    topic_model = fl.topic_models[classification][aspect]
        
    results = []
    final_results = {}

    results = topic_model.search(query_text, case_sensitive=False)
    print(query_text)

    #interact(get_detailed_topics, topic_model=box.children[1])
    
    for result in results:
        uid = fl.uids[classification][result[1]]
        abstract_string = fl.abstract_dict_to_string(fl.papers_dict[uid]['abstract'], aspect)
        
        formated_abstract = abstract_string
        for word in query_text.split():
            #pattern = re.compile(word, re.IGNORECASE)
            #pattern.sub('<strong style="color: red">{}</strong>'.format(word), formated_abstract)
            formated_abstract = re.sub(word, '<strong style="color: red">{}</strong>'.format(word), formated_abstract, flags=re.IGNORECASE)
            #formated_abstract = pattern
        
        
        final_results[uid] = {'title': fl.papers_dict[uid]['title'], 'abstract': formated_abstract}
            
    return final_results

df = pandas.DataFrame()


box3 = widgets.Box(
    layout = box_layout,
    children=[
        widgets.Label(value='Search topic:'),
        widgets.Dropdown(
            options=fl.available_aspects,
            layout={'width': 'max-content'},
            description='Aspect:'
        ),
        widgets.Textarea(
            value='',
            placeholder='Query Sentence',
            description='Query:',
            disabled=False
        ),
        widgets.Checkbox(
            value=False,
            description='Covid-19 papers',
            disabled=False,
            indent=False,
            layout={'width': 'max-content'}
        ),
        
        widgets.Button(
            description='Search'
        ),
        
    ]
)

def btn_eventhandler(btn):
    clear_output(wait=True)
    display(box3)
    
    query_results = get_query_results(box3.children[3].value, box3.children[1].value, box3.children[2].value)

        
    display(Markdown(pandas.DataFrame.from_dict(query_results, orient='index').to_markdown()))

    
box3.children[4].on_click(btn_eventhandler)

#display(box)

In [61]:
box4 = widgets.Box(
    layout = box_layout,
    children=[
        widgets.Label(value='Search topic:'),
        widgets.Dropdown(
            options=fl.available_aspects,
            layout={'width': 'max-content'},
            description='Aspect:'
        ),
        widgets.Dropdown(
            options=['transmission','incubation','environmental stability', 'risk factor', 'vaccines', 'therapeutics', 'virus genetics', 'origin', 'evolution', 'medical care', 'non-pharmaceutical interventions', 'ethical consideration', 'social science consideration',
                        'diagnostics','surveillance'],
            layout={'width': 'max-content'},
            description='Query:'
        ),
        widgets.Checkbox(
            value=False,
            description='Covid-19 papers',
            disabled=False,
            indent=False,
            layout={'width': 'max-content'}
        ),
        
        widgets.Button(
            description='Search'
        ),
        
    ]
)

def btn_eventhandler(btn):
    clear_output(wait=True)
    display(box4)
    
    query_results = get_query_results(box4.children[3].value, box4.children[1].value, box4.children[2].value)

        
    display(Markdown(pandas.DataFrame.from_dict(query_results, orient='index').to_markdown()))

    
box4.children[4].on_click(btn_eventhandler)

In [62]:
def get_visualization(classification_bool, aspect):
    classification = ''
    if(classification_bool):
        classification='covid_papers'
    else:
        classification='all_papers'
        
    topic_model = fl.topic_models[classification][aspect]
    
    topic_model.visualize_documents(doc_topics=topic_model.get_doctopics())
    

box5 = widgets.Box(
    layout = box_layout,
    children=[
        widgets.Label(value='Visualize the documents in aspect:'),
        widgets.Dropdown(
            options=fl.available_aspects,
            layout={'width': 'max-content'},
            description='Aspect:'
        ),
        widgets.Checkbox(
            value=False,
            description='Covid-19 papers',
            disabled=False,
            indent=False,
            layout={'width': 'max-content'}
        ),
        
        widgets.Button(
            description='Search'
        ),
        
    ]
)

def btn_eventhandler(btn):
    clear_output(wait=True)
    display(box5)
    
    query_results = get_visualization(box5.children[2].value, box5.children[1].value)

    
box5.children[3].on_click(btn_eventhandler)

In [63]:
def display_abstract(uid):
    whole_text_dict = {}
    whole_text = "<h3>"+fl.papers_dict[uid]['title']+"</h3><div class='abstract'>"
    
    for i in range(len(RESEARCH_ASPECTS)):
        for sentence in fl.papers_dict[uid]['abstract'][RESEARCH_ASPECTS[i]]:
            whole_text_dict[sentence[0]] = ""
            for word in sentence[1].split():
                #whole_text_dict[sentence[0]] = "<span style='color:"+COLORS[i]+"'> "+sentence[1]+"</span>"
                whole_text_dict[sentence[0]]+= "<div class='tooltip' style='color:"+COLORS[i]+"'> "+word+"<span class='tooltiptext'>Tooltip text</span></div> "
            

    for key in sorted(whole_text_dict):
        whole_text+= whole_text_dict[key]
        
    whole_text += "</div>"

    caption = ""
    for i in range(len(RESEARCH_ASPECTS)):
        caption += "<div style='display: table-row'><div style='width: 20px;display: table-cell; height:10px; background-color:"+COLORS[i]+"'></div><div style='display: table-cell'>"+RESEARCH_ASPECTS[i]+"</div></div>"
    
    display(Markdown(whole_text))
    display(Markdown(caption))
    

box6 = widgets.Box(
    layout = box_layout,
    children=[
        widgets.Label(value='Visualize the abstract of:'),
        widgets.Dropdown(
            options=fl.papers_dict.keys(),
            layout={'width': 'max-content'},
            description='UID:'
        ),
        
        widgets.Button(
            description='Visualize'
        ),
        
    ]
)

def btn_eventhandler(btn):
    clear_output(wait=True)
    display(box6)
    
    query_results = display_abstract(box6.children[1].value)

    
box6.children[2].on_click(btn_eventhandler)

# RESEARCHER INTERFACE #

### Get topic representations ###

In [66]:
display(box1)

Box(children=(Label(value='Get topics for:'), Dropdown(description='Aspect:', index=4, layout=Layout(width='ma…

,Topics words,Number of papers
18,cases patients data number results china epidemic days confirmed model,1862
17,sars-cov-2 virus pandemic coronavirus infection spread health viral social using,1086
39,patients severe coronavirus respiratory sars-cov-2 clinical acute studies syndrome treatment,833
43,model number data population social growth epidemic distancing states measures,247
21,care healthcare health pandemic workers equipment ppe personal recommendations patients,213
40,anxiety staff health mental psychological social students quality stress medical,78
27,health public capacity countries response emergency bcg vaccination policy mitigation,65
38,mask cfr masks ventilator ventilation air filter fit patient ifr,26
8,del shelter los por pacientes las han una enfermedad febrero,5
34,hcq noisy l-ove covers customised centralised submission der randomised epistemonikos,4


In [69]:
fl.topic_models['covid_papers']['Whole'].get_docs(topic_ids=[13])

[('In this Editorial, the Editor‐in‐Chief Professor Miguel A. De la Rosa introduces the new members of the Editorial Board and discusses the effects of COVID‐19 on the journal.[Image: see text]',
  62,
  0.6456694893270462,
  13)]

In [77]:
fl.papers_dict['0xb02y2e']

'y55ymsjg'

In [ ]:
fl.uids['covid_papers'][62]

In [78]:
fl.papers_dict['y55ymsjg']

{'title': 'A warm welcome to our new editors',
 'abstract': {'Background': [],
  'Purpose': [(0,
    'In this Editorial, the Editor‐in‐Chief Professor Miguel A. De la Rosa introduces the new members of the Editorial Board and discusses the effects of COVID‐19 on the journal.')],
  'Method': [],
  'Finding_Contribution': [(1, '[Image: see text]')],
  'Other': [],
  'Whole': []},
 'is_covid': True,
 'ents': ['Editorial',
  'Editor‐in‐Chief Professor Miguel A. De la Rosa',
  'members',
  'Editorial Board',
  'effects',
  'COVID‐19',
  'journal',
  'Image']}

### Get documents in topic model ###

In [83]:
display(box2)

Box(children=(Label(value='Get topics for:', layout=Layout(width='max-content')), Dropdown(description='Aspect…

|          | title                                                                                                                                                                                                                                                                                                                                                                                                    | abstract                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |    score |
|:---------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------:|
| t60ewsen | Helmet Modification to PPE with 3D Printing During the COVID-19 Pandemic at Duke University Medical Center: A Novel Technique                                                                                                                                                                                                                                                                            | Abstract Care for patients during COVID-19 poses challenges that require the protection of staff with recommendations that health care workers wear at minimum, an N95 mask or equivalent while performing an aerosol-generating procedure with a face shield. The United States faces shortages of personal protective equipment, and surgeons who use loupes and headlights have difficulty using these in conjunction with face shields. Most arthroplasty surgeons use surgical helmet systems, but in the current pandemic, many hospitals have delayed elective arthroplasty surgeries and the helmet systems are going unused. As a result, the authors have begun retrofitting these arthroplasty helmets to serve as personal protective equipment (PPE). The purpose of this paper is to outline the conception, design, donning technique, and safety testing of these arthroplasty helmets being re-purposed as PPE.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | 0.932955 |
| duzk4nq5 | Reflexiones derivadas de la pandemia COVID-19                                                                                                                                                                                                                                                                                                                                                            | Resumen Cuando redactábamos las recomendaciones para el plan de contingencia conjunto entre la Sociedad Española de Enfermería Intensiva y Unidades Coronarias (SEEIUC) y la Sociedad Española de Medicina Intensiva, Crítica y Unidades Coronarias (SEMICYUC) los hechos iban por delante de las predicciones ante la magnitud de la pandemia del COVID-19. Socios y socias nos informaban de la falta de equipos de protección individual (EPI), la habilitación rápida de UCI improvisadas en zonas varias de hospitales y el reclutamiento de enfermeras para cubrir turnos. La falta de reconocimiento de la especialidad enfermera en cuidados críticos, englobada dentro la macroespecialidad de enfermería medicoquirúrgica, todavía sin desarrollar, ha puesto de relieve la necesidad urgente de aprender de los errores: especialidad, aumentar dotaciones en los equipos de enfermería y proteger el sistema sanitario público. Abstract While we were drafting the recommendations for the joint contingency plan between the Spanish Society of Intensive Care and Coronary Unit Nursing (SEEIUC) and the Spanish Society of Intensive and Critical Care Medicine and Coronary Units (SEMICYUC), predictions were overtaken by events with regard to the magnitude of the COVID-19 pandemic. Members informed us of the lack of personal protection equipment (PPE), the rapid provision of improvised ICUs in various hospital areas and the recruitment of nurses to cover shifts. The failure to recognise the specialty of critical care nursing, included in the macro-specialty of medical-surgical nursing and not yet developed, has highlighted the urgent need to learn from our mistakes: specialisation, increase the number of nurses in teams and protect the public health system.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.895487 |
| gz6frjfb | Institution of a Novel Process for N95 Respirator Disinfection with Vaporized Hydrogen Peroxide in the setting of the COVID-19 Pandemic at a Large Academic Medical Center                                                                                                                                                                                                                               | Abstract Personal protective equipment (PPE) has been an invaluable yet limited resource when it comes to protecting healthcare workers against infection during the COVID-19 pandemic. In the US, N95 respirator supply chains are severely strained and conservation strategies are needed. A multidisciplinary team at the Washington University School of Medicine, Barnes Jewish Hospital, and BJC Healthcare was formed to implement a program to disinfect N95 respirators. The process described extends the life of N95 respirators using vaporized hydrogen peroxide (VHP) disinfection and allows healthcare workers to retain their own N95 respirator across a large metropolitan health care system.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.878052 |
| 2gudro8v | The Rapid Response to the COVID-19 Pandemic by the Arthroplasty Divisions at Two Academic Referral Centers                                                                                                                                                                                                                                                                                               | Abstract The COVID-19 pandemic has created widespread changes across all of healthcare. The impacts on the delivery of orthopaedic services has been challenged as a result. In order to ensure and provide for adequate health care resources in terms of hospital capacity, personnel and personal protective equipment (PPE), service lines such as adult reconstruction and lower limb arthroplasty have stopped or substantially limited elective surgeries and have been forced to re-engineer care processes for a high-volume of patients. Herein, we summarize the similar approaches by two arthroplasty divisions in high volume academic referral centers in 1) the cessation of elective surgeries, 2) workforce restructuring, 3) phased delivery of outpatient and inpatient care, and 4) educational restructuring.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.868643 |
| qyh1vkgi | COVID-19 Guidance for Triage of Operations for Thoracic Malignancies: A Consensus Statement from Thoracic Surgery Outcomes Research Network                                                                                                                                                                                                                                                              | Abstract The extraordinary demands of managing the COVID-19 pandemic has disrupted the world’s ability to care for patients with thoracic malignancies. As a hospital’s COVID-19 population increases and hospital resources are depleted, the ability to provide surgical care is progressively restricted - forcing surgeons to prioritize among their cancer populations. Representatives from multiple cancer, surgical and research organizations have come together to provide a guide for triaging patients with thoracic malignancies, as the impact of COVID-19 evolves as each hospital.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.849489 |
| uc225mhn | COVID-19 Guidance for Triage of Operations for Thoracic Malignancies: A Consensus Statement from Thoracic Surgery Outcomes Research Network                                                                                                                                                                                                                                                              | Abstract The extraordinary demands of managing the COVID-19 pandemic has disrupted the world’s ability to care for patients with thoracic malignancies. As a hospital’s COVID-19 population increases and hospital resources are depleted, the ability to provide surgical care is progressively restricted - forcing surgeons to prioritize among their cancer populations. Representatives from multiple cancer, surgical and research organizations have come together to provide a guide for triaging patients with thoracic malignancies, as the impact of COVID-19 evolves as each hospital.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.849489 |
| qgr0e0gt | Elective surgery in the time of COVID-19                                                                                                                                                                                                                                                                                                                                                                 | Abstract The COVID-19 pandemic has placed a significant strain on the United States health care system, and frontline healthcare workers are rapidly altering their professional responsibilities to help meet hospital needs. In an effort to decrease disease transmission and conserve personal protective equipment (PPE), surgeons have witnessed one of the most dramatic changes in their practices with rapidly decreasing numbers of elective surgeries.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.845735 |
| t5rdljyk | Communication Skills in the Age of COVID-19                                                                                                                                                                                                                                                                                                                                                              | COVID-19 has placed enormous stress on hospitals and health care providers worldwide. Limitations in hospital capacity may result in difficult decisions in how life-sustaining technologies are allocated among patients. The authors of this essay provide guidance on how to communicate with patients who are seriously ill from COVID-19 infection.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.836863 |
| 713l5543 | Surgical decision-making and prioritization for cancer patients at the onset of the COVID-19 pandemic: A multidisciplinary approach                                                                                                                                                                                                                                                                      | Abstract In the midst of the coronavirus disease 2019 (COVID-19) pandemic, governmental agencies, state medical boards, and healthcare organizations have called for restricting “elective” operations to mitigate the risk of transmission of the virus amongst patients and healthcare providers and to preserve essential resources for potential regional surges of COVID patients. While the fear of delaying surgical care for many of our patients is deeply challenging for us as cancer care providers, we must balance our personal commitment to providing timely and appropriate oncologic care to our cancer patients with our societal responsibility to protect our patients (including those on whom we are operating), co-workers, trainees, families, and community, from undue risks of contracting and propagating COVID-19. Herein, we present guidelines for surgical decision-making and case prioritization developed among all adult disease specialties in the MD Anderson Cancer Center Departments of Surgical Oncology and Breast Surgical Oncology in Houston, Texas.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.828004 |
| f634kpug | Attending to the Emotional Well-Being of the Health Care Workforce in a New York City Health System During the COVID-19 Pandemic                                                                                                                                                                                                                                                                         | The COVID-19 pandemic has placed an enormous strain on health care workers, and its potential impact has implications for the physical and emotional well-being of the work force. As hospital systems run far over capacity, facing possible shortages of critical care medical resources and personal protective equipment as well as clinician deaths, the psychological stressors necessitate a strong well-being support model for staff. At the Mount Sinai Health System (MSHS) in New York City, health care workers have been heroically providing frontline care to COVID-19 patients while facing their own appropriate fears for their personal safety in the setting of contagion. This moral obligation cannot be burdened by unacceptable risks; the health system’s full support is required to address the needs of its workforce. In this Invited Commentary, the authors describe how an MSHS Employee, Faculty, and Trainee Crisis Support Task Force—created in early March 2020 and composed of behavioral health, human resources, and well-being leaders from across the health system—used a rapid needs assessment model to capture the concerns of the workforce related to the COVID-19 pandemic. The task force identified 3 priority areas central to promoting and maintaining the well-being of the entire MSHS workforce during the pandemic: meeting basic daily needs; enhancing communications for delivery of current, reliable, and reassuring messages; and developing robust psychosocial and mental health support options. Using a work group strategy, the task force operationalized the rollout of support initiatives for each priority area. Attending to the emotional well-being of health care workers has emerged as a central element in the MSHS COVID-19 response, which continues to be committed to the physical and emotional needs of a workforce that courageously faces this crisis.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | 0.821354 |
| rmn7qyrx | La gestión de la asistencia neurológica en tiempos de la pandemia de COVID-19                                                                                                                                                                                                                                                                                                                            | Abstract Introducción: La pandemia de Covid-19 ha supuesto la necesidad de tomar de decisiones para mantener la asistencia neurológica, sin precedentes. En este artículo se analizan esa toma de decisiones operativas. Desarrollo: Los autores refieren las fórmulas empleadas como son la realización de un plan de reorganización funcional, estrategias para la hospitalización y urgencias, la realización de consultas telefónicas para el mantenimiento de la atención neurológica, la asistencia en una área externa al hospital para pacientes preferentes, las decisiones sobre exploraciones complementarias y tratamientos intrahospitalarios periódicos o implantar un teléfono para la priorización de crisis epilépticas. Conclusión: A pesar de la situación de aislamiento, los servicios de neurología deben mantener la atención de sus pacientes, a través de distintas fórmulas operativas, que como cualquier elemento de gestión, deberán evaluarse. Abstract Introduction: The COVID-19 epidemic has led to the need for unprecedented decisions to be made to maintain the provision of neurological care. This article addresses operational decision-making during the epidemic. Development: We report the measures taken, including the preparation of a functional reorganisation plan, strategies for hospitalisation and emergency management, the use of telephone consultations to maintain neurological care, provision of care at a unit outside the hospital for priority patients, decisions about complementary testing and periodic in-hospital treatments, and the use of a specific telephone service to prioritise patients with epileptic seizures. Conclusion: Despite the situation of confinement, neurology departments must continue to provide patient care through different means of operation. Like all elements of management, these must be evaluated.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0.814622 |
| 7e8kb3ba | LA PSIQUIATRÍA EN LAS SECUELAS DE COVID-19                                                                                                                                                                                                                                                                                                                                                               | Resumen La pandemia por Covid-19 ha obligado a modificar sustancialmente la forma de trabajar de los profesionales de la salud mental y puede tener un impacto diferido sobre los pacientes. La Psiquiatría post-Covid-19 pondrá en valor la especialidad como parte de la medicina, los aspectos psicológicos de la práctica médica, la psiquiatría de enlace e interconsulta hospitalaria, la atención domiciliaria, y la atención ambulatoria virtual o telemática. Las consecuencias del confinamiento poblacional, el duelo por las muertes solitarias, y el impacto de la crisis sanitaria sobre los profesionales de la salud mental, desde los dispositivos hospitalarios hasta los comunitarios, de rehabilitación, y la atención primaria, centrarán nuestros esfuerzos durante el desescalado y el medio plazo. Es previsible un aumento de la demanda de atención psiquiátrica a medio y largo plazo y un impacto en la docencia e investigación en salud mental. Abstract The COVID-19 pandemic has forced mental health professionals to substantially change the way they work and may have a delayed impact on patients. The aftermath of COVID-19 will shine a light on certain aspects of psychiatry addressed in this article: psychiatry as a medical specialty, the psychological aspects of medical practice, liaison and consultative psychiatry, home hospitalization, and virtual or telemedicine outpatient care. The consequences of population lockdown, complicated grief over solitary deaths, and the impact of the health crisis on mental health professionals – from hospitals to community services, rehabilitation facilities, and primary care – will be the focus of our efforts during the period of lockdown easing and in the medium term. There will be a foreseeable increase in demand for psychiatric care in the medium and long term along with an impact on mental health education and research.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.779264 |
| 0btjz1lp | Bracing for impact with new 4R’s in the COVID-19 pandemic- a provincial thoracic radiation oncology consensus                                                                                                                                                                                                                                                                                            | Abstract As COVID-19 pandemic continues to explode, cancer centers worldwide are trying to adapt and are struggling with this constantly changing scenario. Intending to ensure patient safety and deliver quality care, we sought consensus on the preferred thoracic radiation regimen in a Canadian province with 4 new R’s of COVID era.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0.763312 |
| mhdmy8kq | DECLARACIÓN DE CONSENSO EN MEDICINA CRÍTICA PARA LA ATENCIÓN MULTIDISCIPLINARIA DEL PACIENTE CON SOSPECHA O CONFIRMACIÓN DIAGNÓSTICA DE COVID-19                                                                                                                                                                                                                                                         | Resumen El comportamiento de la infección por SARS-CoV-2 obligó a la Organización Mundial de la Salud a emitir una convocatoria global de activación de mecanismos de emergencia para atender la crisis de salud pública latente. Las Unidades de Cuidados Intensivos son uno de los principales recursos de los sistemas sanitarios dada la tasa de neumonías complicadas que presentan los pacientes infectados. En respuesta a los distintos lineamientos y diferentes niveles de evidencia de la información disponible, la Asociación Colombiana de Medicina crítica y Cuidados Intensivos (AMCI), convocó un equipo multidisciplinario de expertos en medicina crítica para establecer una declaratoria de consenso de buena práctica clínica para la atención de pacientes con COVID-19. Su objetivo es facilitar y estandarizar la toma de decisiones en los aspectos más relevantes desde la organización administrativa de las áreas de atención hasta el abordaje clínico del paciente, teniendo en cuenta la seguridad del personal sanitario, la infraestructura y recursos con los que cuenta el país para responder a la emergencia. Este documento está sujeto a la evolución del conocimiento y a los resultados de investigaciones en curso. Abstract The behaviour of the SARS-CoV-2 infection has forced World Health Organisation to call for the global activation of emergency and contingency mechanisms to respond to the world-wide public health crisis. Intensive Care Units are one of the main health care resources in this situation due to the number of expected cases of severe pneumonia from infected patients. Different guidelines and different levels of evidence from the available information have motivated Asociación Colombiana de Medicina crítica y Cuidados Intensivos (AMCI) to assemble a multidisciplinary team of experts in critical care medicine to prepare a consensus statement of good clinical practice for the care of patients with COVID-19. Its aim is to simplify and standardise decision-making in the most relevant aspects: from the administrative organisation of the care facilities, to the clinical approach of patients, taking into account the safety of health care professionals, and the infrastructure and resources available in Colombia to respond to the emergency. This document will be adapted to any new knowledge and results of on-going research.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0.757854 |
| 91weyqmm | Telemedicine in the Time of Coronavirus                                                                                                                                                                                                                                                                                                                                                                  | ABSTRACT Within weeks, COVID-19 has transformed our practice of palliative care and clinical medicine as we know it. Telemedicine has emerged as a critical technology to bring medical care to patients while attempting to reduce the transmission of COVID-19 among patients, families, and clinicians. It is also increasingly necessary to preserve scarce resources like personal protective equipment. In this article, we share just-in-time tips to support palliative care clinicians and program leaders in providing the best care possible by telemedicine. These quick, practical tips cover telemedicine set-up, patient considerations, and clinician considerations. Next steps include ensuring equitable access to affordable telemedicine technology for vulnerable populations through creative solutions and financing, and dedicated attention to telemedicine evaluation and quality improvement.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.73917  |
| ce1us64t | Goals of Care in a Pandemic: Our Experience and Recommendations                                                                                                                                                                                                                                                                                                                                          | Abstract: We propose that the palliative care team response will occur in two waves: first communication and second symptom management. Our experience with discussing goals of care with the family of a COVID positive patient highlighted some expected and unexpected challenges. We describe these challenges along with recommendations for approaching these conversations. We also propose a framework for proactively mobilizing the palliative care workforce to aggressively address goals of care in all patients, with the aim of reducing the need for rationing of resources.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0.738103 |
| 21vrjals | ATENCIÓN DE LA URGENCIA QUIRÚRGICA DURANTE LA PANDEMIA COVID-19. RECOMENDACIONES DE LA ASOCIACIÓN ESPAÑOLA DE CIRUJANOS                                                                                                                                                                                                                                                                                  | RESUMEN La infección por el nuevo coronavirus SARS-CoV-2 (enfermedad por coronavirus 2019 -covid-19-) ha determinado la necesidad de la reorganización de muchos centros hospitalarios en el mundo. España, como uno de los epicentros de la enfermedad, ha debido asumir cambios en la práctica totalidad de su territorio. Sin embargo, y desde el inicio de la pandemia, en todos los centros que atienden urgencias quirúrgicas ha sido necesario el mantenimiento de su cobertura, aunque igualmente ha sido inevitable introducir directrices especiales de ajuste al nuevo escenario que permitan el mantenimiento de la excelencia en la calidad asistencial. Este documento desarrolla una serie de indicaciones generales para la cirugía de urgencias y la atención al politraumatizado, desarrolladas desde la literatura disponible y consensuadas por un subgrupo de profesionales desde el grupo general Cirugía-AEC-COVID19. Estas medidas van encaminadas a contemplar un riguroso control de la exposición en pacientes y profesionales, a tener en cuenta las implicaciones de la pandemia sobre diferentes escenarios perioperatorios relacionados con la urgencia y a una adaptación ajustada a la situación del centro en relación con la atención a pacientes infectados. ABSTRACT New coronavirus SARS-CoV-2 infection (coronavirus disease 2019 -covid-19-) has determined the necessity of reorganization in many centers all over the world. Spain, as an epicenter of the disease, has been forced to assume health policy changes in all the territory. However, and from the beginning of the pandemic, every center attending surgical urgencies had to guarantee the continuous coverage adopting correct measures to maintain the excellence of quality of care. This document resumes general guidelines for emergency surgery and trauma care, obtained from the available bibliography and evaluated by a subgroup of professionals designated from the general group of investigators Cirugía-AEC-COVID19 from the Spanish Association of Surgeons, directed to minimize professional exposure, to contemplate pandemic implications over different urgent perioperative scenarios and to adjust decision making to the occupational pressure caused by covid-19 patients.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.736508 |
| h5iew8oo | Atención al ictus agudo durante la pandemia por COVID-19. Recomendaciones Plan Ictus Madrid                                                                                                                                                                                                                                                                                                              | RESUMEN Introducción: La pandemia por COVID-19 ha obligado a una reorganización de los sistemas sanitarios y una saturación excepcional de sus recursos. En este contexto es vital asegurar la atención al ictus agudo y optimizar los procesos asistenciales del código ictus para reducir el riesgo de contagios y racionalizar el uso de recursos hospitalarios. Para ello desde el Grupo multidisciplinar Ictus Madrid proponemos una serie de recomendaciones. Métodos: Revisión bibliográfica no sistemática de las publicaciones disponibles con los términos “stroke” y “covid-19” o “coronavirus” o “SARS-COV-2”, así como otras conocidas por los autores. En base a ésta se redacta un documento de recomendaciones que es sometido a consenso por el Grupo multidisciplinar Ictus Madrid y su Comité de Neurología. Resultados: Las recomendaciones se estructuran en cinco líneas fundamentales: (1) Coordinar la actuación para garantizar el acceso a la asistencia hospitalaria de los pacientes con ictus, (2) Reconocer a los pacientes con ictus potencialmente infectados por COVID-19, (3) Organización adecuada para garantizar la protección de los profesionales sanitarios frente al riesgo de contagio por COVID-19, (4) la realización de Neuroimagen y otros procedimientos que conlleven contactos de riesgo de infección COVID-19 hay que procurar reducirlos y asegurar la protección, y (5) alta y seguimiento seguros procurando optimizar la ocupación hospitalaria. Resumimos el procedimiento de forma esquemática con el acrónimo CORONA (COordinar, Reconocer, Organizar, Neuroimagen, Alta). Conclusiones: Estas recomendaciones pueden servir de apoyo para la organización del sistema sanitario en la atención al ictus agudo y la optimización de sus recursos, garantizando la protección de sus profesionales. ABSTRACT Background: The COVID-19 pandemic has forced a reorganization of healthcare systems and an exceptional saturation of their resources. In this context, it is vital to ensure acute stroke care and optimize the care processes of the stroke code to reduce the risk of contagion and rationalize the use of hospital resources. To do this, the Ictus Madrid multidisciplinary group proposes a series of recommendations. Methods: Non-systematic bibliographic review of the available publications with the terms “stroke” and “covid-19” or “coronavirus” or “SARS-COV-2”, as well as other already known for the authors. We provide a document of recommendations as a result of the consensus of the Ictus Madrid multidisciplinary group and its Neurology Committee. Results: Our recommendations are structured on five lines: (1) Coordinate to guarantee the access to hospital care for stroke patients, (2) Recognize potentially COVID-19 infected stroke patients, (3) Organize to ensure the protection of healthcare professionals from COVID-19 infections, (4) Neuroimaging and other procedures potentially associated to risks for COVID-19 infection should be reduced and secured to avoid contagion, and (5) At home as soon as possible and supported follow-up to optimize hospital occupancy. The procedure is shown summarized under the acronym CORONA (Coordinate, Recognize, Organize, Neuroimaging, At home). Conclusions: These recommendations can support the organization of healthcare services for acute stroke care and the optimization of their resources, guaranteeing the protection of healthcare professionals.                                                                                                                                                                                                                                                                                                                                                                                     | 0.733548 |
| cxfsf1ic | Telehealth for global emergencies: Implications for coronavirus disease 2019 (COVID-19)                                                                                                                                                                                                                                                                                                                  | The current coronavirus (COVID-19) pandemic is again reminding us of the importance of using telehealth to deliver care, especially as means of reducing the risk of cross-contamination caused by close contact. For telehealth to be effective as part of an emergency response it first needs to become a routinely used part of our health system. Hence, it is time to step back and ask why telehealth is not mainstreamed. In this article, we highlight key requirements for this to occur. Strategies to ensure that telehealth is used regularly in acute, post-acute and emergency situations, alongside conventional service delivery methods, include flexible funding arrangements, training and accrediting our health workforce. Telehealth uptake also requires a significant change in management effort and the redesign of existing models of care. Implementing telehealth proactively rather than reactively is more likely to generate greater benefits in the long-term, and help with the everyday (and emergency) challenges in healthcare.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.73165  |
| jjjk4z9a | Physiotherapy management for COVID-19 in the acute hospital setting: clinical practice recommendations                                                                                                                                                                                                                                                                                                   | Abstract This document outlines recommendations for physiotherapy management for COVID-19 in the acute hospital setting. It includes: recommendations for physiotherapy workforce planning and preparation; a screening tool for determining requirement for physiotherapy; and recommendations for the selection of physiotherapy treatments and personal protective equipment. It is intended for use by physiotherapists and other relevant stakeholders in the acute care setting caring for adult patients with confirmed or suspected COVID-19.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.731387 |
| 3vwi5myc | Perioperative Management of Patients Infected with the Novel Coronavirus: Recommendation from the Joint Task Force of the Chinese Society of Anesthesiology and the Chinese Association of Anesthesiologists                                                                                                                                                                                             | The outbreak of the new Coronavirus disease, COVID-19, has been involved in 77,262 cases in China as well as in 27 other countries as of February 24, 2020. Because the virus is novel to human beings, and there is no vaccine yet available, every individual is susceptible and can become infected. Healthcare workers are at high risk, and unfortunately, more than 3,000 healthcare workers in China have been infected. Anesthesiologists are among healthcare workers who are at an even higher risk of becoming infected because of their close contact with infected patients and high potential of exposure to respiratory droplets or aerosol from their patients’ airways. In order to provide healthcare workers with updated recommendations on the management of patients in the perioperative setting as well as for emergency airway management outside of the operating room, the two largest anesthesia societies, the Chinese Society of Anesthesiology (CSA) and the Chinese Association of Anesthesiologists (CAA) have formed a task force to produce the recommendations. The task force hopes to help healthcare workers, particularly anesthesiologists, optimize the care of their patients and protect patients, healthcare workers, and the public from becoming infected. The recommendations were created mainly based on the practice and experience of anesthesiologists who provide care to patients in China. Therefore, adoption of these recommendations outside of China must be done with caution, and the local environment, culture, uniqueness of the healthcare system, and patients’ needs should be considered. The task force will continuously update the recommendations and incorporate new information in future versions.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0.719589 |
| qskqikk9 | Rapid De-Escalation and Triaging Patients in Community-Based Palliative Care                                                                                                                                                                                                                                                                                                                             | Abstract Context The COVID-19 pandemic created a rapid and unprecedented shift in our medical system. Medical providers, teams, and organizations have needed to shift their visits away from face-to-face visits and toward telehealth (both by phone and through video). Palliative care teams who practice in the community setting are faced with a difficult task: How do we actively triage the most urgent visits while keeping our vulnerable patients safe from the pandemic? Measures The following are recommendations created by the Palo Alto Medical Foundation Palliative Care and Support Services team to help triage and coordinate for timely, safe, and effective palliative care in the community and outpatient setting during the ongoing COVID-19 pandemic. Patients are initially triaged based on location followed by acuity. Interdisciplinary care is implemented using strict infection control guidelines in the setting of limited personal protective equipment (PPE) resources. We implement thorough screening for COVID-19 symptoms at multiple levels before a patient is seen by a designated provider. Conclusions/Lessons Learned We recommend active triaging, communication, frequent screening for COVID-19 symptoms for palliative care patients been evaluated in the community setting. An understanding of infection risk, mutual consent between designated providers, patients, and their families are crucial to maintaining safety while delivering community-based palliative care during the COVID-19 pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.71844  |
| vzsdvrwo | Rewriting the rules for care of MDS and AML patients in the time of COVID-19                                                                                                                                                                                                                                                                                                                             | Abstract The care of patients with myelodysplastic syndromes (MDS) and acute myeloid leukemia (AML) has been radically altered by COVID-19, especially in New York City, the epicenter of the pandemic. Here we summarize how telemedicine, virtual visits, delayed transfusions, and chemotherapy, preferably selecting self-administered medications and visits by home healthcare workers, are employed to minimize exposure of our high-risk population of patients to the virus. The unique challenges of transplants during the pandemic and the consequences of an abrupt halt in all non-essential research activities are described. Not all the changes forced by COVID-19 are detrimental.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.716035 |
| 2u379fh3 | Clinical and surgical consequences of the COVID-19 pandemic for patients with pediatric urological problems Statement of the EAU guidelines panel for paediatric urology, March 30 2020                                                                                                                                                                                                                  | Summary The COVID-19-pandemic forces hospitals to reorganize into a dual patient flow system. Healthcare professionals are forced to make decisions in patient prioritization throughout specialties. Most pediatric urology pathologies do not require immediate or urgent care, however, delay may compromise future renal function or fertility. Contact with patients and parents, either physical in safe conditions or by (video)telephone must continue. The Paediatric-Urology-Guidelines-panel of the EAU proposes recommendations on prioritization of care. Pediatric-Urology program directors must ensure education, safety and attention for mental health of staff. Upon resumption of care, adequate prioritization must ensure minimal impact on outcome.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | 0.715943 |
| 0f7cgukn | An Epidemic in the Midst of a Pandemic: Opioid Use Disorder and COVID-19                                                                                                                                                                                                                                                                                                                                 | The COVID-19 pandemic is a particularly grave risk to the millions of Americans with opioid use disorder, who—already vulnerable and marginalized—are heavily dependent on face-to-face health care delivery. These authors propose rapid and coordinated action on the part of clinicians and policymakers to mitigate risks of disrupted care for these patients.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.714453 |
| wxfprt12 | Surgery in COVID-19 patients: operational directives                                                                                                                                                                                                                                                                                                                                                     | The current COVID-19 pandemic underlines the importance of a mindful utilization of financial and human resources. Preserving resources and manpower is paramount in healthcare. It is important to ensure the ability of surgeons and specialized professionals to function through the pandemic. A conscious effort should be made to minimize infection in this sector. A high mortality rate within this group would be detrimental. This manuscript is the result of a collaboration between the major Italian surgical and anesthesiologic societies: ACOI, SIC, SICUT, SICO, SICG, SIFIPAC, SICE, and SIAARTI. We aim to describe recommended clinical pathways for COVID-19-positive patients requiring acute non-deferrable surgical care. All hospitals should organize dedicated protocols and workforce training as part of the effort to face the current pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | 0.70785  |
| q4u5t8zs | Head and neck oncology during the COVID-19 pandemic: Reconsidering traditional treatment paradigms in light of new surgical and other multilevel risks                                                                                                                                                                                                                                                   | Abstract The COVID-19 pandemic demands reassessment of head and neck oncology treatment paradigms. Head and neck cancer (HNC) patients are generally at high-risk for COVID-19 infection and severe adverse outcomes. Further, there are new, multilevel COVID-19-specific risks to patients, surgeons, health care workers (HCWs), institutions and society. Urgent guidance in the delivery of safe, quality head and neck oncologic care is needed. Novel barriers to safe HNC surgery include: 1) imperfect presurgical screening for COVID-19; 2) prolonged SARS-CoV-2 aerosolization; 3) occurrence of multiple, potentially lengthy, aerosol generating procedures (AGPs) within a single surgery; 4) potential incompatibility of enhanced personal protective equipment (PPE) with routine operative equipment; 5) existential or anticipated PPE shortages. Additionally, novel, COVID-19-specific multilevel risks to HNC patients, HCWs and institutions, and society include: use of immunosuppressive therapy, nosocomial COVID-19 transmission, institutional COVID-19 outbreaks, and, at some locations, societal resource deficiencies requiring health care rationing. Traditional head and neck oncology doctrines require reassessment given the extraordinary COVID-19-specific risks of surgery. Emergent, comprehensive management of these novel, multilevel surgical risks are needed. Until these risks are managed, we temporarily favor nonsurgical therapy over surgery for most mucosal squamous cell carcinomas, wherein surgery and nonsurgical therapy are both first-line options. Where surgery is traditionally preferred, we recommend multidisciplinary evaluation of multilevel surgical-risks, discussion of possible alternative nonsurgical therapies and shared-decision-making with the patient. Where surgery remains indicated, we recommend judicious preoperative planning and development of COVID-19-specific perioperative protocols to maximize the safety and quality of surgical and oncologic care.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | 0.703058 |
| sjflcgjz | Embracing telemedicine into your otolaryngology practice amid the COVID-19 crisis: An invited commentary                                                                                                                                                                                                                                                                                                 | Abstract The COVID-19 pandemic has quickly and radically altered how Otolaryngologists provide patient care in the outpatient setting. Continuity of care with established patients as well as establishment of a professional relationship with new patients is challenging during this Public Health Emergency (PHE). Many geographic areas are under “stay at home” or “shelter in place” directives from state and local governments to avoid COVID-19 exposure risks. Medicare has recently allowed “broad flexibilities to furnish services using remote communications technology to avoid exposure risks to health care providers, patients, and the community. ” [1] The implementation of telemedicine, or virtual, services, will help the Otolaryngologists provide needed care to patients while mitigating the clinical and financial impact of the pandemic. The significant coding and billing issues related to implementing telemedicine services are discussed to promote acceptance of this technology by the practicing Otolaryngologist. Of particular importance, outpatient visit Current Procedural Terminology® codes (99201-99215) may be used for telehealth visits performed in real-time audio and video.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 0.696126 |
| 0di2kt7z | Management of neurological care during the COVID-19 pandemic                                                                                                                                                                                                                                                                                                                                             | Abstract Introduction The COVID-19 epidemic has led to the need for unprecedented decisions to be made to maintain the provision of neurological care. This article addresses operational decision-making during the epidemic. Development We report the measures taken, including the preparation of a functional reorganisation plan, strategies for hospitalisation and emergency management, the use of telephone consultations to maintain neurological care, provision of care at a unit outside the hospital for priority patients, decisions about complementary testing and periodic in-hospital treatments, and the use of a specific telephone service to prioritise patients with epileptic seizures. Conclusion Despite the situation of confinement, neurology departments must continue to provide patient care through different means of operation. Like all elements of management, these must be evaluated.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0.679239 |
| slp6uuzx | Recomendaciones de consenso respecto al soporte respiratorio no invasivo en el paciente adulto con insuficiencia respiratoria aguda secundaria a infección por SARS-CoV-2*                                                                                                                                                                                                                               | Resumen La enfermedad por coronavirus 2019 (COVID-19) es una infección del tracto respiratorio causada por un nuevo coronavirus emergente, que se reconoció por primera vez en Wuhan, China, en diciembre de 2019. Actualmente, la Organización Mundial de la Salud (OMS) ha definido la infección como pandemia y existe una situación de emergencia sanitaria y social para el manejo de esta nueva infección. Mientras que la mayoría de las personas con COVID-19 desarrollan solo una enfermedad leve o no complicada, aproximadamente el 14% desarrolla una enfermedad grave que requiere hospitalización y oxígeno, y el 5% puede requerir ingreso en una Unidad de Cuidados Intensivos (1). En casos severos, COVID-19 puede complicarse por el síndrome de dificultad respiratoria aguda (SDRA), sepsis y shock séptico y fracaso multiorgánico. Este documento de consenso se ha preparado sobre directrices basadas en evidencia desarrolladas por un panel multidisciplinario de profesionales médicos de cuatro sociedades científicas españolas (Sociedad Española de Medicina Intensiva y Unidades Coronarias [SEMICYUC], Sociedad Española de Neumología y Cirugía Torácica [SEPAR], Sociedad Española de Urgencias y Emergencias [SEMES], Sociedad Española de Anestesiología, Reanimación y Terapéutica del Dolor [SEDAR]) con experiencia en el manejo clínico de pacientes con COVID-19 y otras infecciones virales, incluido el SARS, así como en sepsis y SDRA. El documento proporciona recomendaciones clínicas para el soporte respiratorio no invasivo (ventilación no invasiva, oxigenoterapia de alto flujo con cánula nasal) en cualquier paciente con presentación sospechada o confirmada de COVID-19 con insuficiencia respiratoria aguda. Esta guía de consenso debe servir como base para una atención optimizada y garantizar la mejor posibilidad de supervivencia, así como permitir una comparación fiable de las futuras intervenciones terapéuticas de investigación que formen parte de futuros estudios observacionales o de ensayos clínicos. Abstract Coronavirus disease 2019 (COVID-19) is a respiratory tract infection caused by a newly emergent coronavirus, that was first recognized in Wuhan, China, in December 2019. Currently, the World Health Organization (WHO) has defined the infection as a global pandemic and there is a health and social emergency for the management of this new infection. While most people with COVID-19 develop only mild or uncomplicated illness, approximately 14% develop severe disease that requires hospitalization and oxygen support, and 5% require admission to an intensive care unit (1). In severe cases, COVID-19 can be complicated by the acute respiratory distress syndrome (ARDS), sepsis and septic shock, and multiorgan failure. This consensus document has been prepared on evidence-informed guidelines developed by a multidisciplinary panel of health care providers from four Spanish scientific societies (Spanish Society of Intensive Care Medicine [SEMICYUC], Spanish Society of Pulmonologists [SEPAR], Spanish Society of Emergency [SEMES], Spanish Society of Anesthesiology, Reanimation, and Pain [SEDAR]) with experience in the clinical management of patients with COVID-19 and other viral infections, including SARS, as well as sepsis and ARDS. The document provides clinical recommendations for the noninvasive respiratory support (noninvasive ventilation, high flow oxygen therapy with nasal cannula) in any patient with suspected or confirmed presentation of COVID-19 with acute respiratory failure. This consensus guidance should serve as a foundation for optimized supportive care to ensure the best possible chance for survival and to allow for reliable comparison of investigational therapeutic interventions as part of randomized controlled trials. | 0.67871  |
| urxkhg5h | Recomendaciones de consenso respecto al soporte respiratorio no invasivo en el paciente adulto con insuficiencia respiratoria aguda secundaria a infección por SARS-CoV-2                                                                                                                                                                                                                                | Resumen La enfermedad por coronavirus 2019 (COVID-19) es una infección del tracto respiratorio causada por un nuevo coronavirus emergente que se reconoció por primera vez en Wuhan, China, en diciembre de 2019. Actualmente la Organización Mundial de la Salud (OMS) ha definido la infección como pandemia y existe una situación de emergencia sanitaria y social para el manejo de esta nueva infección. Mientras que la mayoría de las personas con COVID-19 desarrollan solo una enfermedad leve o no complicada, aproximadamente el 14% desarrollan una enfermedad grave que requiere hospitalización y oxígeno, y el 5% pueden requerir ingreso en una Unidad de Cuidados Intensivos. En casos severos, COVID-19 puede complicarse por el síndrome de dificultad respiratoria aguda (SDRA), sepsis y shock séptico y fracaso multiorgánico. Este documento de consenso se ha preparado sobre directrices basadas en evidencia desarrolladas por un panel multidisciplinario de profesionales médicos de cuatro sociedades científicas españolas (Sociedad Española de Medicina Intensiva y Unidades Coronarias [SEMICYUC], Sociedad Española de Neumología y Cirugía Torácica [SEPAR], Sociedad Española de Urgencias y Emergencias [SEMES], Sociedad Española de Anestesiología, Reanimación y Terapéutica del Dolor [SEDAR]) con experiencia en el manejo clínico de pacientes con COVID-19 y otras infecciones virales, incluido el SARS, así como en sepsis y SDRA. El documento proporciona recomendaciones clínicas para el soporte respiratorio no invasivo (ventilación no invasiva, oxigenoterapia de alto flujo con cánula nasal) en cualquier paciente con presentación sospechada o confirmada de COVID-19 con insuficiencia respiratoria aguda. Esta guía de consenso debe servir como base para una atención optimizada y garantizar la mejor posibilidad de supervivencia, así como permitir una comparación fiable de las futuras intervenciones terapéuticas de investigación que formen parte de futuros estudios observacionales o de ensayos clínicos. Abstract Coronavirus disease 2019 (COVID-19) is a respiratory tract infection caused by a newly emergent coronavirus, that was first recognized in Wuhan, China, in December 2019. Currently, the World Health Organization (WHO) has defined the infection as a global pandemic and there is a health and social emergency for the management of this new infection. While most people with COVID-19 develop only mild or uncomplicated illness, approximately 14% develop severe disease that requires hospitalization and oxygen support, and 5% require admission to an intensive care unit. In severe cases, COVID-19 can be complicated by the acute respiratory distress syndrome (ARDS), sepsis and septic shock, and multiorgan failure. This consensus document has been prepared on evidence-informed guidelines developed by a multidisciplinary panel of health care providers from four Spanish scientific societies (Spanish Society of Intensive Care Medicine [SEMICYUC], Spanish Society of Pulmonologists [SEPAR], Spanish Society of Emergency [SEMES], Spanish Society of Anesthesiology, Reanimation, and Pain [SEDAR]) with experience in the clinical management of patients with COVID-19 and other viral infections, including SARS, as well as sepsis and ARDS. The document provides clinical recommendations for the noninvasive respiratory support (noninvasive ventilation, high flow oxygen therapy with nasal cannula) in any patient with suspected or confirmed presentation of COVID-19 with acute respiratory failure. This consensus guidance should serve as a foundation for optimized supportive care to ensure the best possible chance for survival and to allow for reliable comparison of investigational therapeutic interventions as part of randomized controlled trials.         | 0.676677 |
| nlv1za6u | Which Melbourne metropolitan areas are vulnerable to COVID-19 based on age, disability and access to health services? Using spatial analysis to identify service gaps and inform delivery                                                                                                                                                                                                                | Abstract Ageing adults (65+) with disability are especially vulnerable to COVID-19 and upon contracting, are a cohort most likely to require palliative care. Therefore, it is very important that health services - particularly health services providing palliative care - are proximately available. Treating the Melbourne metropolitan area as a case-study, a spatial analysis was conducted to clarify priority areas with a significantly high percentage and number of ageing adults (65+) with disability, and high barriers to accessing primary health services. After, travel times from priority areas to (i) palliative medicine, and (ii) hospital services were calculated. The geographic dispersion of areas with people vulnerable to COVID-19 with poor access to palliative care and health services are clarified. Unique methods of health service delivery are required to ensure that vulnerable populations in under-serviced metropolitan areas receive prompt and adequate care. The spatial methodology employed can be implemented in different contexts to support evidence-based COVID-19 and pandemic palliative care service decisions.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 0.669411 |
| x2ct9w1q | Recommendations from the CSO-HNS taskforce on performance of tracheotomy during the COVID-19 pandemic                                                                                                                                                                                                                                                                                                    | INTRODUCTION: The performance of tracheotomy is a common procedural request by critical care departments to the surgical services of general surgery, thoracic surgery and otolaryngology - head & neck surgery. A Canadian Society of Otolaryngology – Head & Neck Surgery (CSO-HNS) task force was convened with multi-specialty involvement from otolaryngology-head & neck surgery, general surgery, critical care and anesthesiology to develop a set of recommendations for the performance of tracheotomies during the COVID-19 pandemic. MAIN BODY: The tracheotomy procedure is highly aerosol generating and directly exposes the entire surgical team to the viral aerosol plume and secretions, thereby increasing the risk of transmission to healthcare providers. As such, we believe extended endotracheal intubation should be the standard of care for the entire duration of ventilation in the vast majority of patients. Pre-operative COVID-19 testing is highly recommended for any non-emergent procedure. CONCLUSION: The set of recommendations in this document highlight the importance of avoiding tracheotomy procedures in patients who are COVID-19 positive if at all possible. Recommendations for appropriate PPE and environment are made for COVID-19 positive, negative and unknown patients requiring consideration of tracheotomy. The safety of healthcare professionals who care for ill patients and who keep critical infrastructure operating is paramount.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.668418 |
| kjx03hju | Rapid Implementation of Inpatient Telepalliative Medicine Consultations during COVID-19 Pandemic                                                                                                                                                                                                                                                                                                         | Abstract As COVID-19 cases increase throughout the country and healthcare systems grapple with the need to decrease provider exposure and minimize personal protective equipment (PPE) use while maintaining high quality patient care, our specialty is called upon to consider new methods of delivering inpatient palliative care. Telepalliative medicine has been used to great effect in outpatient and home-based palliative care, but has had fewer applications in the inpatient setting. As we plan for decreased provider availability due to quarantine and redeployment and seek to reach increasingly isolated hospitalized patients in the face of COVID-19, the need for telepalliative medicine in the inpatient setting is now clear. We describe our rapid and ongoing implementation of telepalliative medicine consultation for our inpatient palliative care teams and discuss lessons learned and recommendations for programs considering similar care models.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 0.665442 |
| mhblodgj | Covid-19: UK could delay non-urgent care and call doctors back from leave and retirement                                                                                                                                                                                                                                                                                                                 | Healthcare staff on leave and those who have retired could be called “back to duty,” and non-urgent care could be delayed, as doctors are forced to prioritise dealing with covid-19, the UK government’s action plan lays out. The government will also implement a “distribution strategy for the UK’s stockpiles of key medicines and equipment (e.g. protective clothing),” the document said. However, the plan did not include details of how or when these measures would be …                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.657435 |
| qc4ublel | Risks of viral contamination in healthcare professionals during laparoscopy in the Covid-19 pandemic                                                                                                                                                                                                                                                                                                     | Abstract The Covid-19 pandemic has markedly changed our practices. This article analyses the risks of contamination among healthcare professionals (HCPs) during laparoscopic surgery on patients with Covid-19. Harmful effects of aerosols from a pneumoperitoneum with the virus present have not yet been quantified. Measures for the protection of HCPs are an extrapolation of those taken during other epidemics. They must still be mandatory to minimise the risk of viral contamination. Protection measures include personal protection equipment for HCPs, adaptation of surgical technique (method for obtaining pneumoperitoneum, filters, preferred intracorporeal anastomosis, precautions during the exsufflation of the pneumoperitoneum), and organisation of the operating room.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.653158 |
| c97myb3u | Educating Surgeons to Educate Patients About the COVID-19 Pandemic                                                                                                                                                                                                                                                                                                                                       | Abstract The spring of 2020 has been a trying time for the global medical community as it has faced the latest pandemic, COVID-19. This contagious and lethal virus has impacted patients and health care workers alike. Elective surgeries have been suspended, and the very core of our health care system is being strained. The following brief communication reviews pertinent details about the virus, delaying elective surgeries, and what patients can do during this time. The goal is to disseminate factual data that surgeons can then use to educate their patients.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.638127 |
| yk6dn1yf | Creation of a vascular surgical Hub responding to the COVID-19 emergency: the italian USL Toscana Centro model                                                                                                                                                                                                                                                                                           | Abstract Spreading of the COVID-19 pandemic in Italy forced health facilities to drastically change their organization to face the overwhelming number of infected patients needing hospitalization. The USL (Unità Sanitaria Locale) Toscana Centro adopted a protocol of reorganization of the Vascular Surgery Unit during the first difficult weeks of the epidemic, consisting in the creation of a Vascular Hub for urgent cases with a profound reorganization of activities, wards, surgical operators, operating blocks and ICU beds. The aim of this paper is to share the protocol with all the vascular community, hoping to help other institutions to face the emergency during the hard weeks coming.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.636189 |
| x0qanojj | Coping with COVID: How a Research Team Learned To Stay Engaged in This Time of Physical Distancing                                                                                                                                                                                                                                                                                                       | Physical distancing imposed by the COVID-19 pandemic has led to alterations in routines and new responsibilities for much of the research community. We provide some tips for how research teams can cope with physical distancing, some of which require a change in how we define productivity. Importantly, we need to maintain and strengthen social connections in this time when we can’t be physically together.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 0.633004 |
| tq1sk01g | Operating during the COVID-19 pandemic: How to reduce medical error                                                                                                                                                                                                                                                                                                                                      | Abstract Our professional and private lives changed on March 11 2020 when the coronavirus disease 2019 (COVID-19) was declared a pandemic by the WHO. By March 16, surgical training was suspended, MRCS and FRCS examinations cancelled and all courses postponed. In theory, essential cancer surgery, emergency and trauma operating will continue. All elective, non-essential cases are currently cancelled. While we adapt to our new ways of working, we remind ourselves that surgeons are flexible, resilient and, ultimately, we are doctors in the first instance. We present a short article on operating during the COVID-19 pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.63163  |
| 3pdo8gor | Practice considerations for proton beam radiotherapy of uveal melanoma during the COVID-19 pandemic: PTCOG Ocular experience                                                                                                                                                                                                                                                                             | Abstract Uveal melanoma (UM) is a rare but life-threatening cancer of the eye. In light of the COVID-19 pandemic, hospitals and proton eye therapy facilities must analyze several factors to ensure appropriate treatment protocols for patients and provider teams. Practice considerations to limit COVID-19 transmission in the proton ocular treatment setting for UM are necessary. The Particle Therapy Co-Operative Group (PTCOG) is the largest international community of particle/proton therapy providers. Participating experts are/were affiliated with the member institutions of the PTCOG Ocular subcommittee with long-standing high-volume proton ocular programs. The practices reviewed in this document must be taken in conjunction with local hospital procedures, multidisciplinary recommendations, and regional/national guidelines, as each community may have its unique needs, supplies, and protocols. Importantly, as the pandemic evolves, so will the strategies and recommendations. Given the unique circumstances for UM patients, along with indications of potential ophthalmologic transmission as a result of healthcare providers working in close proximity to patients and intrinsic infectious risk from eyelashes, tears and hair, practice strategies may be adapted to reduce the risk of viral transmission. Certainly, providers and health care systems will continue to examine and provide as safe and effective care as possible for patients in the current environment.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.63038  |
| c78vokcc | COVID-19: Peer Support and Crisis Communication Strategies to Promote Institutional Resilience                                                                                                                                                                                                                                                                                                           | Caring for patients with the novel coronavirus infection is placing great stress on health care systems and health care workers. On the basis of their experiences responding to other pandemics, the authors summarize lessons learned and offer some best practices for facilitating organizational resilience and supporting health care workers during the COVID-19 pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.627801 |
| i9ykx1vz | Recomendaciones De Consenso Separ Y Aeer Sobre El Uso De La Broncoscopia Y La Toma De Muestras De La Vía Respiratoria En Pacientes Con Sospecha O Con Infección Confirmada Por Covid-19                                                                                                                                                                                                                  | ABSTRACT This consensus document has been drawn up by the Techniques and Transplantation and Nursing areas of the Spanish Society of Pulmonology and Thoracic Surgery (SEPAR) and the Spanish Society of Respiratory Endoscopy (AEER) with the aim of providing information on the safe and effective use of bronchoscopy in patients with suspected or confirmed COVID-19 infection. Our priority is to ensure the safety of our patients, the health workers caring for them, and the community in general. At this stage in the pandemic, our information on the use of bronchoscopy in patients of this type is based on the experience of hospitals in other countries, and scientific publications are scarce. The objective of this document is to compile these experiences, based on recommendations from official agencies, in a document offering guidance in daily clinical practice.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.627282 |
| ip359nhc | Risques de contamination virale des soignants au cours d’une laparoscopie pendant la pandémie de la Covid-19                                                                                                                                                                                                                                                                                             | Résumé La pandémie de la Covid-19 a notablement modifié nos pratiques. Le but de cet article était d’analyser les risques de contamination des soignants au cours des interventions laparoscopiques réalisées chez des patients atteints de Covid-19. L’effet délétère de l’aérosolisation du pneumopéritoine avec présence du virus n’est actuellement pas quantifié. Même si les mesures de protection des soignants sont une extrapolation des mesures prises pour d’autres épidémies, elles doivent être de rigueur afin de minimiser le risque de contamination virale. Les mesures de protection sont les équipements de protection individuelle des soignants, l’adaptation de la technique chirurgicale (méthode de création du pneumopéritoine, filtres, anastomoses plutôt intracorporelles, précautions lors de l’exsufflation du pneumopéritoine), et l’organisation de la salle d’opération. Abstract The Covid-19 pandemic has markedly changed our practices. This article analyses the risks of contamination among healthcare professionals (HCPs) during laparoscopic surgery on patients with Covid-19. Harmful effects of aerosols from a pneumoperitoneum with the virus present have not yet been quantified. Measures for the protection of HCPs are an extrapolation of those taken during other epidemics. They must still be mandatory to minimise the risk of viral contamination. Protection measures include personal protection equipment for HCPs, adaptation of surgical technique (method for obtaining pneumoperitoneum, filters, preferred intracorporeal anastomosis, precautions during the exsufflation of the pneumoperitoneum), and organisation of the operating room.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.611346 |
| kec74uka | COVID-19 and ENT Surgery                                                                                                                                                                                                                                                                                                                                                                                 | ABSTRACT In Otorhinolaryngology - Head and Neck Surgery, clinical examination and invasive procedures on the respiratory tract and on airway-connected cavities such as paranasal sinuses and the middle ear expose people to direct transmission of SARS-CoV-2 by inhalation or ocular projection of contaminated droplets, and to indirect transmission by contact with contaminated hands, objects or surfaces. Estimating an R0 of Covid-19 at around 3 justified postponing non-urgent face-to-face consultations and expanding the use of teleconsultation in order to limit the risks of SARS-CoV-2 infection of patients or health workers and comply with the lockdown. The health authority recommends cancellation of all medical or surgical activities which are not urgent as long as this does not involve a loss of chance for the patient. The purpose of this cancellation is to significantly increase critical care capacity, prioritize the reception of patients with Covid-19, prioritize the allocation of staff and provision of the equipment necessary for their medical or surgical management, and contribute to the smooth running of downstream critical care within their establishment. Another goal is to reduce the risks of patient contamination within healthcare facilities. This document provides guidance on how to proceed with and adapt ENT surgery in the current pandemic context, as well as on the management of postponed operations. This best practice advice must of course be adapted in each region according to the development of the epidemic and pre-existing arrangements. Their local application can only be decided within the framework of collaboration between the ENT teams, the operational hygiene units and all the other specialties concerned.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 0.607722 |
| 4zxktm0r | Adult Cardiac Surgery and the COVID-19 Pandemic: Aggressive Infection Mitigation Strategies are Necessary in the Operating Room and Surgical Recovery                                                                                                                                                                                                                                                    | Abstract The COVID-19 pandemic necessitates aggressive infection mitigation strategies to reduce the risk to patients and healthcare providers. This document is intended to provide a framework for the adult cardiac surgeon to consider in this rapidly changing environment. Pre, intra, and post-operative detailed protective measures are outlined. These are guidance recommendations during a pandemic surge to be utilized for ALL patients while local COVID-19 disease burden remains elevated.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 0.606444 |
| 4wqyelox | Adult Cardiac Surgery and the COVID-19 Pandemic: Aggressive Infection Mitigation Strategies are Necessary in the Operating Room and Surgical Recovery                                                                                                                                                                                                                                                    | Abstract The COVID-19 pandemic necessitates aggressive infection mitigation strategies to reduce the risk to patients and healthcare providers. This document is intended to provide a framework for the adult cardiac surgeon to consider in this rapidly changing environment. Pre, intra, and post-operative detailed protective measures are outlined. These are guidance recommendations during a pandemic surge to be utilized for ALL patients while local COVID-19 disease burden remains elevated.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 0.606444 |
| 190axgea | APA calls for telecommunications use to prevent virus spread                                                                                                                                                                                                                                                                                                                                             | The American Psychological Association (APA) on March 24 called on states and insurers to move quickly to allow people to connect with their mental health providers remotely using telehealth as the need for mental health services rises during the COVID‐19 pandemic. “ We are asking state policymakers to temporarily suspend state licensing requirements for telepsychological services, which would allow patients greater access to their providers and ensure continuity of care during this crisis,” Arthur C. Evans Jr., Ph.D., APA's CEO, said in a statement. “ The federal government has just identified psychologists as critical, essential workers in the U.S. response to COVID‐19. But this does not mean that psychologists and psychology trainees must do work in person,” he said. “ Essential psychological services can, and in many cases, should be delivered through telehealth. It is critically important that psychologists are able to meet the needs of their patients and communities during this difficult time, without further increasing the risk of contagion. ”                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | 0.604699 |
| kcp0izf2 | Strategy for the practice of digestive and oncological surgery during the Covid-19 epidemic                                                                                                                                                                                                                                                                                                              | Summary The Covid-19 pandemic is changing the organization of healthcare and has a direct impact on digestive surgery. Healthcare priorities and circuits are being modified. Emergency surgery is still a priority. Functional surgery is to be deferred. Laparoscopic surgery must follow strict rules so as not to expose healthcare professionals (HCPs) to added risk. The question looms large in cancer surgery–go ahead or defer? There is probably an added risk due to the pandemic that must be balanced against the risk incurred by deferring surgery. For each type of cancer–colon, pancreas, oesogastric, hepatocellular carcinoma–morbidity and mortality rates are stated and compared with the oncological risk incurred by deferring surgery and/or the tumour doubling time. Strategies can be proposed based on this comparison. For colonic cancers T1-2, N0, it is advisable to defer surgery. For advanced colonic lesions, it seems judicious to undertake neoadjuvant chemotherapy and then wait. For rectal cancers T3-4 and/or N+, chemoradiotherapy is indicated, short radiotherapy must be discussed (followed by a waiting period) to reduce time of exposure in the hospital and to prevent infections. Most complex surgery with high morbidity and mortality–oesogastric, hepatic or pancreatic–is most often best deferred.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | 0.601257 |
| vpmrwanr | COVID-19 and telemedicine: Immediate action required for maintaining healthcare providers well-being                                                                                                                                                                                                                                                                                                     | Abstract The well-being of the health care workforce is the cornerstone of every well-functioning health system. As a result of the pandemic, medical healthcare providers are under an enormous amount of workload pressure along with increased total health expenditures. The overwhelming burden of COVID-19 illness could lead to caregiver burnout. Direct-to-consumer telemedicine can enable patients to connect with their healthcare provider at a distance. This virtual platform could be used by smartphones or webcam-enabled computers and allows physicians to effectively screen patients with early signs of COVID-19 before they reach to hospital.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 0.595176 |
| hj77rg85 | Guidance for Cardiac Electrophysiology During the Coronavirus (COVID-19) Pandemic from the Heart Rhythm Society COVID-19 Task Force; Electrophysiology Section of the American College of Cardiology; and the Electrocardiography and Arrhythmias Committee of the Council on Clinical Cardiology, American Heart Association                                                                            | Abstract Covid-19 is a global pandemic that is wreaking havoc with the health and economy of much of human civilization. Electrophysiologists have been impacted personally and professionally by this global catastrophe. In this joint document from representatives of the HRS, ACC and AHA we identify the potential risks of exposure to patients, allied health care staff, industry representatives and hospital administrators. We describe the impact of COVID-19 on cardiac arrhythmias and methods of triage based on acuity and patient comorbidities. We provide guidance for managing invasive and non-invasive electrophysiology procedures, clinic visits and cardiac device interrogations. We discuss resource conservation and the role of tele-medicine in remote patient care along with management strategies for affected patients.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | 0.593434 |
| 8kuz6g7y | European Association of Urology Guidelines Office Rapid Reaction Group: An Organisation-wide Collaborative Effort to Adapt the European Association of Urology Guidelines Recommendations to the Coronavirus Disease 2019 Era                                                                                                                                                                            | Abstract The coronavirus disease 2019 (COVID-19) pandemic is unlike anything seen before by modern science-based medicine. Health systems across the world are struggling to manage it. Added to this struggle are the effects of social confinement and isolation. This brings into question whether the latest guidelines are relevant in this crisis. We aim to support urologists in this difficult situation by providing tools that can facilitate decision making, and to minimise the impact and risks for both patients and health professionals delivering urological care, whenever possible. We hope that the revised recommendations will assist urologist surgeons across the globe to guide the management of urological conditions during the current COVID-19 pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.591759 |
| zolwoedn | North Carolina frees up $30 million for BH during COVID‐19 crisis                                                                                                                                                                                                                                                                                                                                        | North Carolina's seven regional managed care agencies for behavioral health are receiving $30 million so they can keep a variety of programs running during the COVID‐19 crisis, Carolina Public Press reported March 31. The money, which an N.C. Department of Health and Human Services (DHHS) spokesperson said represented unused Medicaid funds, has been shifted to the state's seven local management entities/managed care organizations, commonly called LME/MCOs. The LME/MCOs manage care for beneficiaries receiving mental health, developmental disabilities and substance abuse services who are on Medicaid, uninsured or underinsured. The letter also gave the LME/MCOs authority to use up to 15% of risk reserves to support contracted providers, but with a significant qualifier. The agencies, the DHHS said, would be able to use their reserves to support providers that are “maintaining consumers in their existing residential placements, utilizing telehealth capabilities to the maximum extent possible, providing services that keep consumers from needing access to emergency departments and inpatient services, and otherwise making efforts to deliver high‐quality services during this crisis. ” The department's caveat that local providers make every effort to keep their clients out of hospitals is part of a concerted campaign to maintain as much bed space and equipment as possible as the number of COVID‐19 cases rises.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | 0.590526 |
| 9wfk73kt | Are German endoscopy units prepared for the COVID-19 pandemic? A nationwide survey                                                                                                                                                                                                                                                                                                                       | Objective: The COVID-19 pandemic challenges health care systems worldwide. In this situation, guidelines for health care professionals in endoscopy units with increased risk of infection from inhalation of airborne droplets, conjunctival contact and faeces are urgently needed. Recently, the European Society of Gastrointestinal Endoscopy (ESGE) and the German Society for Pneumology (DGP) issued recommendations. However, real-world data on the conditions and requirements of endoscopy units to adhere to this guidance are missing. Design: We conducted an internet-based survey among German endoscopy units from all levels of care from April 1st to 7th, 2020. The survey comprised 33 questions and was distributed electronically by the German Society of Gastroenterology, Digestive and Metabolic Diseases (DGVS) and the DGP. Results: In total, 656 endoscopy units completed the survey. Overall, 253 units (39%) cancelled fewer than 40% of their procedures. Of note, private practices cancelled less procedures than hospital-based units. Complete separation of high-risk and COVID-19 positive patients was achieved in only 20% of the units. Procedural measures were well adopted, with 91% of the units systematically identifying patients at risk and 85% using risk-adapted personal protective equipment (PPE). For the future, shortages in PPE (81%), staff (69%) and relevant financial losses (77%) were expected. Conclusion: Concise definitions of non-urgent, elective interventions and endoscopic surveillance strategies are needed to better guide endoscopic activity and intervention cancellations. In the short term, a lack of PPE can constitute considerable impairment of endoscopy units' operability and patient outcomes.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0.586814 |
| 6lql1m5m | Recommendation to Optimize Safety of Elective Surgical Care While Limiting the Spread of COVID-19: Primum Non Nocere                                                                                                                                                                                                                                                                                     | Abstract COVID-19 has drastically altered our lives in an unprecedented manner, shuttering industries, and leaving most of the country in isolation as we adapt to the evolving crisis. Orthopedic surgery has not been spared from these effects, with the postponement of elective procedures in an attempt to mitigate disease transmission and preserve hospital resources as the pandemic continues to expand. During these turbulent times, it is crucial to understand that while patient and care-providers safety is paramount, canceling or postponing essential surgical care is not without consequences, and may be irreversibly detrimental to a patient’s health and quality of life in some cases. The optimal solution of how to effectively balance the resumption of standard surgical care while doing everything possible to limit the spread of COVID-19 is undetermined, and could include strategies such as social distancing, screening forms and tests including temperature screening, segregation of inpatient and outpatient teams, proper use of protective gear, and the use of ambulatory surgery centers (ASCs) to provide elective, yet ultimately essential, surgical care while conserving resources and protecting the health of patients and health-care providers. Of importance, these recommendations do not and should not supersede evolving United States Centers for Disease Control and Prevention (CDC), and relevant federal, state and local public health guidelines. Level of Evidence: Level V.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.581758 |
| quobbmos | Using Technology to Maintain the Education of Residents During the COVID-19 Pandemic                                                                                                                                                                                                                                                                                                                     | Background The COVID-19 pandemic presents a unique challenge to surgical residency programs. Due to the restrictions recommended by the Centers for Disease Control and Prevention and other organizations, the educational landscape for surgical residents is rapidly changing. In addition, the time course of these changes is undefined. Methods We attempt to define the scope of the problem of maintaining surgical resident education while maintaining the safety of residents, educators, and patients. Within the basic framework of limiting in-person gatherings, postponing or canceling elective operations in hospitals, and limiting rotations between sites, we propose innovative solutions to maintain rigorous education. Results We propose several innovative solutions including the flipped classroom model, online practice questions, teleconferencing in place of in-person lectures, involving residents in telemedicine clinics, procedural simulation, and the facilitated use of surgical videos. Although there is no substitute for hands-on learning through operative experience and direct patient care, these may be ways to mitigate the loss of learning exposure during this time. Conclusions These innovative solutions utilizing technology may help to bridge the educational gap for surgical residents during this unprecedented circumstance. The support of national organizations may be beneficial in maintaining rigorous surgical education.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.580881 |
| lsk9vq9h | Foot and ankle service adaptation in response to COVID-19 and beyond                                                                                                                                                                                                                                                                                                                                     | Abstract The disruption to healthcare provision as a result of the COVID-19 pandemic has compelled us to streamline healthcare delivery. This has given us an opportunity to implement healthcare technology, reform inter-disciplinary collaboration and ultimately enhance patient care. We discuss some of the advances made by the foot and ankle department at our hospital. These innovations have broad applicability and will hopefully ignite discussion amoung a number of healthcare teams about improving the future care of their patients.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.58086  |
| zrtfyz8s | Managing Cancer Care during the COVID-19 Pandemic and Beyond                                                                                                                                                                                                                                                                                                                                             | Abstract COVID-19 pandemic has posed insurmountable challenges to health care systems globally. Cancer therapy is complex, and outcomes are centered on timing. Many oncology societies and health ministries have issued guidelines for cancer care to enable oncologists and patients navigate the crisis. Lessons learned should inform care models for future pandemics.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0.57894  |
| sb45v082 | The Changing Face of Orthopedic Education: Searching for the New Reality After COVID-19                                                                                                                                                                                                                                                                                                                  | Abstract The COVID-19 pandemic has had immediate impact on the practice of medicine and orthopaedic education. As the practice of social distancing has been put into place to help slow the spread of disease as well as to conserve medical supplies and equipment, elective surgery has come to a grinding halt. This dramatic change has forced our leaders to critically evaluate the delivery of education and skills training for our residents, fellows as well as all orthopaedic surgeons. We must continue to develop technologies such as virtual meeting platforms, distance learning, simulation-based training, virtual reality andaugmented reality to augment the new world of orthopaedic education.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 0.575761 |
| lchsdys1 | Limiting spread of COVID-19 in an orthopaedic department—a perspective from Spain                                                                                                                                                                                                                                                                                                                        | Besides national and international recommendations, orthopaedic departments face significant changes in daily activity and serious issues to maintain their standards in musculoskeletal care during the pandemic Covid-19 crisis that we are facing. This report retrospectively addresses measures that were progressively put in place to modify in a week time the activity of a busy orthopaedic department in a large tertiary university hospital in face of the pandemic. Surgical priorities and surgical outcomes are key aspects to consider. The experience may offer some insight to areas where the spread of the disease may be slower or delayed. Abrupt stop of scheduled surgery and clinics is useful to adapt an orthopaedic department to the overall hospital resource reorganization. Orthopaedic surgeons need to be aware of the risks to patients and personnel in view of underdiagnosed cases, which make pre-operative Covid-19 evaluation mandatory for all surgical cases.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.575505 |
| sroy543x | Acute Stroke Care in the Coronavirus Disease 2019 Pandemic                                                                                                                                                                                                                                                                                                                                               | Abstract Coronavirus disease 2019 (COVID-19) is a pandemic respiratory disease with serious public health risk, and has taken the world off guard with its rapid spread. As the COVID-19 pandemic intensifies, overwhelming the healthcare system and the medical community, current practice for the management of acute ischemic stroke (AIS) will require modification, and guidelines should be relaxed while maintaining high standard quality of care. The aim of these suggestions is to avoid contributing to the rapid spread of COVID-19 as well as to conserve what are likely to be very limited resources (including personnel, intensive care/hospital beds as well as physicians) while maintain high quality care for patients with AIS. We present our recommendations for the management of acute stroke during the COVID-19 pandemics.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.57337  |
| x5zwhkkj | Disruptive Modifications to Cardiac Critical Care Delivery During the Covid-19 Pandemic: An International Perspective                                                                                                                                                                                                                                                                                    | Abstract The COVID-19 pandemic has presented a major unanticipated stress on our workforce, organizational structure, systems of care, and critical resource supply. In order to ensure provider safety, maximize efficiency, and optimize patient outcomes, health systems need to be agile. Critical care cardiologists may be uniquely positioned to treat the numerous respiratory and cardiovascular complications of the SARS-CoV-2 virus and support clinicians without critical care training who may be suddenly asked to care for critically ill patients. This manuscript draws upon the experiences of colleagues from heavily impacted regions of the United States and Europe as well as lessons learned from military mass casualty medicine. We offer pragmatic suggestions on how to implement scalable models for critical care delivery, cultivate educational tools for team training, and embrace technologies such as telemedicine to enable effective collaboration despite social distancing imperatives.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.572119 |
| t6blhwy2 | Brief guideline for the prevention of COVID-19 infection in head and neck and otolaryngology surgeons                                                                                                                                                                                                                                                                                                    | Abstract Importance Anatomically, viral density is greater in the nasal cavity and the nasopharynx. It is to be expected that instrumentation in or through those areas will entail a higher risk of transmission. That's why head and neck and otolaryngologist surgeons are among the most vulnerable health professionals. Observations Surgeons should essentially perform procedures they require. Surgeries should be performed with personal protective equipment suitable for the high risk of aerosolization: goggles, N95 face mask, facial mask, blood-repelling gown and gloves. It is advisable to have the cooperative COVID-19 test in all patients. Telemedicine is a useful resource if resources allow it. Conclusions and relevance Otolaryngologists and related specialists are among the groups at higher risk when performing surgeries and upper airway examinations. There are no emergencies in a pandemic. The care of health professionals is crucial to combating this health situation.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.571699 |
| ziddepr7 | The Past, Present, and Future of Orthopaedic Education: Lessons Learned from the COVID-19 Pandemic                                                                                                                                                                                                                                                                                                       | Abstract The COVID-19 global pandemic has upended nearly every medical discipline, dramatically impacted patient care, and has had far-reaching effects on surgeon education. In many areas of the country, elective orthopaedic surgery has completely stopped to ensure that resources are available for the critically ill and to minimize the spread of disease. COVID-19 is forcing many around the world to reevaluate existing processes and organizations and adapt to carry out business, of which medicine and education are not immune. The majority of national and international orthopaedic conferences, training programs, and workshops have been postponed or canceled, and we are now critically evaluating the delivery of education to our colleagues as well as residents and fellows. This manuscript describes the evolution of orthopaedic education and significant paradigm shifts necessary to continue to teach ourselves and the future leaders of our noble profession.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.56909  |
| 2z5ixtk3 | Anti-cancer therapy and clinical trial considerations for gynecologic oncology patients during the COVID-19 pandemic crisis                                                                                                                                                                                                                                                                              | Abstract Objectives The COVID-19 pandemic has consumed considerable resources and has impacted the delivery of cancer care. Patients with cancer may have factors which place them at high risk for COVID 19 morbidity or mortality. Highly immunosuppressive chemotherapy regimens and possible exposure to COVID-19 during treatment may put patients at additional risk. The Society of Gynecologic Oncology convened an expert panel to address recommendations for best practices during this crisis to minimize risk to patients from deviations in cancer care and from COVID-19 morbidity. Methods An expert panel convened to develop initial consensus guidelines regarding anti-neoplastic therapy during the COVID-19 pandemic with respect to gynecologic cancer care and clinical trials. Results COVID-19 poses special risks to patients who are older, have medical co-morbidities, and cancer. In addition, this pandemic will likely strain resources, making delivery of cancer care or conduct of clinical trials unpredictable. Recommendations are to limit visits and contact with health care facilities by using telemedicine when appropriate, and choosing regimens which require less frequent visits and which are less immunosuppressive. Deviations will occur in clinical trials as a result of limited resources, and it is important to understand regulatory obligations to trial sponsors as well as to the IRB to ensure that clinical trial and patient safety oversight are maintained. Conclusions The ongoing crisis will strain resources needed to deliver cancer care. When alterations to the delivery of care are mandated, efforts should be taken to minimize risks and maximize safety while approximating standard practice.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.567976 |
| 4z2myiet | Clear plastic drapes may be effective at limiting aerosolization and droplet spray during extubation: implications for COVID-19                                                                                                                                                                                                                                                                          | Protection of frontline HCPs is paramount. However, PPE is a limited resource and often requires providers to be adaptive and resourceful in a crisis. The inexpensive and simple method of using clear drapes during extubation (and possibly intubation) of COVID-19 patients may be considered by frontline HCPs and infection control specialists as an additional precaution. Modifications of the clear plastics can be adapted for surgical procedures that may be AGMPs.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | 0.566815 |
| hn1z0817 | Precautions and Procedures for Coronary and Structural Cardiac Interventions during the COVID-19 Pandemic: Guidance from Canadian Association of Interventional Cardiology                                                                                                                                                                                                                               | Abstract The globe is currently in the midst of a COVID-19 pandemic resulting in significant morbidity and mortality. This pandemic has placed considerable stress on health care resources and providers. This document from the Canadian Association of Interventional Cardiology - Association Canadienne de Cardiologie d'intervention, specifically addresses the implications for the care of patients in the Cardiac Catheterization Laboratory (CCL) in Canada during the COVID-19 pandemic. The key principles of this document are to maintain essential interventional cardiovascular care while minimizing risks of COVID-19 to patients/staff and maintaining the overall healthcare resources. As the COVID-19 pandemic evolves, procedures will be increased or reduced based on the current level of restriction to health care services. While some consistency across the country is desirable, provincial and regional considerations will influence how these recommendations are implemented. We believe the framework and recommendations in this document will provide crucial guidance for clinicians and policy makers on the management of coronary and structural procedures in the CCL as the COVID-19 pandemic escalates and eventually abates.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0.560637 |
| zawfotcx | Creating a Palliative Care Inpatient Response Plan for COVID-19—The UW Medicine Experience                                                                                                                                                                                                                                                                                                               | Abstract Context The coronavirus disease 2019 (COVID-19) pandemic is stressing health care systems throughout the world. Significant numbers of patients are being admitted to the hospital with severe illness, often in the setting of advanced age and underlying comorbidities. Therefore, palliative care is an important part of the response to this pandemic. The Seattle area and UW Medicine have been on the forefront of the pandemic in the U.S. Methods UW Medicine developed a strategy to implement a palliative care response for a multihospital health care system that incorporates conventional capacity, contingency capacity, and crisis capacity. The strategy was developed by our palliative care programs with input from the health care system leadership. Results In this publication, we share our multifaceted strategy to implement high-quality palliative care in the context of the COVID-19 pandemic that incorporates conventional, contingency, and crisis capacity and focuses on the areas of the hospital caring for the most patients: the emergency department, intensive care units, and acute care services. The strategy focuses on key content areas, including identifying and addressing goals of care, addressing moderate and severe symptoms, and supporting family members. Conclusion Strategy planning for delivery of high-quality palliative care in the context of the COVID-19 pandemic represents an important area of need for our health care systems. We share our experiences of developing such a strategy to help other institutions conduct and adapt such strategies more quickly.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 0.560589 |
| gl6teurb | Interventional oncology at the time of COVID-19 pandemic: problems and solutions                                                                                                                                                                                                                                                                                                                         | Abstract The COVID-19 pandemic has deeply impacted the activity of interventional oncology in hospitals and cancer centers. In this review based on official recommendations of different international societies, but also on local solutions found in different expert large-volume centers, we discuss the changes that need to be done for the organization, safety, and patient management in interventional oncology. A literature review of potential solutions in a context of scarce anesthesiologic resources, limited staff and limited access to hospital beds are proposed and discussed based on the literature data.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.559963 |
| froywddm | Preventing intra-hospital infection and transmission of COVID-19 in healthcare workers                                                                                                                                                                                                                                                                                                                   | Abstract Coronavirus disease 2019 (COVID-19) poses an occupational health risk to healthcare workers. Several thousand healthcare workers have already been infected, mainly in China. Preventing intra-hospital transmission of the communicable disease is therefore a priority. Based on the Systems Engineering Initiative for Patient Safety model, the strategies and measures to protect healthcare workers in an acute tertiary hospital are described along the domains of work task, technologies and tools, work environmental factors, and organizational conditions. The principle of zero occupational infection remains an achievable goal that all healthcare systems need to strive for in the face of a potential pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0.557983 |
| vwu27sw2 | UV Sterilization of Personal Protective Equipment with Idle Laboratory Biosafety Cabinets During the Covid-19 Pandemic                                                                                                                                                                                                                                                                                   | Personal protective equipment (PPE), including surgical masks and N95 respirators, is crucially important to the safety of both patients and medical personnel, particularly in the event of infectious pandemics. As the incidence of Coronavirus Disease (COVID-19) is increasing exponentially in the United States and worldwide, healthcare provider demand for these necessities is currently outpacing supply. As such, strategies to safely expand the lifespan of the supply of medical equipment are critically important. In the recent days, weeks, and months, in the midst of the current pandemic, there has been a concerted effort to identify viable ways to conserve Personal Protective Equipment, including sterilization after use. Some hospitals have already begun using UV-C light to sterilize N95 respirators, but many lack the space or equipment to implement existing protocols. In this study, we outline a procedure by which N95 respirators may be sterilized using ultraviolet (UV) radiation in biosafety cabinets (BSCs), a common element of many academic, public health, and hospital laboratories. The primary obstacle to this approach is the possibility the UV radiation levels vary within BSCs. To account for this potential variation in dosing across the base of the BSC, we tested the UV-C radiation in two randomly chosen idle BSCs in our research institute and observed a maximum ratio between the minimum and maximum recorded intensities within a given BSC to be 1.98. Based on these values, we calculated that an N95 mask placed within a BSC with a manufacturer reported fluence of 100 μW/cm^2 should be effectively sanitized for reuse after approximately 15-20 minutes per side. Our results provide support to healthcare organizations looking for alternative methods to extend their reserves of PPE. It is our hope that with an easily implemented strategy, as we have presented here, idle BSCs can be utilized to alleviate the PPE shortage by providing a way to sterilize PPE to allow safe daily re-use. This should be tested on a larger scale, and confirmed in a virology laboratory before adoption, though we contend that in extremis, this method would be preferred compared to re-use without sterilization.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0.554781 |
| jnb4ad03 | Resource Re-Allocation During the COVID-19 Pandemic in a Suburban Hospital System: Implications for Outpatient Hip and Knee Arthroplasty                                                                                                                                                                                                                                                                 | Abstract The COVID pandemic of 2020 has emerged as a global threat to patients, healthcare providers and to the global economy. Due to this particular novel and highly infectious strain of coronavirus, the rapid community spread and clinical severity of the subsequent respiratory syndrome created a substantial strain on hospitals and healthcare systems around the world. The rapid surge of patients presenting over a small time period for emergent clinical care, admission to the hospital and intensive care units with many requiring mechanically assisted ventilators for respiratory support demonstrated the potential to overwhelm healthcare workers, hospitals and healthcare systems. The purpose of our article is to describe an effective system for re-deployment of healthcare supplies, resources and personnel to hospitals within a suburban academic hospital system to optimize the care of COVID patients, while treating orthopaedic patients in an equally ideal setting to maximize their surgical and clinical care. This article will provide a particular focus on the current and future role of a specialty hip and knee hospital and its partnering ambulatory surgery center in the context of an outpatient arthroplasty program.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.554088 |
| 9r60pgqv | Assessing the Burden of Nondeferrable Major Uro-oncologic Surgery to Guide Prioritisation Strategies During the COVID-19 Pandemic: Insights from Three Italian High-volume Referral Centres                                                                                                                                                                                                              | Abstract The coronavirus 2019 (COVID-19) pandemic has led to an unprecedented emergency scenario for all aspects of health care, including urology. At the time of writing, Italy was the country with the highest rates of both infection and mortality. A panel of experts recently released recommendations for prioritising urologic surgeries in a low-resource setting. Of note, major cancer surgery represents a compelling challenge. However, the burden of these procedures and the impact of such recommendations on urologic practice are currently unknown. To fill this gap, we assessed the yearly proportion of high-priority major uro-oncologic surgeries at three Italian high-volume academic centres. Of 2387 major cancer surgeries, 32.3% were classified as high priority (12.6% of radical nephroureterectomy, 17.3% of nephrectomy, 33.9% of radical prostatectomy, and 36.2% of radical cystectomy cases). Moreover, 26.4% of high-priority major cancer surgeries were performed in patients at higher perioperative risk (American Society of Anesthesiologists score ≥3), with radical cystectomy contributing the most to this cohort (50%). Our real-life data contextualise ongoing recommendations on prioritisation strategies during the current COVID-19 pandemic, highlighting the need for better patient selection for surgery. We found that approximately two-thirds of elective major uro-oncologic surgeries can be safely postponed or changed to another treatment modality when the availability of health care resources is reduced. Patient summary: We used data from three high-volume Italian academic urology centres to evaluate how many surgeries performed for prostate, bladder, kidney, and upper tract urothelial cancer can be postponed in times of emergency. We found that approximately two-thirds of patients with these cancers do not require high-priority surgery. Conversely, of patients requiring high-priority surgery, approximately one in four is considered at high perioperative risk. These patients may pose challenges in allocation of resources in critical scenarios such as the current COVID-19 pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.553042 |
| pzjxsh02 | Collaborative Multi-Disciplinary Incident Command at Seattle Children’s Hospital for Rapid Preparatory Pediatric Surgery Countermeasures to the COVID-19 Pandemic                                                                                                                                                                                                                                        | Abstract Washington State was the first U.S. state to have a patient test positive for COVID-19. Prior to this, our children’s hospital proactively implemented an incident command structure which allowed for collaborative creation of safety measures, policies and procedures for both patients, families, staff and providers. Although the treatment and protective standards are continuously evolving, this commentary shares our thoughts on how an institution, specifically surgical services may develop collaborative process improvement to accommodate for rapid and ongoing change. Specific changes outlined include [1] early establishment of incident command [2] personal protective equipment conservation, [3] workforce safety, [4] surgical and ambulatory patient triage, and [5] optimization of trainee education. Please note that the contents of this manuscript are shared in the interest of providing collaborative information and are under continuous development as our regional situation changes. We recognize the limitations of this commentary and do not suggest that our approaches represent validated best practices.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.551187 |
| hm516x6p | Endoscopy during the Covid-19 outbreak: experience and recommendations from a single center in a high-incidence scenario                                                                                                                                                                                                                                                                                 | Abstract A dramatic SARS-Cov-2 outbreak is hitting Italy hard. To face the new scenario all the hospitals have been re-organised in order to reduce all the outpatient services and to devote almost all their personnel and resources to the management of Covid-19 patients. As a matter of fact, all the services have undergone a deep re-organization guided by: the necessity to reduce exams, to create an environment that helps reduce the virus spread, and to preserve the medical personnel from infection. In these days a re-organization of the endoscopic unit, sited in a high-incidence area, has been adopted, with changes to logistics, work organization and patients selection. With the present manuscript, we want to support gastroenterologists and endoscopists in the organization of a “new” endoscopy unit that responds to the “new” scenario, while remaining fully aware that resources availability and local circumstances may extremely vary from unit to unit.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.551054 |
| 59d9ze84 | Caring for the carers: Ensuring the provision of quality maternity care during a global pandemic                                                                                                                                                                                                                                                                                                         | Abstract The COVID-19 pandemic is impacting health systems worldwide. Maternity care providers must continue their core business in caring and supporting women, newborns and their families whilst also adapting to a rapidly changing health system environment. This article provides an overview of important considerations for supporting the emotional, mental and physical health needs of maternity care providers in the context of the unprecedented crisis that COVID-19 presents. Cooperation, planning ahead and adequate availability of PPE is critical. Thinking about the needs of maternity providers to prevent stress and burnout is essential. Emotional and psychological support needs to be available throughout the response. Prioritising food, rest and exercise are important. Healthcare workers are every country’s most valuable resource and maternity providers need to be supported to provide the best quality care they can to women and newborns in exceptionally trying circumstances.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0.55071  |
| ggqydlf8 | Prostate Cancer Radiotherapy Recommendations in Response to COVID-19                                                                                                                                                                                                                                                                                                                                     | Abstract Purpose During a global pandemic the benefit of routine visits and treatment of cancer patients must be weighed against the risks to patients, staff, and society. Prostate cancer is one of the most common cancers Radiation Oncology departments treat, and efficient resource utilization is essential in the setting of a pandemic. Herein, we aim to establish recommendations and a framework by which to evaluate prostate radiotherapy management decisions. Patients and Methods Radiation Oncologists from the United States and United Kingdom rapidly conducted a systematic review and agreed upon recommendations to safely manage prostate cancer patients during the COVID-19 pandemic. A RADS framework was created: Remote visits, and Avoidance, Deferment, and Shortening of radiotherapy was applied to determine appropriate approaches. Results Recommendations are provided by National Comprehensive Cancer Network (NCCN) risk group, including clinical node positive, post-prostatectomy, oligometastatic, and low volume M1 disease. Across all prostate cancer stages, telemedicine consultations and return visits were recommended when resources/staff available. Delays in consultations and return visits was deemed safe based on stage of disease between 1-6 months. Treatment can be avoided or delayed until safe for very low, low, and favorable intermediate-risk disease. Unfavorable intermediate-risk, high-risk, clinical node positive, recurrence post-surgery, oligometastatic, and low-volume M1 disease can receive neoadjuvant hormone therapy for 4-6 months as necessary. Ultrahypofractionation was preferred for localized, oligometastatic, and low volume M1, and moderate hypofractionation was preferred for post-prostatectomy and clinical node positive disease. Salvage was preferred to adjuvant radiation. Conclusion Resources can be reduced for all identified stages of prostate cancer. The RADS (Remote visits, and Avoidance, Deferment, and Shortening of radiotherapy) framework can be applied to other disease sites to help with decision making in a global pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.550576 |
| od48j12b | French consensus regarding precautions during tracheostomy and post-tracheostomy care in the context of COVID-19 pandemic                                                                                                                                                                                                                                                                                | Abstract Tracheostomy post-tracheostomy care are regarded as at high risk for contamination of health care professionals with the new coronavirus (SARS-CoV-2). Considering the rapid spread of the infection, all patients in France must be considered as potentially infected by the virus. Nevertheless, patients without clinical or radiological (CT scan) markers of COVID-19, and with negative nasopharyngeal sample within 24h of surgery, are at low risk of being infected. Instructions for personal protection include specific wound dressings and decontamination of all material used. The operating room should be ventilated after each tracheostomy and the pressure of the room should be neutral or negative. Percutaneous tracheostomy is to be preferred over surgical cervicotomy in order to reduce aerosolization and to avoid moving patients from the intensive care unit to the operating room. Ventilation must be optimized during the procedure, to limit patient oxygen desaturation. Drug assisted neuromuscular blockage is advised to reduce coughing during tracheostomy tube insertion. An experienced team is mandatory to secure and accelerate the procedure as well as to reduce risk of contamination.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.550243 |
| 6aaqj1ns | Precautions for Operating Room Team Members during the COVID-19 Pandemic                                                                                                                                                                                                                                                                                                                                 | Abstract Background The novel corona virus SARS-CoV-2 (COVID-19) can infect healthcare workers. We developed an institutional algorithm to protect operating room team members during the COVID-19 pandemic and rationally conserve personal protective equipment (PPE). Study Design An interventional platform (operating room, interventional suites, and endoscopy) PPE taskforce was convened by the hospital and medical school leadership and tasked with developing a common algorithm for PPE use, to be used throughout the interventional platform. In conjunction with our infection disease experts, we developed our guidelines based upon potential patterns of spread, risk of exposure and conservation of PPE. Results A decision tree algorithm describing our institutional guidelines for precautions for operating room team members was created. This algorithm is based on urgency of operation, anticipated viral burden at the surgical site, opportunity for a procedure to aerosolize virus, and likelihood a patient could be infected based on symptoms and testing. Conclusion Despite COVID-19 being a new threat, we have shown that by developing an easy-to-follow decision tree algorithm for the interventional platform teams, we can ensure optimal healthcare worker safety.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.549655 |
| f1gum03h | Crisis Symptom Management and Patient Communication Protocols Are Important Tools for All Clinicians Responding to COVID-19                                                                                                                                                                                                                                                                              | Abstract Symptom management and skilled communication with patients and families are essential clinical services in the midst of the COVID-19 pandemic. While palliative care specialists have training in these skills, many front-line clinicians from other specialties do not. It is imperative that all clinicians responding to the COVID-19 crisis have access to clinical tools to support symptom management and difficult patient and family communication.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.542443 |
| ndi3f5oo | MH coalition asks government for needed protective gear                                                                                                                                                                                                                                                                                                                                                  | A mental health coalition on April 1 in urged the federal government to provide personal protection equipment (PPE) to all behavioral health care professionals on the frontlines of the COVID‐19 pandemic. The letter, signed by the National Council for Behavioral Health, the National Alliance on Mental Illness, Mental Health America and the American Psychiatric Association, states that the lack of PPE is putting behavioral health care workers at risk.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.541967 |
| 9jddzpzp | Consensus français sur la réalisation de trachéotomies et les soins de trachéotomies pendant la pandémie de COVID-19                                                                                                                                                                                                                                                                                     | ABSTRACT Tracheostomy post-tracheostomy care are regarded as at high risk for contamination of health care professionals with the new coronavirus (SARS-CoV-2). Considering the rapid spread of the infection, all patients in France must be considered as potentially infected by the virus. Nevertheless, patients without clinical or radiological (CT scan) markers of COVID-19, and with negative nasopharyngeal sample within 24h of surgery, are at low risk of being infected. Instructions for personal protection include specific wound dressings and decontamination of all material used. The operating room should be ventilated after each tracheostomy and the pressure of the room should be neutral or negative. Percutaneous tracheostomy is to be preferred over surgical cervicotomy in order to reduce aerosolization and to avoid moving patients from the intensive care unit to the operating room. Ventilation must be optimized during the procedure, to limit patient oxygen desaturation. Drug assisted neuromuscular blockage is advised to reduce coughing during tracheostomy tube insertion. An experienced team is mandatory to secure and accelerate the procedure as well as to reduce risk of contamination.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.537643 |
| yahdp3td | Intersecting U.S. Epidemics: COVID-19 and Lack of Health Insurance                                                                                                                                                                                                                                                                                                                                       | The COVID-19 pandemic has led to an unprecedented surge in unemployment in the United States. For many, job loss carries the added sting of losing health insurance. The authors discuss the problem of lack of health insurance during a time of risk for severe illness and offer potential solutions that policymakers should consider to mitigate harm.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 0.535759 |
| ncbv6s6l | COVID-19 and ENT Pediatric otolaryngology during the COVID-19 pandemic. Guidelines of the French Association of Pediatric Otorhinolaryngology (AFOP) and French Society of Otorhinolaryngology (SFORL)                                                                                                                                                                                                   | ABSTRACT Objective: joint guidelines of the French Pediatric Otolaryngology Society (AFOP) and of the French Society of otorhinolaryngology – Head and neck Surgery (SFORL) on the management of paediatric otolaryngology patients in the context of the COVID-19 pandemic. Methods: A nation-wide workgroup drew guidelines based on clinical experience, national and local recommendations and scientific literature. Proposals may have to be updated on a day-to-day basis. Results: In children, incidence of symptomatic COVID-19 (1-5%) is low and of good prognosis. The indications for nasal flexible endoscopy should be drastically limited. If undertaken, full Personal Protective Equipment (PPE) including FFP2 masks are required, as well as use of a sheath. Saline nose wash done by caregivers other than parents at home should require PPE. Unless foreign body tracheobronchial aspiration is clinically obvious, CT-scan should be performed to confirm indication of endoscopy. Surgical indications should be limited to emergencies and to cases that cannot be delayed beyond 2 months (especially endonasal, endopharyngeal laryngo-tracheobronchial procedures). Postponement should ideally be a group decision and recorded as such in the medical file. Surgical techniques should be adapted to limit the risk of viral dissemination in the air, avoiding the use of drills, microdebriders, monopolar cautery or lasers. Continuous suction should be placed near the operating field. In case of confirmed Covid-19 cases, or suspected cases (or in some centres systematically), PPE with FFP2 mask should be worn by all staff members present in the operating room.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | 0.5355   |
| poaslccs | Implementation of a Novel Remote Physician SBRT Coverage Process during the Coronavirus Pandemic                                                                                                                                                                                                                                                                                                         | Abstract Introduction During the COVID-19 pandemic, alternative methods of care are needed to reduce the relative risk of transmission in departments. Also needed is the ability to provide vital radiation oncological care if radiation oncologists (RO) are reallocated to other departments. We implemented a novel remote RO SBRT coverage practice, requiring it to be reliable, of high audio and visual quality, timely, and the same level of specialty care as our current in-person treatment coverage practice. Methods All observed failure modes were recorded during implementation over the first 15 sequential fractions. The time from CBCT to treatment was calculated before and after implementation to determine timeliness of remote coverage. Image quality metrics were calculated between the imaging console screen and the RO’s shared screen. Comfort levels with audio/visual communication as well as overall comfort in comparison to in-person RO coverage was evaluated using Likert scale surveys after treatment. Results Remote RO SBRT coverage was successfully implemented in 14/15 fractions with 3 observed process failures that were all corrected before treatment. Average times of pre-treatment coverage before and after implementation were 8.74 and 8.51min, respectively. The cross correlation between the imaging console screen and RO’s shared screen was r=0.96 and lag was 0.05s. The average value for all survey questions was above 4.5, approaching in-person RO coverage comfort levels. Conclusions Our novel method of remote RO SBRT coverage permits reduced personnel and patient interactions surrounding RT procedures. This may help to reduce transmission of COVID-19 in our department and provides a means for SBRT coverage if ROs are reallocated to other areas of the hospital for COVID-19 support.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.535173 |
| 2tgchh89 | Implementation of a Novel Remote Physician SBRT Coverage Process during the Coronavirus Pandemic                                                                                                                                                                                                                                                                                                         | INTRODUCTION: During the COVID-19 pandemic, alternative methods of care are needed to reduce the relative risk of transmission in departments. Also needed is the ability to provide vital radiation oncological care if radiation oncologists (RO) are reallocated to other departments. We implemented a novel remote RO SBRT coverage practice, requiring it to be reliable, of high audio and visual quality, timely, and the same level of specialty care as our current in-person treatment coverage practice. METHODS: All observed failure modes were recorded during implementation over the first 15 sequential fractions. The time from CBCT to treatment was calculated before and after implementation to determine timeliness of remote coverage. Image quality metrics were calculated between the imaging console screen and the RO′s shared screen. Comfort levels with audio/visual communication as well as overall comfort in comparison to in-person RO coverage was evaluated using Likert scale surveys after treatment. RESULTS: Remote RO SBRT coverage was successfully implemented in 14/15 fractions with 3 observed process failures that were all corrected before treatment. Average times of pre-treatment coverage before and after implementation were 8.74 and 8.51min, respectively. The cross correlation between the imaging console screen and RO′s shared screen was r=0.96 and lag was 0.05s. The average value for all survey questions was above 4.5, approaching in-person RO coverage comfort levels. CONCLUSIONS: Our novel method of remote RO SBRT coverage permits reduced personnel and patient interactions surrounding RT procedures. This may help to reduce transmission of COVID-19 in our department and provides a means for SBRT coverage if ROs are reallocated to other areas of the hospital for COVID-19 support.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.533964 |
| nwtfs1t9 | Grief During the COVID-19 Pandemic: Considerations for Palliative Care Providers                                                                                                                                                                                                                                                                                                                         | Abstract The COVID-19 pandemic is anticipated to continue spreading widely across the globe throughout 2020. To mitigate the devastating impact of COVID-19, social distancing and visitor restrictions in health care facilities have been widely implemented. Such policies and practices, along with the direct impact of the spread of COVID-19, complicate issues of grief that are relevant to medical providers. We describe the relationship of the COVID-19 pandemic to anticipatory grief, disenfranchised grief, and complicated grief for individuals, families, and their providers. Furthermore, we provide discussion regarding countering this grief through communication, advance care planning, and self-care practices. We provide resources for health care providers, in addition to calling on palliative care providers to consider their own role as a resource to other specialties during this public health emergency.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.530398 |
| 81m48udk | Characterizing occupations that cannot work from home: a means to identify susceptible worker groups during the COVID-19 pandemic                                                                                                                                                                                                                                                                        | Introduction As the COVID-19 pandemic spreads globally, public health guidance is advising all workers to work from home. However, not all workers are employed in occupations which can be done from home. These workers are at an increased risk for exposure to SARS-CoV-2due to increased interaction with the public and other workers, and also potentially at an increased risk for job displacement as more extreme public health measures (such as closing of retail operations or enforcing shelter in place) occur. Methods To characterize the occupations least likely to be able to work from home, national employment and wage data maintained by the United States Bureau of Labor Statistics (BLS) was merged with measures from the BLS O*NET survey data, which ranks occupations by a variety of physical, ergonomic, psychosocial, and structural exposures. Noting that the work that could most easily be done at home would be work done on a computer that does not rely on interaction with the public, O*NET measures quantifying the importance of computer use at work, and the importance of working with or performing for the public at work were utilized. Results From this analysis, 19.5% (28.2 M) of the United States workforce covered by O*NET are employed in occupations where working from home would be difficult, due to minimal use of computer at work, and a high level of interaction with the public. These workers tend to be in service occupations, including retail and food service, protective service occupations, and transportation occupations such as bus drivers. About 25% (35.6 M) of the United States workforce covered by O*NET are employed in occupations where working from home could likely be more easily accommodated, as these workers do much of their work on computers and interaction with the public is not important. These workers tend to be in technology, computer, management, administrative, financial, engineering, and some science occupations, which typically have higher pay than occupations which cannot be done at home. Conclusions The workers in occupations that have minimal computer use, and high interaction with the public are least likely to be able to work from home during a public health emergency. These workers could also be at an increased risk for job displacement if public-facing establishments close or alter their business model in the face of increased public health restrictions. Occupations where working from home is not possible tend to have lower annual median incomes than occupations where working from home is possible, increasing the vulnerability of these workers. Characterizing which occupational groups are least likely to be able to work from home can inform public health risk management and prioritize occupational sectors where additional workplace protections are necessary.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.527578 |
| 94sg088l | Palliative Care Pandemic Pack: a Specialist Palliative Care Service response to planning the COVID-19 pandemic.                                                                                                                                                                                                                                                                                          | Abstract: Specialist Palliative Care services (SPCS) have a vital role to play in the global COVID-19 pandemic. Core expertise in complex symptom management, decision-making in uncertainty, advocacy and education, and ensuring a compassionate response are essential, and SPCS are well positioned to take a proactive approach in crisis management planning. SPCS resource capacity is likely to be overwhelmed, and consideration needs to be given to empowering and supporting high-quality primary palliative care in all care locations. Our local SPCS have developed a Palliative Care Pandemic Pack to disseminate succinct and specific information, guidance, and resources designed to enable the rapid up-skilling of non-specialist clinicians needing to provide palliative care. It may be a useful tool for our SPCS colleagues to adapt as we face this global challenge collaboratively.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.525371 |
| jigauy1f | Healthcare worker mask reuse in a global pandemic: Using idle resources to create an inexpensive, scalable, and accessible UV system for N95 sterilization                                                                                                                                                                                                                                               | As the current COVID-19 pandemic illustrates, not all hospitals and other facilities are equipped with enough personal protective equipment to meet the demand in a crisis. Healthcare workers around the world utilize N95 masks to protect themselves and their patients, yet during this global pandemic they are forced to re-wear what is intended to be single-use masks. This poses significant risk to these healthcare workers along with the populations they are trying to protect. Ultraviolet germicidal irradiation (UVGI) has been validated previously as a way to effectively sterilize these masks between use, however, not all facilities have access to the high cost commercial UV-C lamp sterilization equipment. However, UV-C bulbs are sitting idle in biosafety cabinets (BSCs) at universities and research facilities around the globe that have been shuttered to slow the spread of COVID-19. These bulbs may also be available in existing medical centers where infectious diseases are commonly treated. Therefore, we have developed a method to modify existing light fixtures, or create custom light fixtures compatible with new or existing common UV-C bulbs. This system is scalable and can be created for less than 50 US dollars, on site, at the point of need, and leverages resources that are currently untapped and sitting unused in public and private research facilities. The freely-accessible design can be easily modified for use around the world. Hospitals can obtain this potentially life-saving UVGI resource with minimal funds, via collaboration between research facilities to obtain the UV-C meters and limited availability UVGI bulbs. While mask reuse is not ideal, we must do what we can in emergency situations to protect our frontline healthcare workers and the communities they serve.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.524809 |
| lf9j7d4p | Adaptations de la prise en charge des patients en Nutrition Artificielle à Domicile au cours de l’épidémie virale COVID-19 en France: avis du Comité de Nutrition à Domicile de la Société Francophone de Nutrition Clinique et Métabolisme (SFNCM)                                                                                                                                                      | Résumé La nutrition artificielle à domicile, entérale ou parentérale, s’adresse à des malades chroniques fragiles. La situation actuelle de pandémie COVID-19 peut compromettre leur prise en charge à plusieurs niveaux : difficultés d’accès aux établissements de santé largement réorientés vers la prise en charge des malades COVID-19, possible pénurie d’infirmières à domicile, forte réduction des visites des prestataires de service à domicile, tensions sur les solutions hydro-alcooliques, les masques et les régulateurs de débit. Le but de ces recommandations établies par le Comité de Nutrition à Domicile de la Société Francophone de Nutrition Clinique et Métabolisme est, d’une part, de préciser la prise en charge minimale de ces patients, tant en termes de suivi que de matériels, mais surtout de s’adapter aux tensions actuelles relatives aux personnes et aux matériels, afin de poursuivre une prise en charge de qualité et de ne pas compromettre l’état de santé des patients en nutrition artificielle à domicile pendant la crise. Abstract Home artificial nutrition, whether enteral or parenteral, is provided to chronic and fragile patients. The current COVID-19 epidemics may compromise their care at several levels: difficulties to access to hospitals mainly focused on treating COVID-19 patients, possible lack of nurses at home, strong reduction of visits by homecare providers, tended flow or lack of hand sanitizers, surgical masks and pumps. The aim of these recommendations put together by the French-speaking Society for Clinical Nutrition and Metabolism (SFNCM)’s Home Artificial Nutrition Committee is to define in terms of healthcare resources the minimum care to provide to these patients. We also aim to help cope with the possible tensions, in order to secure the care we must provide to home artificial nutrition patients during this crisis.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | 0.519733 |
| nfzgpicu | Additional challenges faced by cancer patients in Gaza due to COVID-19                                                                                                                                                                                                                                                                                                                                   | Cancer patients in conflict settings experience significant barriers in accessing chemotherapy and radiotherapy as well as palliative care and psychosocial support. Now they face an additional possible risk of infection by SARS-CoV-2 novel coronavirus and the indirect impact of the COVID-19 pandemic on movement restrictions and their access to care. In this commentary, we highlight that despite the low COVID-19 burden in conflict settings like Gaza, COVID-19 could lead to further inequity in cancer care and poorer outcomes for Palestinians with cancer. This is due to the pre-existing shortage in cancer resources as well as the lack of context-specific guidelines to prepare for COVID-19 in war-torn settings.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0.519278 |
| arivuags | COVID-19 Global Pandemic Planning: Decontamination and Reuse Processes for N95 Respirators                                                                                                                                                                                                                                                                                                               | Coronavirus disease 2019 (COVID-19) is an illness caused by a novel coronavirus, severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The disease was first identified as a cluster of respiratory illness in Wuhan City, Hubei Province, China in December 2019, and has rapidly spread across the globe to greater than 200 countries. Healthcare providers are at an increased risk for contracting the disease due to occupational exposure and require appropriate personal protective equipment (PPE), including N95 respirators. The rapid worldwide spread of high numbers of COVID-19 cases has facilitated the need for a substantial supply of PPE that is largely unavailable in many settings, thereby creating critical shortages. Creative solutions for the decontamination and safe reuse of PPE to protect our frontline healthcare personnel are essential. Here, we describe the development of a process that began in late February 2020 for selecting and implementing the use of hydrogen peroxide vapor (HPV) as viable method to reprocess N95 respirators. Since pre-existing HPV decontamination chambers were not available, we optimized the sterilization process in an operating room after experiencing initial challenges in other environments. Details are provided about the prioritization and implementation of processes for collection and storage, pre-processing, HPV decontamination, and post-processing of filtering facepiece respirators (FFRs). Important lessons learned from this experience include, developing an adequate reserve of PPE for effective reprocessing and distribution, and identifying a suitable location with optimal environmental controls (i.e., operating room). Collectively, information presented here provides a framework for other institutions considering decontamination procedures for N95 respirators.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.518789 |
| t624zyyc | Maximizing the Calm Before the Storm: Tiered Surgical Response Plan for Novel Coronavirus (COVID-19)                                                                                                                                                                                                                                                                                                     | Abstract The novel coronavirus (COVID-19) was first diagnosed in Wuhan, China in December 2019 and has now spread throughout the world, being verified by the World Health Organization as a Pandemic on March 11th. This had led to the calling of a national emergency on March 13th in the United States. Many hospitals, healthcare networks, and specifically Departments of Surgery are asking the same questions of how to cope and plan for surge capacity, personnel attrition, novel infrastructure utilization, and resource exhaustion. Herein, we present a tiered plan for surgical department planning based on incident command levels. This includes Acute Care Surgeon deployment (given their critical care training and vertically integrated position in the hospital), recommended infrastructure and transfer utilization, triage principles, and faculty, resident and advanced care practitioner deployment.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.515476 |
| h41vu6vu | The Restructuring of Structural Heart Disease Practice During The Covid-19 Pandemic                                                                                                                                                                                                                                                                                                                      | Abstract Patients with structural heart disease (SHD) are at increased risk of adverse outcomes from the coronavirus disease 19 (COVID-19) due to advanced age and comorbidity. In the midst of a global pandemic of a novel infectious disease, reality-based considerations comprise an important starting point for formulating clinical management pathways. The aim of these “crisis-driven” recommendations is (1) to ensure appropriate and timely treatment of SHD patients, (2) to minimize the risk of COVID-19 exposure to patients and healthcare workers, and (3) to limit resource utilization under conditions of constraint. While the degree of disruption to usual practice will vary across the United States and elsewhere, we hope that early experiences from a Heart Team operating in the current global epicenter of COVID-19 may prove useful for others adapting their practice in advance of local surges of COVID-19.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.515312 |
| 4fi0y96f | Approaches to the management of patients in oral and maxillofacial surgery during COVID-19 pandemic                                                                                                                                                                                                                                                                                                      | Abstract Oral and maxillofacial surgery is correlated with a high risk of SARS-CoV-2 transmission. Therefore, the aim of the review is to collect and discuss aspects of the management of patients in oral and maxillofacial surgery during the COVID-19 pandemic. In order to save resources and to avoid unnecessary exposure to infected patients, there is the need to schedule interventions depending on their priority. During the peak of the pandemic, no elective surgery should be performed. Even urgent procedures might be postponed if there is a view to recovery of a COVID-19 patient within a few days. Emergency procedures do not allow any delay. Specialties with overlap in therapies should have well defined arrangements among each other concerning the treatment spectra in order to avoid redundancy and loss of resources. Inpatient and outpatient units have to be organized in such a way that the risk of cross-infection among patients is reduced to a minimum. Especially, testing of patients for SARS-CoV-2 is important to detect the infected patients at an early stage. When surgery is performed on COVID-19 patients, adequate personal protective equipment is crucial. There must be negative pressure in the operating room, and aerosol formation must be reduced to a minimum. In order to address the COVID-19 challenge adequately, significant changes in the infrastructure of outpatient units, inpatient units, and operating rooms are needed. In addition, the demands concerning personal protective equipment increase significantly. The major aim is to protect patients as well as the medical staff from unnecessary infection, and to keep the healthcare system running effectively. Therefore, every effort should be taken to make the necessary investments.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.514928 |
| p09u3tg5 | Recommendations and guidance for providing pharmaceutical care services during COVID-19 pandemic: A China perspective                                                                                                                                                                                                                                                                                    | Abstract Background The novel coronavirus pneumonia (COVID-19), which was first detected in Wuhan City, has now became a pandemic that affecting patients around the world. Particularly, the community patient population are at high risk of infection and are facing potential failure of proper medication use during the pandemic. Objective To discuss community pharmacists’ role and the content of pharmaceutical care (PC) during the novel coronavirus pandemic to promote effective prevention and control and safe drug use of the community patient population. Method Collect and summarize the experience Chinese community pharmacies gained from providing pharmacy services during the COVID-19 outbreak, and taking patients' PC needs into consideration, analyze and discuss the methods and strategies that community pharmacies and pharmacists shall use to provide PC during the pandemic. Results Community pharmacy management teams shall support PC services by providing adequate supply of COVID-19 related medications and preventative products, following environment regulations, and providing sufficient staff trainings. Pharmacists shall use various approaches to provide PC services in drug dispensing, consulting and referrals, chronic disease management, safe use of infusions, patient education, home care guidance and psychological support to promote the COVID-19 pandemic control and ensure safe medication use of community patients during the pandemic. Conclusion PC services in communities during the COVID-19 shall possess different properties due to disease characteristics and related change in patients' need. Community pharmacies shall work as a strong supporter of patient's medication and protective equipment supply. Community pharmacists shall be prepared to provide skilled and effective PC services for community patient population to ensure medication safety and promote the overall COVID-19 pandemic control.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.514825 |
| gc4yaq4b | First statement on preparation for the COVID-19 pandemic in large German Speaking University-based radiation oncology departments                                                                                                                                                                                                                                                                        | The COVID-19 pandemic is challenging modern radiation oncology. At University Hospitals, we have a mandate to offer high-end treatments to all cancer patients. However, in times of crisis we must learn to prioritize resources, especially personnel. Compromising oncological outcome will blur all statistics, therefore all measures must be taken with great caution. Communication with our neighboring countries, within societies and between departments can help meet the challenge. Here, we report on our learning system and preparation measures to effectively tackle the COVID-19 challenge in University-Based Radiation Oncology Departments.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.514756 |
| 0ei5v1qn | Declaración jurada de salud del viajero para prevenir el coronavirus (COVID-19): Documento técnico                                                                                                                                                                                                                                                                                                       | El documento contiene la Declaración Jurada de Salud del Viajero para prevenir el coronavirus (COVID-19).                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.510506 |
| ry7zbp3m | Economic Recovery After the COVID-19 Pandemic: Resuming Elective Orthopedic Surgery and Total Joint Arthroplasty                                                                                                                                                                                                                                                                                         | Abstract Background The economic effects of the COVID-19 crisis are not like anything the U.S. health care system has ever experienced. Methods As we begin to emerge from the peak of the COVID-19 pandemic, we need to plan the sustainable resumption of elective procedures. We must first ensure the safety of our patients and surgical staff. It must be a priority to monitor the availability of supplies for the continued care of patients suffering from COVID-19. As we resume elective orthopedic surgery and total joint arthroplasty, we must begin to reduce expenses by renegotiating vendor contracts, use ambulatory surgery centers and hospital outpatient departments in a safe and effective manner, adhere to strict evidence-based and COVID-19–adjusted practices, and incorporate telemedicine and other technology platforms when feasible for health care systems and orthopedic groups to survive economically. Results The return to normalcy will be slow and may be different than what we are accustomed to, but we must work together to plan a transition to a more sustainable health care reality which accommodates a COVID-19 world. Conclusion Our goal should be using these lessons to achieve a healthy and successful 2021 fiscal year.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.507524 |
| 9wpns6w8 | Systematic review of international guidelines for tracheostomy in COVID-19 patients.                                                                                                                                                                                                                                                                                                                     | At this moment, the world leaves under the SARS-CoV-2 outbreak pandemic. As Otolaryngologists - Head & Neck Surgeons, we need to perform and participate in examinations and procedures within the head and neck region and airway that are at particularly high risk of exposure and infection because of aerosol and droplet contamination. One of those surgical procedures on demand at this moment is tracheostomy, due the increasing admission in ICU departments and the increased need of ventilatory support secondary to respiratory distress syndrome. This review of international guidelines for tracheostomy in COVID-19 infected patients, aiming to summarize in a systematic way the available recommendations from 18 guidelines from all over the world.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0.506132 |
| pvruhsys | Specific Considerations for the Protection of Patients and Echocardiography Service Providers When Performing Perioperative or Periprocedural Transesophageal Echocardiography During the 2019 Novel Coronavirus Outbreak: Council on Perioperative Echocardiography Supplement to the Statement of the American Society of Echocardiography Endorsed by the Society of Cardiovascular Anesthesiologists | Abstract Notice and Disclaimer This statement reflects recommendations based on expert opinion, national guidelines, and available evidence. Our knowledge with regard to COVID-19 continues to evolve, as do our institutional protocols for dealing with invasive and non-invasive procedures and practice of personal protective equipment. Readers are urged to follow national guidelines and their institutional recommendations regarding best practices to protect their patients and themselves. These reports are made available by ASE as a courtesy reference source for its members. The reports contain recommendations only and should not be used as the sole basis to make medical practice decisions or for disciplinary action against any employee. The statements and recommendations contained in these reports are primarily based on the opinions of experts, rather than on scientifically-verified data. ASE makes no express or implied warranties regarding the completeness or accuracy of the information in these reports, including the warranty of merchantability or fitness for a particular purpose. In no event shall ASE be liable to you, your patients, or any other third parties for any decision made or action taken by you or such other parties in reliance on this information. Nor does your use of this information constitute the offering of medical advice by ASE or create any physician-patient relationship between ASE and your patients or anyone else.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | 0.504637 |
| kkpaovhh | Covid-19: What’s the current advice for UK doctors?                                                                                                                                                                                                                                                                                                                                                      | UK employers have a legal obligation under the Health and Safety at Work Act 1974 to protect staff from harm. And the Control of Substances Hazardous to Health Regulations place a duty to carry out individual risk assessments to identify hazards, quantify risks, and put suitable controls in place, says Steven Nimmo, editor of the Occupational Medicine Journal . “ If the risk assessment establishes that personal protective equipment (PPE) is required then your employer must provide it, properly fit it, and provide suitable instruction and training in its use,” he says. Public Health England’s guidance says that clinicians preparing to assess a patient with suspected covid-19 must wear PPE, which as a minimum should be a correctly fitted FFP3 respirator, gown, gloves, and eye protection.1 Doctors seeing patients with confirmed covid-19 must wear full PPE, including a FFP3 respirator, disposable eye protection, and preferably a visor, a long sleeved disposable gown, and gloves, PHE says. For symptomatic, unconfirmed patients, doctors should wear a fluid resistant surgical mask, gloves, apron and eye protection if there is a risk of splashing into the eyes, PHE recommends.2 “Pregnant women and children are not at high risk,” Nimmo says. “ But the above legal obligations still apply. Immunosuppressed people may well be …[TRUNCATED]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.503661 |
| x9yo2vcs | Recomendaciones éticas para la toma de decisiones difíciles en las unidades de cuidados intensivos ante la situación excepcional de crisis por la pandemia por covid-19: revisión rápida y consenso de expertos                                                                                                                                                                                          | Abstract In view of the exceptional public health situation caused by the COVID-19 pandemic, a consensus work has been promoted from the ethics group of the Spanish Society of the Spanish Society of Intensive, Critical Medicine and Coronary Units (SEMICYUC), with the objective of finding some answers from ethics to the crossroads between the increase of people with intensive care needs and the effective availability of means. In a very short period, the medical practice framework has been changed to a "catastrophe medicine" scenario, with the consequent change in the decision-making parameters. In this context, the allocation of resources or the prioritization of treatment become crucial elements, and it is important to have an ethical reference framework to be able to make the necessary clinical decisions. For this, a process of narrative review of the evidence has been carried out, followed by u. Unsystematic consensus of experts, which has resulted in both the publication of a position paper and recommendations from SEMICYUC itself, and the consensus between 18 scientific societies and 5 institutes / chairs of bioethics and palliative care of a framework document of reference for general ethical recommendations in this context of crisis.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0.502725 |
| 4qdh4cgu | An update on COVID-19 for the otorhinolaryngologist – a Brazilian Association of Otolaryngology and Cervicofacial Surgery (ABORL-CCF) Position Statement                                                                                                                                                                                                                                                 | Abstract Introduction We are facing a pandemic with a great impact worldwide, as a result of the rapid spread of the novel coronavirus (COVID-19). The medical community is still getting to know behavior of this virus and the consequences from a population point of view. All this knowledge is extremely dynamic, so some behaviors are still not well established. Otorhinolaryngologists have a central role in the management of this situation, in which they must assess the patient, avoid contamination to and by health professionals and other patients. Thus, the recommendations of the Brazilian Association of Otorhinolaryngology and Cervical-Facial Surgery (ABORL-CCF) have the main objective of reducing the spread of the new coronavirus during otorhinolaryngological care and assisting in the management of these patients. Methods Review of the main recommendations of national and international scientific societies, decisions by government agencies and class councils. The topics will be related to the general aspects of COVID-19, personal protective equipment, care in patient assistance, endoscopic exam routines and the management of sinonasal, otological and pediatric evaluations related to COVID-19. Results The use of personal protective equipment is considered crucial in routine ENT care. We recommend postponing appointments, exams and elective surgeries to reduce the spread of COVID-19. Similarly, we recommend changing routines in several areas of otolaryngology. Additionally, guidance is provided on the use of telemedicine resources during the pandemic period. Conclusions We are still at the beginning of the COVID-19 pandemic and scientific evidence is still scarce and incomplete, so these ABORL-CCF recommendations for otorhinolaryngologists may be updated based on new knowledge and the pattern of the new coronavirus spread.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0.502559 |
| 86sskox6 | COVID‐19 causes more delay in managed care launch for Illinois youth                                                                                                                                                                                                                                                                                                                                     | The state of Illinois is further delaying the transition to a managed care health coverage system for children in the custody of the Department of Children and Family Services (DCFS), this time citing the COVID‐19 outbreak as the reason for delay, the Southern Illinoisan reported March 16. “ Recognizing the unprecedented challenges families, medical providers, and state agencies are facing from COVID‐19, the launch of YouthCare services for DCFS youth in care will be postponed a minimum of 30 days,” the Illinois Department of Healthcare and Family Services said in a statement. Most Medicaid recipients in Illinois are already under a managed care system, known as IlliniCare. Under that system, the state pays a flat, monthly per‐patient fee to private insurance companies to manage the care of Medicaid patients. The state has been planning for some time to shift foster children and other minors in the custody of DCFS into a managed care system, YouthCare, but that transition was delayed several times. Those delays have mostly been out of concern that there won't be a large enough network of participating health care providers that can serve the unique physical, mental and behavioral health needs of that population. And so, for now, they remain in a traditional fee‐for‐service coverage plan, DHFS and DCFS officials stated.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0.501284 |
| mrsjhjh4 | Estimating Preventable COVID19 Infections Related to Elective Outpatient Surgery in Washington State: A Quantitative Model                                                                                                                                                                                                                                                                               | Background: As the number of suspected and confirmed COVID19 cases in the US continues to rise, the US surgeon general, Centers for Disease Control and Prevention, and several specialty societies have issued recommendations to consider canceling elective surgeries. However, these recommendations have also faced controversy and opposition. Objective: The goal of this study is to provide a quantitative analysis and model for preventable COVID19 infections from elective outpatient or ambulatory surgery cases, which can also be adapted to analyze COVID19 transmission in other healthcare settings. Furthermore, given the controversy over the appropriate handling of elective surgical cases during this pandemic, we hope that our results may have a positive impact on health policy and public health. Methods: Using previously published information on elective ambulatory or outpatient surgical procedures and publicly available data on COVID19 infections in the US and on the Diamond Princess cruise ship, we calculated a transmission rate and generated a mathematical model to predict a lower bound for the number of healthcare-acquired COVID19 infections that could be prevented by canceling or postponing elective outpatient surgeries in Washington state. Results: Our model predicts that over the course of 30 days, at least 2445 preventable patient infections and at least 1557 preventable healthcare worker (HCW) infections would occur in WA state alone if elective outpatient procedures were to continue as usual. The majority of these infections are caused by transmission from HCW who became infected at work. Conclusion: Given the large numbers of COVID19 infections that could be prevented by canceling elective outpatient surgeries, our findings support the recommendations of the US Surgeon General, CDC, American College of Surgeons (ACS), American Society of Anesthesiologists (ASA), and Anesthesia Patient Safety Foundation (APSF) to consider rescheduling or postponing elective surgeries until the COVID19 pandemic is under better control in the US.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.498681 |
| jkd5ld1i | How should front-line general practitioners use personal protective equipment (PPE)?                                                                                                                                                                                                                                                                                                                     | The COVID-19 outbreak continues to evolve with the number of cases increasing in Malaysia, placing a significant burden on general practitioners (GPs) to assess and manage suspected cases. GPs must be well equipped with knowledge to set up their clinics, use Personal Protective Equipment (PPE) appropriately, adopt standard protocols on triaging and referrals, as well as educate patients about PPE. The correct use of PPE will help GPs balance between personal safety and appropriate levels of public concern.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.497793 |
| tyobde1j | How Should U.S. Hospitals Prepare for Coronavirus Disease 2019 (COVID-19)?                                                                                                                                                                                                                                                                                                                               | Estimates suggest that COVID-19 will stress bed capacity, equipment, and health care personnel in U.S. hospitals in ways not previously experienced. How can health systems prepare to care for a large influx of patients with this disease?                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0.497078 |
| t2gm9i0f | Breast radiotherapy under COVID-19 pandemic resource constraints -- approaches to defer or shorten treatment from a Comprehensive Cancer Center in the United States                                                                                                                                                                                                                                     | Abstract Introduction Breast radiotherapy accounts for a significant proportion of patient volume in contemporary radiation oncology practice. In the setting of anticipated resource constraints and widespread community infection with SARS-CoV-2 during the COVID-19 pandemic, measures for balancing both infectious and oncologic risk among patients and providers must be carefully considered. Here, we present evidence-based guidelines for omitting or abbreviating breast cancer radiotherapy, where appropriate, in an effort to mitigate risk to patients and optimize resource utilization. Methods Multidisciplinary breast cancer experts at a high-volume comprehensive cancer center convened contingency planning meetings over the early days of the COVID-19 pandemic to review the relevant literature and establish recommendations for the application of hypofractionated and abbreviated breast radiation regimens. Results Substantial evidence exists to support omitting radiation among certain favorable risk subgroups of breast cancer patients and for abbreviating or accelerating regimens among others. For those who require either whole-breast or post-mastectomy radiation, with or without coverage of the regional lymph nodes, a growing body of literature supports various hypofractionated approaches that appear safe and effective. Conclusion In the setting of a public health emergency with the potential to strain critical healthcare resources and place patients at infection risk, the parsimonious application of breast radiotherapy may alleviate a significant clinical burden without compromising long term oncologic outcomes. The judicious and personalized use of immature study data may be warranted in the setting of a competing mortality risk from this widespread pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.492288 |
| yn87zcpe | Recommendations for the surgical management of gynecological cancers during the COVID-19 pandemic - FRANCOGYN group for the CNGOF                                                                                                                                                                                                                                                                        | Abstract Introduction In the context of the COVID-19 pandemic, specific recommendations are required for the management of patients with gynecologic cancer. Materials and method The FRANCOGYN group of the National College of French Gynecologists and Obstetricians (CNGOF) convened to develop recommendations based on the consensus conference model. Results If a patient with a gynecologic cancer presents with COVID-19, surgical management should be postponed for at least 15 days. For cervical cancer, radiotherapy and concomitant radiochemotherapy could replace surgery as first-line treatment and the value of lymph node staging should be reviewed on a case-by-case basis. For advanced ovarian cancers, neoadjuvant chemotherapy should be preferred over primary cytoreduction surgery. It is legitimate not to perform hyperthermic intraperitoneal chemotherapy during the COVID-19 pandemic. For patients who are scheduled to undergo interval surgery, chemotherapy can be continued and surgery performed after 6 cycles. For patients with early stage endometrial cancer of low and intermediate preoperative ESMO risk, hysterectomy with bilateral adnexectomy combined with a sentinel lymph node procedure is recommended. Surgery can be postponed for 1–2 months in low-risk endometrial cancers (FIGO Ia stage on MRI and grade 1–2 endometrioid cancer on endometrial biopsy). For patients of high ESMO risk, the MSKCC algorithm (combining PET-CT and sentinel lymph node biopsy) should be applied to avoid pelvic and lumbar-aortic lymphadenectomy. Conclusion During the COVID-19 pandemic, management of a patient with cancer should be adapted to limit the risks associated with the virus without incurring loss of chance.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.48722  |
| j7vuoer6 | Recomendaciones para la atención oftalmológica durante el estado de alarma por la pandemia de enfermedad por coronavirus COVID-19                                                                                                                                                                                                                                                                        | ABSTRACT Objective: Minimize exposure to the SARS-CoV-2, reduce the chances of cross-transmission between patients and healthcare personnel, and prevent the development of postoperative complications from the management of patients with eye diseases during the 2019 coronavirus disease pandemic (COVID -19). Methods: COVID-19 literature review and consensus establishment between different Spanish ophthalmology societies in order to provide guidelines and recommendations of maximum resources primarily conditioned by the state of alert, confinement and social distancing that occurs in Spain since March 16, 2020. Results: The recommendations will promote the adoption of action and protection measures for eye care in outpatient clinics, surgical areas and hospitalization, for unconfirmed (asymptomatic and symptomatic) and confirmed COVID-19 patients. Measures must be adapted to the circumstances and availability of Personal Protective Equipment (PPE) in each of the centers and Autonomous Communities, which will be updated according to the pandemic phases and the measures adopted by the Spanish Government. Conclusions: During the COVID-19 pandemic, attention to the potential health risks to the population caused by coronavirus should prevail over the possible progression of the common eye diseases. Ophthalmologists and other eye care professionals must assume a possible progression of these diseases due to the impossibility of adequate patient follow-up.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.486059 |
| by2j7byz | Establishing Telemedicine in an Academic Total Joint Arthroplasty Practice: Needs and Opportunities Highlighted by the COVID-19 Pandemic                                                                                                                                                                                                                                                                 | Abstract The COVID-19 pandemic has prompted rapid restructuring of the healthcare system in an effort to stop the spread of the virus and to treat patients who are acutely ill with COVID-19, while continuing to provide outpatient care for the remainder of patients. To help control spread of this pandemic, many centers have boosted telemedicine capability to care for patients who would typically be seen in person in outpatient settings, including total joint arthroplasty clinics. We review key components relevant to the establishment and effective use of telemedicine, focused on patient education, practice logistics, technological considerations, and sensitive patient health information–associated compliance factors, which are necessary to provide care remotely for total joint arthroplasty patients.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.482977 |
| 423utgpm | Cuidados intensivos durante la epidemia de coronavirus 2019                                                                                                                                                                                                                                                                                                                                              | Resumen El 31 de diciembre de 2019, la Comisión de Salud de la provincia China de Hubei, dio a conocer por primera vez un grupo de casos inexplicables de neumonía, que posteriormente la OMS definió como el nuevo coronavirus de 2019 (SARS-CoV-2). El SARS-CoV-2ha presentado una transmisión rápida de persona a persona y actualmente es una pandemia mundial. En la mayor serie de casos descrita hasta la fecha de pacientes hospitalizados con enfermedad por SARS-CoV-2 (2019-nCoViD), el 26% requirió atención en una unidad de cuidados intensivos (UCI). Esta pandemia está provocando una movilización de la comunidad científica sin precedentes, lo que lleva asociado un numero exponencialmente creciente de publicaciones en relación con la misma. La presente revisión bibliográfica narrativa, tiene como objetivo reunir las principales aportaciones en el área de los cuidados intensivos hasta la fecha en relación con la epidemiología, clínica, diagnóstico y manejo de 2019-nCoViD. Abstract On 31 December 2019, the Health Commission of Hubei Province of China first unveiled a group of unexplained cases of pneumonia, which WHO subsequently defined as the new coronavirus of 2019 (SARS-CoV-2). SARS-CoV-2 has presented rapid person-to-person transmission and is currently a global pandemic. In the largest number of cases described to date of hospitalized patients with SARS-CoV-2 disease (2019-nCoViD), 26% required care in an intensive care unit (ICU). This pandemic is causing an unprecedented mobilization of the scientific community, which has been associated with an exponentially growing number of publications in relation to it. This narrative literature review aims to gather the main contributions in the area of intensive care to date in relation to the epidemiology, clinic, diagnosis and management of 2019-nCoViD.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.481518 |
| dm1wkpnv | Can N95 respirators be reused after disinfection? And for how many times?                                                                                                                                                                                                                                                                                                                                | The Coronavirus Disease 2019 (COVID-19) pandemic has led to a major shortage of N95 respirators, which protect healthcare professionals and the public who may come into contact with the virus. It is necessary to determine the conditions that would allow the safe reuse respirators and personal protection in this crisis. We found that heating (<100 °C) under various humidities (up to 100% RH at 75 °C) and ultraviolet (UV) irradiation were the most promising candidates for mask reuse in the modern hospital infrastructure (up to 20 cycles), when tested on a fabric with particle filtration efficiency ≥95%. Treatments involving certain liquids and vapors may require caution, as steam, alcohol, and bleach all led to degradation in filtration efficiency, leaving the user vulnerable to viral aerosols.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.478325 |
| jfd5gd2p | The COVID-19 Pandemic: Effects on Low- and Middle-Income Countries                                                                                                                                                                                                                                                                                                                                       | Coronavirus Disease 2019 (COVID-19) is spreading rapidly around the world with devastating consequences on patients, health care workers, health systems, and economies. As it reaches low- and middle-income countries, its effects could be even more dire, because it will be difficult for them to respond aggressively to the pandemic. There is a great shortage of all health care providers, who will be at risk due to a lack of personal protection equipment. Social distancing will be almost impossible. The necessary resources to treat patients will be in short supply. The end result could be a catastrophic loss of life. A global effort will be required to support faltering economies and health care systems.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 0.477022 |
| p8cmm6ty | Suggestions on the prevention of COVID-19 for health care workers in department of otorhinolaryngology head and neck surgery                                                                                                                                                                                                                                                                             | Abstract The epidemic of the Coronavirus Disease 2019 (COVID-19) has presented as a grim and complex situation recently. More than 77,000 cases of COVID-19 has been confirmed in China until February 25th, 2020, which are causing great impact on economy and society, as well as seriously interfering with ordinary medical practice in the department of otorhinolaryngology head and neck surgery. This article discussed medical precautions required in the clinic, inpatient ward and operation room of otorhinolaryngology head and neck department, which aims to protect health care workers from COVID-19.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.476328 |
| elbjs7ft | Pharmacy Emergency Preparedness and Response (PEPR) framework for expanding pharmacy professionals’ roles and contributions to emergency preparedness and response during the COVID-19 pandemic and beyond                                                                                                                                                                                               | Abstract Background Pharmacists have long been involved in public health and emergency preparedness and response (EP&R), including through preventive measures such as screening, vaccinations, testing and pharmaceutical countermeasures, as well as ensuring medication safety and access during natural disasters and pandemics. Pharmacy professionals are considered essential partners in response to the ongoing COVID-19 pandemic. Community and hospital pharmacies are expanding services and hours to provide essential services, putting pharmacists and their co-workers at the frontlines for patient care and safety to improve public health. In addition, pharmacy professionals are increasingly integrating into global, national, state and local EP&R efforts, including into interprofessional teams, such as Medical Reserve Corps (MRCs). However, lacunae exist for further integration of pharmacists into public health and safety initiatives. There are increasing opportunities and recommendations that should be expanded upon to provide improved patient care and population health intervention, and to ensure healthcare worker and public health safety. Objective Develop a Pharmacy Emergency Preparedness and Response (PEPR) Framework and recommendations for pharmacy professional pathways towards full integration within public health EP&R efforts (such as the COVID-19 pandemic), and enhanced recognition of pharmacists’ skills, roles and contributions as integral members of the interprofessional healthcare team. Methods This paper draws on the American Society of Health-System Pharmacists (ASHP) 2003 Statement on the Role of Health-System Pharmacists in Emergency Preparedness and lessons learned from previous and current public health emergencies, such as the 2009 H1N1 pandemic and the current COVID-19 pandemic, to provide expanded guidance for pharmacists and pharmacy professionals across all practice settings in EP&R. The PEPR framework also incorporates information and recommendations from The Pharmacy Organizations’ Joint Policy Recommendations to Combat the COVID-19 Pandemic (March 2020), CDC-NIOSH, Health Departments and Emergency Preparedness guidance and resources, Boards of Pharmacy, and other pharmacy professional organizations and educational institutions. Results Based on the methods and resources utilized in developing this PEPR framework, five key focus areas were identified as follow: 1) Emergency preparedness and response 2) Operations management 3) Patient care and population health interventions 4) Public health pharmacy education and continuing professional education 5) Evaluation, research and dissemination for impact and outcomes Conclusion and Recommendations: Pharmacists and pharmacy professionals have been at the frontlines in responding to the COVID-19 pandemic. Yet, challenges remain, such as limited availability of personal protection equipment, high risk of infectious exposures inherent in healthcare professions, and legislative hurdles resulting in lack of provider status and related reimbursements. Recommendations to enhance pharmacy's scope as public health professionals involved in EP&R include targeted training and education on key framework areas and policymaking. Pharmacy professionals should further integrate with interdisciplinary public health teams. Additional research and dissemination on impacts and outcomes of EP&R can enhance recognition of pharmacy professionals' contribution and value during public health emergencies. The PEPR Framework can be utilized to develop, implement, evaluate, and disseminate results in order to strengthen existing efforts and to establish new initiatives in EP&R.                                                                                           | 0.476113 |
| ywgx1xmh | Guidelines of clinical practice for the management of swallowing disorders and recent dysphonia in the contexte of the COVID-19 pandemic                                                                                                                                                                                                                                                                 | ABSTRACT Procedures putting healthcare workers in close contact with the airway are particularly at risk of contamination by the SARS-Cov-2 virus, especially when exposed to sputum, coughing, or a tracheostomy. In the current pandemic phase, all patients should be considered as potentially infected. Thus, the level of precaution recommended for the caregivers depends more on the type of procedure than on the patient's proved or suspected COVID-19 status. Procedures that are particularly at high risk of contamination are clinical and flexible endoscopic pharyngo-laryngological evaluation, and probably also video fluoroscopic swallowing exams. Voice rehabilitation should not be considered urgent at this time. Therefore, recommendations presented here mainly concern the management of swallowing disorders, which can sometimes be dangerous for the patient, and recent dysphonia. In cases where they are considered possible and useful, teleconsultations should be preferred to face-to-face assessments or rehabilitation sessions. The latter must be maintained only in few selected situations, after team discussions or in accordance with the guidelines provided by health authorities.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 0.475342 |
| peo85pdn | Treatment of primary and metastatic peritoneal tumors in the Covid-19 pandemic                                                                                                                                                                                                                                                                                                                           | Abstract The Covid-19 pandemic is profoundly changing the organization of healthcare access. This is particularly so for peritoneal neoplastic diseases, for which curative treatment mobilizes substantial personnel, operating room and intensive care resources. The BIG-RENAPE and RENAPE groups have made tentative proposals for prioritizing care provision. A tightening of the usual selection criteria is needed for curative care: young patients with few or no comorbidities and limited peritoneal extension. It is desirable to prioritize disease conditions for which cytoreduction surgery with or without associated hyperthermic intraoperative peritoneal chemotherapy (HIPEC) is the gold-standard treatment, and for which systemic chemotherapy cannot be a temporary or long-term alternative: pseudomyxoma peritonei, resectable malignant peritoneal mesotheliomas, peritoneal metastases of colorectal origin if they are resectable and unresponsive to systemic chemotherapy after up to 12 courses, first-line ovarian carcinomatosis if resectable or in interval surgery after at most six courses of systemic chemotherapy. Addition of HIPEC must be discussed case by case in an expert center. The prioritization of indications must consider local conditions and the phase of the epidemic to allow optimal peri-operative care.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.474601 |
| iwyn32tv | Medication management and adherence during the COVID-19 pandemic: Perspectives and experiences from LMICs                                                                                                                                                                                                                                                                                                | Abstract The current coronavirus disease 2019 (COVID-19) pandemic is placing a huge strain on health systems worldwide. Suggested solutions like social distancing and lockdowns in some areas to help contain the spread of the virus may affect special patient populations like those with chronic illnesses who are unable to access healthcare facilities for their routine care and medicines management. Retail pharmacy outlets are the likely facilities for easy access by these patients. The contribution of community pharmacists in these facilities in managing chronic conditions and promoting medication adherence during this COVID-19 pandemic will be essential in easing the burden on already strained health systems. This paper highlights the pharmaceutical care practices of community pharmacists for patients with chronic diseases. during this pandemic. This would provide support for the call by the WHO to maintain essential services during the pandemic, in order to prevent non-COVID disease burden on healthcare systems particularly in low-and middle-income countries.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.474441 |
| vfez5k02 | Digital Orthopaedics. A Glimpse into the Future in The Midst of a Pandemic                                                                                                                                                                                                                                                                                                                               | Abstract Background The response to COVID-19 catalyzed the adoption and integration of digital health tools into the healthcare delivery model for musculoskeletal patients. The change, suspension or relaxation of Medicare and federal guidelines enabled the rapid implementation of these technologies. The expansion of payment models for virtual care facilitated its rapid adoption. The authors aim to provide several examples of digital health solutions utilized to manage orthopaedic patients during the pandemic and discuss what features of these technologies are likely to continue to provide value to patients and clinicians following its resolution. Conclusion The widespread adoption of new technologies enabling providers to care for patients remotely has the potential to permanently change the expectations of all stakeholders about the way care is provided in orthopaedics. The new era of Digital Orthopaedics will see a gradual and non-disruptive integration of technologies that support the patient’s journey through the successful management of their musculoskeletal disease.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | 0.470923 |
| xtudcufl | Ramping Up the Delivery of Cardiac Surgery During the COVID-19 Pandemic: A Guidance Statement from the Canadian Society of Cardiac Surgeons                                                                                                                                                                                                                                                              | Abstract The coronavirus disease 2019 (COVID-19) has had a profound global impact. Its rapid transmissibility has forced whole countries to adopt strict measures to contain its spread. As part of necessary pandemic planning, the majority of Canadian cardiac surgical programs have prioritized and delayed elective procedures in an effort to reduce the burden on the health care system and to mobilize resources in the event of a pandemic surge. While the number of COVID-19 cases continue to increase worldwide, new cases have begun to decline in many jurisdictions. This “flattening of the curve” has inevitably prompted discussions around re-opening of the economy, relaxing some public health restrictions and resuming non-urgent health care delivery.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.467717 |
| cxktdaws | MFM Guidance for COVID-19                                                                                                                                                                                                                                                                                                                                                                                | The World Health Organization (WHO) has declared COVID-19 a global pandemic. Healthcare providers should prepare internal guidelines covering all aspect of the organization in order to have their unit ready as soon as possible. This document addresses the current COVID-19 pandemic for maternal-fetal medicine (MFM) practitioners. The goals the guidelines put forth here are two fold- first to reduce patient risk through healthcare exposure, understanding that asymptomatic health systems/healthcare providers may become the most common vector for transmission, and second to reduce the public health burden of COVID-19 transmission throughout the general population. Box 1 outlines general guidance to prevent spread of COVID-19 and protect our obstetric patients. Section 1 outlines suggested modifications of outpatient obstetrical (prenatal) visits. Section 2 details suggested scheduling of obstetrical ultrasound. Section 3 reviews suggested modification of nonstress tests (NST) and biophysical profiles (BPP). Section 4 reviews suggested visitor policy for obstetric outpatient office. Section 5 discusses the role of trainees and medical education in the setting of a pandemic. These are suggestions, which can be adapted to local needs and capabilities. Guidance is changing rapidly, so please continue to watch for updates.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 0.466288 |
| pqld0tyy | A proposal for the return to routine endoscopy during the COVID-19 pandemic                                                                                                                                                                                                                                                                                                                              | Abstract In response to the COVID-19 pandemic, many jurisdictions and gastroenterological societies around the world have suspended nonurgent endoscopy. Subject to country-specific variability, it is projected that with current mitigation measures in place, the peak incidence of active COVID-19 infections may be delayed by over 6 months. Although this aims to prevent the overburdening of healthcare systems, prolonged deferral of elective endoscopy will become unsustainable. Herein, we propose that by incorporating readily available point-of-care tests and conducting accurate clinical risk assessments, a safe and timely return to elective endoscopy is feasible. Our algorithm not only focuses on the safety of patients and healthcare workers, but also assists in rationalizing the use of invaluable resources such as personal protective equipment.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 0.464907 |
| mstpf2s5 | Recomendaciones de la Sociedad Española de Otorrinolaringología y Cirugía de Cabeza y Cuello para la realización de traqueotomías en relación con pacientes infectados por coronavirus COVID-19                                                                                                                                                                                                          | Resumen La reciente pandemia por coronavirus COVID-19 está incrementando el número de pacientes que, debido a su situación ventilatoria pulmonar, pueden requerir de intubación orotraqueal. La infección por coronavirus COVID-19 ha demostrado una alta tasa de transmisibilidad, sobre todo por vía respiratoria y por dispersión de microgotas. La Sociedad Española de Otorrinolaringología y Cirugía de Cabeza y Cuello, basándose en el artículo de Wei et al. de 2003 en relación con las traqueotomías realizadas por el síndrome respiratorio agudo grave (SARS), realiza una serie de recomendaciones para la realización segura de las traqueotomías. Abstract The recent COVID-19 (coronavirus) pandemic is causing an increase in the number of patients who, due to their pulmonary ventilatory status, may require orotracheal intubation. COVID-19 infection has demonstrated a high rate of transmissibility, especially via the respiratory tract and by droplet spread. The Spanish Society of Otolaryngology and Head and Neck Surgery, based on the article by Wei et al. of 2003 regarding tracheotomies performed due to severe acute respiratory syndrome (SARS), has made a series of recommendations for the safe performance of tracheotomies.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | 0.463374 |
| d2bh3tm8 | Assessing the Hospital Surge Capacity of the Kenyan Health System in the Face of the COVID-19 Pandemic                                                                                                                                                                                                                                                                                                   | Introduction The COVID-19 pandemic will test the capacity of health systems worldwide. Health systems will need surge capacity to absorb acute increases in caseload due to the pandemic. We assessed the capacity of the Kenyan health system to absorb surges in the number of people that will need hospitalization and critical care because of the COVID-19. Methods We assumed that 2% of the Kenyan population get symptomatic infection by SARS-Cov-2 based on modelled estimates for Kenya and determined the health system surge capacity for COVID-19 under three transmission curve scenarios, 6, 12, and 18 months. We estimated four measures of hospital surge capacity namely: 1) hospital bed surge capacity 2) ICU bed surge capacity 3) Hospital bed tipping point, and 5) ICU bed tipping point. We computed this nationally and for all the 47 county governments. Results The capacity of Kenyan hospitals to absorb increases in caseload due to COVID-19 is constrained by the availability of oxygen, with only 58% of hospital beds in hospitals with oxygen supply. There is substantial variation in hospital bed surge capacity across counties. For example, under the 6 months transmission scenario, the percentage of available general hospital beds that would be taken up by COVID-19 cases varied from 12% Tharaka Nithi county, to 145% in Trans Nzoia county. Kenya faces substantial gaps in ICU beds and ventilator capacity. Only 22 out of the 47 counties have at least 1 ICU unit. Kenya will need an additional 1,511 ICU beds and 1,609 ventilators (6 months transmission curve) to 374 ICU beds and 472 ventilators (18 months transmission curve) to absorb caseloads due to COVID-19.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.463346 |
| 7xliszyr | Aerosolization of COVID-19 and Contamination Risks During Respiratory Treatments                                                                                                                                                                                                                                                                                                                         | BACKGROUND: Aerosolized medications are frequently administered across the health care continuum to acutely ill patients. During viral pandemics, the World Health Organization and the Centers for Disease Control and Prevention advise the application of airborne precautions when performing aerosol-generating medical procedures, such as aerosolized medications. OBSERVATIONS: Appropriate personal protective equipment (PPE), including fit-tested particulate respirators should be worn when administering nebulized medications to patients. These PPEs have been in short supply in the US during early phases of the COVID-19 pandemic, which is increasing the risk faced by health care workers (HCWs) who are treating patients using aerosolized medications. Despite taking appropriate precautions, HCWs are becoming infected with COVID-19. This may be related to secondary exposure related to viral longevity in fugitive emissions and viability on fomites. CONCLUSIONS: We have expanded on non-US public health recommendations to provide guidance to frontline HCWs to enhance collaboration between clinicians, who are often siloed in their clinical practices, and ultimately to protect the federal workforce, which cannot sustain a significant loss of frontline HCWs.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.456997 |
| i3b647wv | Hospital pharmacists’ pharmaceutical care for hospitalized patients with COVID-19: Recommendations and guidance from clinical experience                                                                                                                                                                                                                                                                 | Abstract Objective To discuss hospital pharmacists’ role in providing pharmaceutical care for hospitalized patients with COVID-19 to promote patient care and management during the pandemic. Method Based on the method of evidence-based pharmacy, clinical evidence of therapeutical drugs for COVID-19 were retrieved and summarized. Based on clinical experience Chinese hospital pharmacists gained from providing pharmaceutical care services during COVID-19 pandemic, taking COVID-19 hospitalized patients’ needs into consideration, the methods and strategies hospital pharmacists shall use to provide pharmaceutical care were analyzed and summarized. Results Hospital pharmacists shall support pharmaceutical care services by participating in making evidence-based decisions for medication, monitoring and evaluation of medication safety and efficacy, providing strengthened care for special population and patients with combined underlying diseases, monitoring and management of convalescent plasma therapy, providing emotional counselling and psychological support, and providing scientific information about COVID-19 vaccines. Conclusion The need of pharmaceutical care services in COVID-19 hospitalized patients during this pandemic was quite distinguished from the past. Hospital pharmacists shall join the collaborative multidisciplinary team to improve COVID-19 patients’ outcome and reduce mortality, and to facilitate the pandemic control.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 0.456326 |
| 0yzrcfe5 | Family-Centered Care During the COVID-19 Era                                                                                                                                                                                                                                                                                                                                                             | Abstract Family support is more, not less, important during crisis. However, during the COVID-19 pandemic, maintaining public safety necessitates restricting the physical presence of families for hospitalized patients. In response, health systems must rapidly adapt family-centric procedures and tools to circumvent restrictions on physical presence. Strategies for maintaining family integrity must acknowledge clinicians’ limited time and attention to devote to learning new skills. Internet-based solutions can facilitate the routine, predictable, and structured communication which is central to family-centered care. But the reliance on technology may compromise patient privacy and exacerbate racial, socioeconomic, and geographic disparities for populations that lack access to reliable internet access, devices or technological literacy. We provide a toolbox of strategies for supporting family-centered inpatient care during physical distancing responsive to the current clinical climate. Innovations in the implementation of family involvement during hospitalizations may lead to long-term progress in the delivery of family-centered care.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0.454235 |
| 4j3fdjlg | Preparing for a COVID-19 pandemic: a review of operating room outbreak response measures in a large tertiary hospital in Singapore                                                                                                                                                                                                                                                                       | The coronavirus disease 2019 (COVID-19) outbreak has been designated a public health emergency of international concern. To prepare for a pandemic, hospitals need a strategy to manage their space, staff, and supplies so that optimum care is provided to patients. In addition, infection prevention measures need to be implemented to reduce in-hospital transmission. In the operating room, these preparations involve multiple stakeholders and can present a significant challenge. Here, we describe the outbreak response measures of the anesthetic department staffing the largest (1,700-bed) academic tertiary level acute care hospital in Singapore (Singapore General Hospital) and a smaller regional hospital (Sengkang General Hospital). These include engineering controls such as identification and preparation of an isolation operating room, administrative measures such as modification of workflow and processes, introduction of personal protective equipment for staff, and formulation of clinical guidelines for anesthetic management. Simulation was valuable in evaluating the feasibility of new operating room set-ups or workflow. We also discuss how the hierarchy of controls can be used as a framework to plan the necessary measures during each phase of a pandemic, and review the evidence for the measures taken. These containment measures are necessary to optimize the quality of care provided to COVID-19 patients and to reduce the risk of viral transmission to other patients or healthcare workers.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.451646 |
| w9a0jopu | From Isolation to Coordination: How Can Telemedicine Help Combat the COVID-19 Outbreak?                                                                                                                                                                                                                                                                                                                  | The rapid spread of Coronavirus disease 2019 (COVID-19) presents China with a critical challenge. As normal capacity of the Chinese hospitals is exceeded, healthcare professionals struggling to manage this unprecedented crisis face the difficult question of how best to coordinate the medical resources used in highly separated locations. Responding rapidly to this crisis, the National Telemedicine Center of China (NTCC), located in Zhengzhou, Henan Province, has established the Emergency Telemedicine Consultation System (ETCS), a telemedicine-enabled outbreak alert and response network. ETCS is built upon a doctor-to-doctor (D2D) approach, in which health services can be accessed remotely through terminals across hospitals. The system architecture of ETCS comprises three major architectural layers: (1) telemedicine service platform layer, (2) telemedicine cloud layer, and (3) telemedicine service application layer. Our ETCS has demonstrated substantial benefits in terms of the effectiveness of consultations and remote patient monitoring, multidisciplinary care, and prevention education and training.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 0.45017  |
| 4t5t8bcx | Thoracic Anesthesia of Patients with Suspected or Confirmed 2019 Novel Coronavirus Infection: Preliminary Recommendations for Airway Management by the EACTA Thoracic Subspecialty Committee                                                                                                                                                                                                             | Abstract The novel coronavirus has caused a pandemic around the world. Management of patients with suspected or confirmed coronavirus infection who have to undergo thoracic surgery will be a challenge for the anesthesiologists. infection who have to undergo thoracic surgery will be a challenge for the anesthesiologists. The thoracic subspecialty committee of European Association of Cardiothoracic Anaesthesiology (EACTA) has conducted a survey of opinion in order to create recommendations for the anesthetic approach to these challenging patients. It should be emphasized that both the management of the infected patient with COVID-19 and the self-protection of the anesthesia team constitute a complicated challenge. The text focuses therefore on both important topics.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 0.449889 |
| n0ieojnp | Stratégie pour la pratique de la chirurgie digestive et oncologique en situation d’épidémie de COVID-19                                                                                                                                                                                                                                                                                                  | Résumé La pandémie due au COVID-19 modifie l’organisation des soins et change la réalisation de la chirurgie digestive. Les priorités sanitaires et les circuits de soins sont modifiés. La chirurgie d’urgence reste prioritaire. Les interventions pour chirurgie fonctionnelles sont à reporter. La chirurgie par laparoscopie doit suivre des règles strictes pour ne pas exposer les professionnels à un surrisque. Le questionnement principal concerne la chirurgie oncologique, opérer ou reporter ? Il existe probablement un sur risque opératoire induit par la pandémie qui doit être mis en balance avec le retard de prise en charge chirurgical. Pour chaque type de cancer, colon, pancréas, œsogastrique, hépatocarcinome, la morbi-mortalité est rappelée et mise en parallèle avec le risque oncologique lié au retard à la chirurgie et/ou au temps de doublement de la tumeur. Cette comparaison permet de proposer des stratégies, ainsi pour les cancers coliques (T1-2, N0), il est souhaitable de retarder la chirurgie. Pour les lésions coliques avancées, il semble prudent de recommander une chimiothérapie néo adjuvante et d’attendre. Pour les cancers du rectum T3-4 et/ou N+, une radio-chimiothérapie est indiquée, une radiothérapie courte devra être discutée (suivie d’une période d’attente) afin de réduire le temps d’exposition à l’hôpital et d’éviter les infections. La majorité des chirurgies complexes à forte morbi-mortalité, œsogastrique, hépatique ou pancréatique doivent sans doute le plus souvent être reportée. Summary The COVID-19 pandemic is changing the organization of healthcare and has a direct impact on digestive surgery. Healthcare priorities and circuits are being modified. Emergency surgery is still a priority. Functional surgery is to be deferred. Laparoscopic surgery must follow strict rules so as not to expose healthcare professionals (HCPs) to added risk. The question looms large in cancer surgery – go ahead or defer? There is probably an added risk due to the pandemic that must be balanced against the risk incurred by deferring surgery. For each type of cancer – colon, pancreas, oesogastric, hepatocellular carcinoma – morbidity and mortality rates are stated and compared with the oncological risk incurred by deferring surgery and/or the tumour doubling time. Strategies can be proposed based on this comparison. For colonic cancers T1-2, N0, it is advisable to defer surgery. For advanced colonic lesions, it seems judicious to undertake neoadjuvant chemotherapy and then wait. For rectal cancers T3–4 and /or N+, chemoradiotherapy is indicated, short radiotherapy must be discussed (followed by a waiting period) to reduce time of exposure in the hospital and to prevent infections. Most complex surgery with high morbidity and mortality – oesogastric, hepatic or pancreatic – is most often best deferred.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 0.448213 |
| jtr0p930 | Recomendaciones en el manejo de la pandemia por coronavirus SARS-CoV-2 (Covid-19) en pacientes con trasplante renal                                                                                                                                                                                                                                                                                      | Resumen La pandemia por coronavirus SARS-CoV-2 (Covid-19) está evolucionando de manera muy rápida y representa un riesgo especial en pacientes inmunodeprimidos y con comorbilidades añadidas. El conocimiento sobre esta infección emergente va también en aumento, si bien, aún sigue habiendo muchas incógnitas, sobre todo en la población con trasplante renal. Este manuscrito presenta una propuesta de actuación con recomendaciones generales y específicas para proteger y prevenir de la infección a esta población tan vulnerable como son los receptores de un trasplante renal. Abstract The SARS-CoV-2 (Covid-19) coronavirus pandemic is evolving very quickly and means a special risk for both immunosuppressed and comorbid patients. Knowledge about this growing infection is also increasing although many uncertainties remain, especially in the kidney transplant population. This manuscript presents a proposal for action with general and specific recommendations to protect and prevent infection in this vulnerable population such as kidney transplant recipients.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.445025 |
| olveh2gf | Antibiotics alone as an alternative to appendectomy for uncomplicated acute appendicitis in adults: changes in treatment modalities related to the COVID-19 health crisis                                                                                                                                                                                                                                | Summary The massive inflow of patients with COVID-19 requiring urgent care has overloaded hospitals in France and impacts the management of other patients. Deferring hospitalization and non-urgent surgeries has become a priority for surgeons today in order to relieve the health care system. It is obviously not simple to reduce emergency surgery without altering the quality of care or leading to a loss of chance for the patient. Acute appendicitis is a very specific situation and the prevalence of this disease leads us to reconsider this particular disease in the context of the COVID-19 crisis. Indeed, while the currently recommended treatment for uncomplicated acute appendicitis is surgical appendectomy, the non-surgical alternative of medical management by antibiotic therapy alone has been widely evaluated by high-quality studies in the literature. Insofar as the main limitation of exclusively medical treatment of uncomplicated acute appendicitis is the risk of recurrent appendicitis, this treatment option represents an alternative of choice to reduce the intra-hospital overload in this context of health crisis. The aim of this work is therefore to provide physicians and surgeons with a practical guide based on a review of the literature on the medical treatment of uncomplicated acute appendicitis in adults, to offer this alternative treatment to the right patients and under good conditions, especially when access to the operating room is limited or impossible.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0.444127 |
| xjco8xb0 | Caring for children and adolescents with eating disorders in the current COVID-19 pandemic: A Singapore perspective                                                                                                                                                                                                                                                                                      | Abstract Our public pediatric tertiary hospital in Singapore has been a part of a robust public heath response to COVID-19 that has been calibrated in a timely manner to the evolving international situation. As of mid-March, Singapore remains in a containment mode with enhanced surveillance and limited community spread. Within this context, our service for pediatric eating disorder care has had to make significant adaptations to our models of service delivery as well as respond to the changing psychosocial needs of our patients. Given infection control requirements, we have instituted modular staffing for our inpatient and outpatient settings, necessitating task shifting and an increased use of technology for communication. Due to reduced outpatient capacity and the need to minimize non-urgent trips to the hospital, we have implemented telemedicine and have leveraged on partnerships with school counselors and other community partners. “ Coronaphobia” has influenced our patients’ willingness to attend visits and worsened existing health anxiety for some. Responsiveness to families’ and patients’ health and financial concerns has been essential. As COVID-19 impacts more countries, our institution’s experience can provide insight into challenges and possible adaptations to providing ongoing care for eating disorder patients in this environment.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.442074 |
| 6e3gpb93 | Guidelines for dental care provision during the COVID-19 pandemic                                                                                                                                                                                                                                                                                                                                        | Abstract Since the coronavirus disease 2019 (COVID-19) outbreak was declared a pandemic on 11 March 2020. Several dental care facilities in affected countries have been completely closed or have been only providing minimal treatment for emergency cases. However, several facilities in some affected countries are still providing regular dental treatment. This can in part be a result of the lack of universal protocol or guidelines regulating the dental care provision during such a pandemic. This lack of guidelines can on one hand increase the nosocomial COVID-19 spread through dental health care facilities, and on the other hand deprive patients’ in need of the required urgent dental care. Moreover, ceasing dental care provision during such a period will incense the burden on hospitals emergency departments already struggle with the pandemic. This work aimed to develop guidelines for dental patients’ management during and after the COVID-19 pandemic. Guidelines for dental care provision during the COVID-19 pandemic were developed after considering the nature of COVID-19 pandemic, and were based on grouping the patients according to condition and need, and considering the procedures according to risk and benefit. It is hoped that the guidelines proposed in this work will help in the management of dental care around the world during and after this COVID-19 pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 0.441934 |
| xs9lot5s | Delivering oxygen-enriched CPAP respiratory support using a non-invasive ventilation device                                                                                                                                                                                                                                                                                                              | Here we report an investigation to adapt existing non-invasive ventilators (NIV) capable of delivering CPAP for use with oxygen to deliver enriched ventilation of 40%+ FiO2. Our intention is to maximise use of existing resources available to HCPs, as NIV and sleep apnoea (CPAP) machines are widely available, to deliver therapeutic benefit and potentially avoid the need for positive pressure ventilation during the COVID-19 pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.439608 |
| fp9q8ayj | N95 Mask Decontamination using Standard Hospital Sterilization Technologies                                                                                                                                                                                                                                                                                                                              | The response to the COVID19 epidemic is generating severe shortages of personal protective equipment around the world. In particular, the supply of N95 respirator masks has become severely depleted with supplies having to be rationed and health care workers having to use masks for prolonged periods in many countries. We sought to test the ability of 4 different decontamination methods including autoclave treatment, ethylene oxide gassing, ionized hydrogen peroxide fogging and vaporized hydrogen peroxide exposure to decontaminate 4 different N95 masks of experimental contamination with SARS-CoV-2 or vesicular stomatitis virus as a surrogate. In addition, we sought to determine whether masks would tolerate repeated cycles of decontamination while maintaining structural and functional integrity. We found that one cycle of treatment with all modalities was effective in decontamination and was associated with no structural or functional deterioration. Vaporized hydrogen peroxide treatment was tolerated to at least 5 cycles by masks. Most notably, standard autoclave treatment was associated with no loss of structural or functional integrity to a minimum of 10 cycles for the 3 pleated mask models. The molded N95 mask however tolerated only 1 cycle. This last finding may be of particular use to institutions globally due to the virtually universal accessibility of autoclaves in health care settings.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.43887  |
| ct9x5lf4 | Mental Health Services in Lombardy during COVID-19 outbreak                                                                                                                                                                                                                                                                                                                                              | Abstract Lombardy is the Region in Italy the most heavily affected by coronavirus disease (COVID-19) contagion. The Regional Health Authority mandates that mental health services should be guaranteed, identifying mental health as a priority for their citizens. Recommendations for occupational and health safety have been provided to patients and hospital staff, including support for telemedicine activities and remote psychosocial interventions. Services of the Mental Health Departments of Milano “Niguarda” and Brescia “Spedali Civili” Hospitals are providing continued care at a community, residential and hospital level, and to positive COVID-19 psychiatric patients in need of hospitalization.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0.438001 |
| 388c0lnb | The Impact of Coronavirus Disease 2019 (COVID-19) on the Practice of Hand Surgery in Singapore                                                                                                                                                                                                                                                                                                           | The coronavirus disease 2019 (COVID-19) pandemic is rapidly evolving. Tan Tock Seng Hospital and the National Centre for Infectious Disease sees 70% of confirmed and suspected cases in Singapore. This article describes the impact of COVID-19 on the practice of hand and reconstructive microsurgery (HRM) in our institution. It details our department’s response as the situation escalated and the impact on the HRM elective and emergency workload, including the use of personal protective equipment on the surgical practice of HRM, as well as the effects of the condition on social and academic life.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 0.434741 |
| ym2s6fd4 | A War on Two Fronts: Cancer Care in the Time of COVID-19                                                                                                                                                                                                                                                                                                                                                 | Initial reports suggest that COVID-19 can be particularly lethal in patients with cancer. This commentary discusses how to balance a delay in cancer diagnosis or treatment against the risk for a potential COVID-19 exposure, mitigate the risks for significant care disruptions associated with social distancing behaviors, and manage the appropriate allocation of limited health care resources in this unprecedented time of health care crisis.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.43317  |
| 1i5arpf8 | Recommendations for Triage, Prioritization and Treatment of Breast Cancer Patients During the COVID-19 Pandemic                                                                                                                                                                                                                                                                                          | Abstract The Severe Acute Respiratory Syndrome Coronavirus 2 (SARS-CoV-2) associated disease (COVID-19) outbreak seriously challenges globally all health care systems and professionals. Expert projections estimate that despite social distancing and lockdown being practiced, we have yet to feel the full impact of COVID-19. In this manuscript we provide guidance to prepare for the impact of COVID-19 pandemic on breast cancer patients and advise on how to triage, prioritize and organize diagnostic procedures, surgical, radiation and medical treatments.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 0.432598 |
| pctp05hp | Strategies for daily operating room management of ambulatory surgery centers following resolution of the acute phase of the COVID-19 pandemic                                                                                                                                                                                                                                                            | Abstract We performed a narrative review to explore the economics of daily operating room management decisions for ambulatory surgery centers following resolution of the acute phase of the Coronavirus Disease 2019 (COVID-19) pandemic. It is anticipated that there will be a substantive fraction of patients who will be contagious, but asymptomatic at the time of surgery. Use multimodal perioperative infection control practices (e.g., including patient decontamination) and monitor performance (e.g., S. aureus transmission from patient to the environment). The consequence of COVID-19 is that such processes are more important than ever to follow because infection affects not only patients but the surgery center staff and surgeons. Dedicate most operating rooms to procedures that are not airway aerosol producing and can be performed without general anesthesia. Increase throughput by performing nerve blocks before patients enter the operating rooms. Bypass the phase I post-anesthesia care unit whenever possible by appropriate choices of anesthetic approach and drugs. Plan long-duration workdays (e.g., 12-h). For cases where the surgical procedure does not cause aerosol production, but general anesthesia will be used, have initial (phase I) post-anesthesia recovery in the operating room where the surgery was done. Use anesthetic practices that achieve fast initial recovery of the brief ambulatory cases. When the surgical procedure causes aerosol production (e.g., bronchoscopy), conduct phase I recovery in the operating room and use multimodal environmental decontamination after each case. Use statistical methods to plan for the resulting long turnover times. Whenever possible, have the anesthesia and nursing teams stagger cases in more than one room so that they are doing one surgical case while the other room is being cleaned. In conclusion, this review shows that COVID-19 will markedly affect daily ambulatory workflow for patients undergoing general anesthesia, with potentially substantial economic impact for some surgical specialties.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.432329 |
| 67pfen3n | Clinical Guidance for the Management of Patients with Urothelial Cancers During the COVID-19 Pandemic – Rapid Review                                                                                                                                                                                                                                                                                     | Abstract The current COVID-19 pandemic presents a substantial obstacle to cancer patient care. Data from China as well as risk models suppose that cancer patients, particularly those on active, immunosuppressive therapies are at higher risks of severe infection from the illness. In addition, staff illness and restructuring of services to deal with the crisis will inevitably place treatment capacities under significant strain. These guidelines aim to expand on those provided by NHS England regarding cancer care during the coronavirus pandemic by examining the known literature and provide guidance in managing patients with urothelial and rarer urinary tract cancers. In particular, they address the estimated risk and benefits of standard treatments and consider the alternatives in the current situation. As a result, it is recommended that this guidance will help form a framework for shared decision making with patients. Moreover, they do not advise a one-size-fits-all approach but recommend continual assessment of the situation with discussion within and between centres.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0.431516 |
| j3kh7tou | Organisation d’un service de radiothérapie pendant l’épidémie de COVID-19 : expérience du centre hospitalier de Mulhouse                                                                                                                                                                                                                                                                                 | Résumé L’épidémie de COVID-19 continue de croître de manière exponentielle dans notre pays. Si la majorité des formes sont bénignes, les patients atteints de cancer sont à risque de voir se développer une forme grave de la maladie. Les services de radiothérapie sont un lieu à potentiel de contamination en raison du nombre de patients traités et de personnels présent. Leur organisation pendant la période épidémique vise à assurer la continuité des soins tout en limitant le risque de décès dû à une contamination par le SARS-CoV-2 (virus responsable de la COVID-19). Dans le service de radiothérapie du groupe hospitalier de la région de Mulhouse et Sud-Alsace, cette organisation s’articule en cinq points : la protection des personnels médicaux et paramédicaux, la protection des patients en cours de traitement, la détection des patients suspects d’être atteints de COVID-19 et leur prise en charge, la réorganisation du circuit patient et les mesures concernant l’organisation du système qualité du service. Nos pistes de réflexion, débutée dès le début de l’épidémie dans notre département, nous permettent de préserver au maximum l’accès aux soins radiothérapiques en anticipant le risque de diffusion du virus. Grâce à des réunions bihebdomadaires, nous continuons à nous adapter à l’évolution épidémique dans notre service, en tenant compte de nos moyens matériels. La possibilité de réaliser des tests de diagnostic chez tous les patients suspects nous permettrait également d’affiner nos procédures. Summary The COVID-19 outbreak grows exponentially in our country. Despite most of patients develops benign symptoms, cancer patients are at risk of a severe form of the disease. Radiotherapy centres are a potential contamination place due to the number of patients treated and staff present. Their organization during the outbreak period aims to ensure continuity of care while limiting the risk of death from COVID-19. In the radiotherapy department of Mulhouse hospital (France), we pointed five points out: protection of medical and paramedical staff, protection of patients undergoing treatment, detection of patients suspected of being infected by SARS-CoV-2 and their management, reorganization of the patient circuit and measures regarding the quality management. This reflection, which began at the beginning of the outbreak in our city, allows us to preserve the access to radiotherapy treatments by anticipating the risk of spreading the virus. Through biweekly meetings, we continue to adapt to the epidemic in our department, considering our material resources. The ability to perform diagnostic tests in all suspect patients would also allow us to refine our procedures.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.429025 |
| lsvdowvj | Pediatric Airway Management in Coronavirus Disease 2019 Patients: Consensus Guidelines From the Society for Pediatric Anesthesia’s Pediatric Difficult Intubation Collaborative and the Canadian Pediatric Anesthesia Society                                                                                                                                                                            | The severe acute respiratory syndrome Coronavirus 2 (Coronavirus Disease 2019 [COVID-19]) pandemic has challenged medical systems and clinicians globally to unforeseen levels. Rapid spread of COVID-19 has forced clinicians to care for patients with a highly contagious disease without evidence-based guidelines. Using a virtual modified nominal group technique, the Pediatric Difficult Intubation Collaborative (PeDI-C), which currently includes 35 hospitals from 6 countries, generated consensus guidelines on airway management in pediatric anesthesia based on expert opinion and early data about the disease. PeDI-C identified overarching goals during care, including minimizing aerosolized respiratory secretions, minimizing the number of clinicians in contact with a patient, and recognizing that undiagnosed asymptomatic patients may shed the virus and infect health care workers. Recommendations include administering anxiolytic medications, intravenous anesthetic inductions, tracheal intubation using video laryngoscopes and cuffed tracheal tubes, use of in-line suction catheters, and modifying workflow to recover patients from anesthesia in the operating room. Importantly, PeDI-C recommends that anesthesiologists consider using appropriate personal protective equipment when performing aerosol-generating medical procedures in asymptomatic children, in addition to known or suspected children with COVID-19. Airway procedures should be done in negative pressure rooms when available. Adequate time should be allowed for operating room cleaning and air filtration between surgical cases. Research using rigorous study designs is urgently needed to inform safe practices during the COVID-19 pandemic. Until further information is available, PeDI-C advises that clinicians consider these guidelines to enhance the safety of health care workers during airway management when performing aerosol-generating medical procedures. These guidelines have been endorsed by the Society for Pediatric Anesthesia and the Canadian Pediatric Anesthesia Society.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.427671 |
| vsinwqnr | Intensive care management of coronavirus disease 2019 (COVID-19): challenges and recommendations                                                                                                                                                                                                                                                                                                         | Summary As coronavirus disease 2019 (COVID-19) spreads across the world, the intensive care unit (ICU) community must prepare for the challenges associated with this pandemic. Streamlining of workflows for rapid diagnosis and isolation, clinical management, and infection prevention will matter not only to patients with COVID-19, but also to health-care workers and other patients who are at risk from nosocomial transmission. Management of acute respiratory failure and haemodynamics is key. ICU practitioners, hospital administrators, governments, and policy makers must prepare for a substantial increase in critical care bed capacity, with a focus not just on infrastructure and supplies, but also on staff management. Critical care triage to allow the rationing of scarce ICU resources might be needed. Researchers must address unanswered questions, including the role of repurposed and experimental therapies. Collaboration at the local, regional, national, and international level offers the best chance of survival for the critically ill.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 0.422507 |
| 465ltmml | On the frontline against COVID-19: Community pharmacists’ contribution during a public health crisis                                                                                                                                                                                                                                                                                                     | Abstract The global spread of COVID-19 is placing unprecedented demands on healthcare services. In this time of crisis, innovative and adaptive methods of practising will be required across all health professions. In order to maximise the use of current available resources, it is vital that existing services are comprehensively reviewed and full use is made of any unrealised potential among healthcare providers. Community pharmacy is one of a number of health professions that has a key role to play in responding to the current pandemic. As the scope of community pharmacy practice varies considerably across countries, it is important to examine ways in which the profession can assist with the public health response to COVID-19 and maintaining the continuity of healthcare services. This article seeks to highlight roles and activities that community pharmacists can undertake to help in relieving pressure on other areas of the health service, such as general practice. This information could help to inform future decisions about the restructuring of existing health services by governments, public health bodies and policy makers in response to public health crises such as COVID-19.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | 0.419508 |
| d1crwbuz | Aerosolized Hydrogen Peroxide Decontamination of N95 Respirators, with Fit-Testing and Virologic Confirmation of Suitability for Re-Use During the COVID-19 Pandemic                                                                                                                                                                                                                                     | In response to the current urgent demand for N95 respirators by healthcare workers responding to the COVID-19 pandemic, with particular emphasis on needs within local medical systems, we initiated an N95 decontamination study using aerosolized hydrogen peroxide or aHP (7% H2O2 solution), via the Pathogo Curis® (Curis) decontamination system. The study has thus far included 10 cycles of respirator decontamination, with periodic qualitative and quantitative fit testing to verify ongoing respirator integrity through the decontamination process, and support a statistical evaluation of successful respirator fit. In addition, we have conducted virologic testing of respirator surfaces and materials to demonstrate a rigorous verification of decontamination. Given that the current pandemic entails a respiratory viral pathogen, it is critical to address these aspects of respirator safety for reuse. These measures are intended to provide a foundation for a suitable decontamination process, which maintains N95 function, and supports safe respirator reuse by healthcare providers. Current results from both respirator fit testing and virologic testing indicate that the process is effective on the basis of zero failure rate on fit-testing of selected respirators, and on complete decontamination of multiple virus species by aHP treatment, comparable to that observed with commercial spore-based biological indicators of sterilization.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.418121 |
| cr2tbe73 | In states with prominent virus crisis, door wide open for move to telehealth                                                                                                                                                                                                                                                                                                                             | Amid ever‐changing policy developments in response to the COVID‐19 crisis, many states have identified loosening restrictions on telehealth services as a front‐burner item. This has allowed mental health provider agencies, particularly in the states hardest hit so far by the virus that causes COVID‐19, to move more quickly to transition in‐person outpatient services to telehealth.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.415977 |
| axijpn4y | A COVID-19 Infection Risk Model for Frontline Health Care Workers                                                                                                                                                                                                                                                                                                                                        | The number of confirmed COVID-19 cases admitted in hospitals is continuously increasing in the Philippines. Frontline health care workers are faced with imminent risks of getting infected. In this study, we formulate a theoretical model to calculate the risk of being infected in health care facilities considering the following factors: the average number of encounters with a suspected COVID-19 patient per hour; interaction time for each encounter; work shift duration or exposure time; crowd density, which may depend on the amount of space available in a given location; and availability and effectiveness of protective gears and facilities provided for the frontline health care workers. Based on the simulation results, we recommend the following: (i) decrease the rate of patient encounter per frontline health care worker, e.g., maximum of three encounters per hour in a 12-hour work shift duration; (ii) decrease the interaction time between the frontline health care worker and the patients, e.g., less than 40 minutes for the whole day; (iii) increase the clean and safe space for social distancing, e.g., maximum of 10% crowd density, and if possible, implement compartmentalization of patients; and/or (iv) provide effective protective gears and facilities, e.g., 95% effective, that the frontline health care workers can use during their shift. Moreover, the formulated model can be used for other similar scenarios, such as identifying infection risk in public transportation, school classroom settings, offices, and mass gatherings.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0.411855 |
| u2upcaod | Potential magnitude of COVID-19-induced healthcare resource depletion in Ontario, Canada                                                                                                                                                                                                                                                                                                                 | Background: The global spread of coronavirus disease 2019 (COVID-19) continues in several jurisdictions, causing significant strain to healthcare systems. The purpose of our study is to predict the impact of the COVID-19 pandemic on patient outcomes and the healthcare system in Ontario, Canada. Methods: We developed an individual-level simulation to model the flow of COVID-19 patients through the Ontario healthcare system. We simulated different combined scenarios of epidemic trajectory and healthcare capacity. Outcomes include numbers of patients needing admission to the ward, Intensive Care Unit (ICU), and requiring ventilation; days to resource depletion; and numbers of patients awaiting resources and deaths associated with limited access to resources. Findings: We demonstrate that with effective early public health measures system resources need not be depleted. For scenarios considering late or ineffective implementation of physical distancing, health system resources would be depleted within 14-26 days. Resource depletion was also avoided or delayed with aggressive measures to rapidly increase ICU, ventilator, and acute care hospital capacity. Interpretation: We found that without aggressive physical distancing measures the Ontario healthcare system would have been inadequately equipped to manage the expected number of patients with COVID-19, despite the rapid capacity increase. This overall lack of resources would have led to an increase in mortality. By slowing the spread of the disease via ongoing public health measures and having increased healthcare capacity, Ontario may have avoided catastrophic stresses to its health care system.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 0.411435 |
| dvrstz6f | Estimating the burden of United States workers exposed to infection or disease: a key factor in containing risk of COVID-19 infection                                                                                                                                                                                                                                                                    | Introduction: With the global spread of COVID-19, there is a compelling public health interest in quantifying who is at increased risk of disease. Occupational characteristics, such as interfacing with the public and being in close quarters with other workers, not only put workers at high risk for disease, but also make them a nexus of disease transmission to the community. This can further be exacerbated through presenteeism, the term used to describe the act of coming to work despite being symptomatic for disease. Understanding which occupational groups are exposed to infection and disease in the workplace can help to inform public health risk response and management for COVID-19, and subsequent infectious disease outbreaks. Methods: To estimate the burden of United States workers exposed to infection and disease in the workplace, national employment data (by Standard Occupational Classification) maintained by the Bureau of Labor Statistics (BLS) was merged with BLS O*NET survey data, which ranks occupations with particular physical, ergonomic, and structural exposures. For this analysis, occupations reporting exposure to infection or disease more than once a month was the focus. Results: Based on our analyses, approximately 10% (14.4 M) of United States workers are employed in occupations where exposure to disease or infection occurs at least once per week. Approximately 18.4% (26.7 M) of all United States workers are employed in occupations where exposure to disease or infection occurs at least once per month. While the majority of exposed workers are employed in healthcare sectors, other occupational sectors also have high proportions of exposed workers. These include protective service occupations (e.g. police officers, correctional officers, firefighters), office and administrative support occupations (e.g. couriers and messengers, patient service representatives), education occupations (e.g. preschool and daycare teachers), community and social services occupations (community health workers, social workers, counselors), and even construction and extraction occupations (e.g. plumbers, septic tank installers, elevator repair). Conclusions: The large number of persons employed in a wide variety of occupations with frequent exposure to infection and disease underscore the importance of all workplaces developing risk response plans for COVID-19. This work also serves as an important reminder that the workplace is a key locus for public health interventions, which could protect both workers and the communities they serve.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.410958 |
| 7eed7ifd | Annotation: The COVID-19 pandemic and clinical orthopaedic and trauma surgery                                                                                                                                                                                                                                                                                                                            | Abstract This article provides a brief overview of the current COVID-19 pandemic crisis and the impact on trauma and orthopaedic surgeons. The principles of protect, avoid, restrict and abbreviate are recommended. Coordination of response, communication and support are also important. The versatility of orthopaedic surgeons lends them to having an important role.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0.41014  |
| djjarc0f | Practical Steps to Improve Air Flow in Long-Term Care Resident Rooms to Reduce COVID-19 Infection Risk                                                                                                                                                                                                                                                                                                   | Abstract The potential for spread of COVID-19 infections in Skilled Nursing Facilities and other Long-Term Care sites poses new challenges for Nursing Home Administrators to protect patients and staff. It is anticipated that as Acute Care Hospitals reach capacity, Nursing Homes may retain COVID-19 infected residents longer prior to transferring to an Acute Care Hospital. This article outlines 5 pragmatic steps that Long-Term Care facilities can take to manage airflow within resident rooms to reduce the potential for spread of infectious airborne droplets into surrounding areas including hallways and adjacent rooms, using strategies adapted from negative pressure isolation rooms in acute care facilities.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.409863 |
| wwdlqd98 | Recomendaciones de “hacer” y “no hacer” en el tratamiento de los pacientes críticos ante la pandemia por coronavirus causante de COVID-19 de los Grupos de Trabajo de la Sociedad Española de Medicina Intensiva, Crítica y Unidades Coronarias (SEMICYUC)◊                                                                                                                                              | Abstract On March 11, 2020, the Director-General of the World Health Organization (WHO) declared the disease caused by SARS-CoV-2 (COVID-19) as a pandemic. The spread and evolution of the pandemic is overwhelming the healthcare systems of dozens of countries and has led to a myriad of opinion papers, contingency plans, case series and emerging trials. Covering all this literature is complex. Briefly and synthetically, in line with the previous recommendations of the Working Groups, the Spanish Society of Intensive, Critical Medicine and Coronary Units (SEMICYUC) has prepared this series of basic recommendations for patient care in the context of the pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 0.409082 |
| tnctqc1a | Recomendaciones sobre el manejo clínico de la infección por el «nuevo coronavirus» SARS-CoV2. Grupo de trabajo de la Asociación Española de Pediatría (AEP)                                                                                                                                                                                                                                              | Resumen El 31 de diciembre de 2019, la Comisión Municipal de Salud y Sanidad de Wuhan (provincia de Hubei, China) informó sobre la existencia de 27 casos de neumonía de etiología desconocida con inicio de síntomas el 8 de diciembre, incluyendo 7 casos graves, con exposición común a un mercado de marisco, pescado y animales vivos en la ciudad de Wuhan. El 7 de enero de 2020, las autoridades chinas identificaron como agente causante del brote un nuevo tipo de virus de la familia Coronaviridae, denominado temporalmente «nuevo coronavirus», 2019-nCoV. El 30 de enero de 2020 la Organización Mundial de la Salud (OMS) declara el brote una Emergencia Internacional. El día 11 de febrero la OMS le asigna el nombre de SARS-CoV2 e infección COVID-19 (Coronavirus Infectious Disease). El Ministerio de Sanidad convoca a las Sociedades de Especialidades para la elaboración de un protocolo clínico de manejo de la infección. La Asociación Española de Pediatría nombra un grupo de trabajo de las Sociedades de Infectología Pediátrica y Cuidados Intensivos Pediátricos que se encargan de elaborar las presentes recomendaciones con la evidencia disponible en el momento de su realización. Abstract On 31 December 2019, the Wuhan Municipal Committee of Health and Healthcare (Hubei Province, China) reported that there were 27 cases of pneumonia of unknown origin with symptoms starting on the 8 December. There were 7 serious cases with common exposure in market with shellfish, fish, and live animals, in the city of Wuhan. On 7 January 2020, the Chinese authorities identified that the agent causing the outbreak was a new type of virus of the Coronaviridae family, temporarily called «new coronavirus», 2019-nCoV. On January 30th, 2020, the World Health Organisation (WHO) declared the outbreak an International Emergency. On 11 February 2020 the WHO assigned it the name of SARS-CoV2 and COVID-19 (SARS-CoV2 and COVID-19). The Ministry of Health summoned the Specialties Societies to prepare a clinical protocol for the management of COVID-19. The Spanish Paediatric Association appointed a Working Group of the Societies of Paediatric Infectious Diseases and Paediatric Intensive Care to prepare the present recommendations with the evidence available at the time of preparing them.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.407948 |
| mo0srx47 | Impacto de la pandemia COVID-19 en el servicio de urología de un centro de referencia en la Comunidad de Madrid                                                                                                                                                                                                                                                                                          | Abstract The COVID-19 pandemic caused by the SARS-CoV-2 virus has caused tens of thousands of deaths in Spain and has managed to breakdown the healthcare system hospitals in the Community of Madrid, largely due to its tendency to cause severe pneumonia, requiring ventilatory support. This fact has caused our center to collapse, with 130% of its beds occupied by COVID-19 patients, thus causing the absolute cessation of activity of the urology service, the practical disappearance of resident training programs, and the incorporation of a good part of the urology staff into the group of medical personnel attending these patients. In order to recover from this extraordinary level of suspended activity, we will be obliged to prioritize pathologies based on purely clinical criteria, for which tables including the relevance of each pathology within each area of urology are being proposed. Technology tools such as online training courses or surgical simulators may be convenient for the necessary reestablishment of resident education.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | 0.406029 |
| q923fclz | Coronavirus in Italy—Report From the Front Lines | Critical Care Medicine | JN Learning | AMA Ed Hub                                                                                                                                                                                                                                                                                                     | Physicians in Lombardy, Italy, have been overwhelmed by COVID-19 patients requiring critical care. Based on an existing ECMO center network they developed an ICU network to rapidly identify, triage, and manage patients infected with SARS-2-CoV. Maurizio Cecconi, MD, of Humanitas University in Milan discusses the region’s approach to the surge, including clinical and supply management, health care worker training and protection, and ventilation strategies, with JAMA Editor Howard Bauchner.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0.40231  |
| g6zeoth6 | Evaluation of the Anticipated Burden of COVID-19 on Hospital-Based Healthcare Services Across the United States                                                                                                                                                                                                                                                                                          | Background: Coronavirus disease-19 (COVID-19) is a global pandemic, with the potential to infect nearly 60% of the population. The anticipated spread of the virus requires an urgent appraisal of the capacity of US healthcare services and the identification of states most vulnerable to exceeding their capacity Methods: In the American Hospital Association survey for 2018, a database of US community hospitals, we identified total inpatient beds, adult intensive care unit (ICU) beds, and airborne isolation rooms across all hospitals in each state of continental US. The burden of COVID-19 hospitalizations was estimated based on a median hospitalization duration of 12 days and was evaluated for a 30-day reporting period. Results: At 5155 US community hospitals across 48 states in the contiguous US and Washington DC, there were a total of 788,032 inpatient beds, 68,280 adult ICU beds, and 44,222 isolation rooms. The median daily bed occupancy was 62.8% (IQR 58.1%, 66.6%) across states. Nationally, for every 10,000 individuals, there are 24.2 inpatient beds, 2.8 adult ICU beds, and 1.4 isolation beds. There is a 3-fold variation in the number of inpatient beds available across the US, ranging from 16.4 per 10,000 in Oregon to 47 per 10,000 in South Dakota. There was also a similar 3-fold variation in available or non-occupied beds, ranging from 4.7 per 10,000 in Connecticut through 18.3 per 10,000 in North Dakota. The availability of ICU beds is low nationally, ranging from 1.4 per 10,000 in Nevada to 4.7 per 10000 in Washington DC. Hospitalizations for COVID-19 in a median 0.2% (IQR 0.2 %, 0.3%) of state population, or 1.4% of state's older adults (1.0%, 1.9%) will require all non-occupied beds. Further, a median 0.6% (0.5%, 0.8%) of state population, or 3.9% (3.1%, 4.6%) of older individuals would require 100% of inpatient beds. Conclusion: The COVID-19 pandemic is likely to overwhelm the limited number of inpatient and ICU beds for the US population. Hospitals in half of US states would exceed capacity if less than 0.2% of the state population requires hospitalization in any given month.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.402199 |
| ocpdwzrd | Analysis of SteraMist ionized hydrogen peroxide technology as a method for sterilizing N95 respirators and other personal protective equipment                                                                                                                                                                                                                                                           | The COVID-19 pandemic has led to widespread shortages of personal protective equipment (PPE) for healthcare workers, including filtering facepiece respirators (FFRs) such as N95 masks. These masks are normally intended for single use, but their sterilization and subsequent reuse could substantially mitigate a world-wide shortage. In this study, we investigate PPE sterilization using ionized hydrogen peroxide (iHP), generated by SteraMist® equipment (TOMI; Frederick, MD), in a sealed environment chamber installed in the animal facility of an academic medical center. Following one to five sterilization cycles, five N95 mask models from three manufacturers were evaluated using three criteria: efficiency of iHP in sterilization (measured with bacterial spores in standard biological indicator assemblies), ability of sterilized masks to efficiently filter particles down to 0.3μm, and a quantitative fit test. Filtration efficiency was measured using ambient particulate matter at a university lab and an aerosolized NaCl challenge at a National Institute for Occupational Safety and Health (NIOSH) pre-certification laboratory. The data demonstrate that N95 masks sterilized using SteraMist iHP technology retain function up to five cycles, the maximum number tested. Some but not all PPE could also be sterilized in the same manner, and pre-treatment with a handheld iHP generator was required for semi-enclosed surfaces such as respirator hoses. A typical environment chamber with a volume of ~80 m3 can treat ~7000 masks per day, and as well as other items of PPE, making this an effective approach for a busy medical center.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.400238 |
| l0ggbztj | Recomendaciones sobre el tratamiento antitrombótico durante la pandemia COVID-19. Posicionamiento del Grupo de Trabajo de Trombosis Cardiovascular de la Sociedad Española de Cardiología                                                                                                                                                                                                                | RESUMEN La pandemia producida por la infección del nuevo coronavirus SARS-CoV-2, que da lugar a una enfermedad altamente contagiosa (COVID-19), ha producido un colapso de los sistemas sanitarios de todo el mundo. Se ha descrito que estos pacientes sufren un estado inflamatorio que condiciona un alto riesgo trombótico. Sin embargo, apenas hay información sobre cómo abordar el riesgo trombótico, la coagulopatía y el tratamiento anticoagulante de estos pacientes. Por otra parte, incluso los pacientes no infectados por COVID-19 sufren una tremenda influencia en su abordaje habitual por la situación sanitaria actual. El objetivo del presente documento, elaborado por el Grupo de Trabajo de Trombosis Cardiovascular de la Sociedad Española de Cardiología, es presentar la información disponible y dar unas pautas sencillas de tratamiento con fármacos antitrombóticos. ABSTRACT The new coronavirus SARS-CoV-2, which gives rise to the highly contagious COVID-19 disease, has caused a pandemic that is overwhelming health care systems worldwide. Affected patients have been reported to have a heightened inflammatory state that increases their thrombotic risk. However, there is very scarce information on the management of thrombotic risk, coagulation disorders, and anticoagulant therapy. In addition, the situation has also greatly influenced usual care in patients not infected with COVID-19. This article by the Working Group on Cardiovascular Thrombosis of the Spanish Society of Cardiology aims to summarize the available information and to provide a practical approach to the management of antithrombotic therapy.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.397841 |
| xn8jzzsd | COVID-19 pandemic and personal protective equipment shortage: protective efficacy comparing masks and scientific methods for respirator reuse                                                                                                                                                                                                                                                            | Abstract Background and Aims The abrupt outbreak of COVID-19 and its rapid spread over many health care systems in the world led to personal protective equipment (PPE) shortening, which cannot be faced only by the reduction in their consumption nor by the expensive and time-requiring implementation of their production. It is thus necessary to promote PPE rational use, highlighting possible differences in terms of efficacy among them and promoting an effective technique to reuse them. Methods A literature search was performed on PubMed, Scopus, Cochrane database, and Google Scholar and from 25 top cited papers, 15 were selected for relevance and impact. Results Most studies on prior respiratory virus epidemic to date suggest surgical masks not to be inferior compared with N95 respirators in terms of protective efficacy among health care workers. The use of N95 respirators should be then limited in favor of high-risk situations. Concerning respirators reuse, highly energetic short-wave ultraviolet germicidal irradiation (UVGI) at 254 nm was proficiently applied to determine N95 respirators decontamination from viral respiratory agents, but it requires careful consideration of the type of respirator and of the biological target. Conclusions Rational use and successful reuse of respirators can help facing PPE shortening during a pandemic. Further evidences testing UVGI and other decontamination techniques are an unmet need. The definitive answer to pandemic issues can be found in artificial intelligence and deep learning: these groundbreaking modalities could help in identifying high-risk patients and in suggesting appropriate types and use of PPE.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.394334 |
| wskmzg7d | Personal Protective Equipment: Current Best Practices for Orthopaedic Teams                                                                                                                                                                                                                                                                                                                              | Abstract The COVID-19 pandemic caused by the SARS-CoV-2 virus is challenging healthcare providers across the world. Current best practices for personal protective equipment (PPE) during this time are rapidly evolving and fluid due to the novel and acute nature of the pandemic and the dearth of high-level evidence. Routine infection control practices augmented by airborne precautions are paramount when treating the COVID-19 positive patient. Best practices for PPE use in patients who have unknown COVID-19 status are a highly charged and emotional issue. The variables to be considered include protection of patients and healthcare providers, accuracy and availability of testing, and responsible use of PPE resources. This article also explores the concerns of surgeons regarding possible transmission to their own family members as a result of caring for COVID-19 patients.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.393785 |
| hwvb979y | Strategies for the reuse of N95 and N99 respiratory masks during the COVID-19 pandemic                                                                                                                                                                                                                                                                                                                   | The COVID-19 pandemic presents a strain of unprecedented scale on health systems around the world. In order to reliably protect medical personnel, and thus to contain the spread of the pandemic, it is essential to provide N95 or N99 (European FFP2 or FFP3) respiratory masks (FFRs). Such masks are currently in extreme shortage: To guarantee their supply sufficiently and for all cases, it would absolutely necessary to reuse them. In recent years, the scientific literature has laid out various possibilities to disinfect the FFRs for their reuse several times. We identify the most promising disinfection methods for the current critical situation (internationally) and project further methods and modifications beyond these.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 0.392092 |
| 4bncbqsb | How Has COVID-19 affected Our Orthopaedic Implant Industry Partners? Implications for the Surgeon-Industry Relationship in 2020 and Beyond                                                                                                                                                                                                                                                               | Abstract Introduction The COVID-19 pandemic has had far-reaching societal and financial consequences. The purpose of this study was to evaluate how COVID-19 has affected AAHKS Industry Partners and the Surgeon-Industry relationship, emphasizing education, resource allocation, and strategic direction for the 2nd half of 2020. Methods AAHKS Industry Partners were contacted to participate in a blinded survey and optional interview with the AAHKS Industry Relations Committee. Based on the results, a group of AAHKS Member Surgeons with disparate practice types were asked to postulate on how the COVID-19 pandemic has and will affect their practice and relationship with Industry. Results AAHKS Industry Partner responses indicated decreased resource allocation for Regional, ‘Other National,’ and AAHKS Annual meetings (67%, 55%, and 30% respectively). Web-based educational content was expected to increase in 2020 and will likely remain a point of emphasis in 2021 (100% and 70% of responders). For Q3/Q4 2020, a significant emphasis was placed on Site of Service/Outpatient TJA and Covid-19 related Safety Measures (70% and 90% of responders), as well as increased availability of instrumentation and implants (40%, 60% respectively). Discussion The COVID-19 Pandemic has altered the orthopaedic landscape for the foreseeable future. Survey responses by AAHKS Industry Partners demonstrate a continued commitment to surgeon education with and increasing shift to a web-based platform. Increased resource allocation for outpatient TJA and COVID-19 related safety measures were significant. Articulating optimal mechanisms to aid Industry in supporting surgeons with different practice models to meet demand during the second half of fiscal year 2020 will be critical.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 0.389026 |
| u15wuwkt | The prevention of nosocomial SARS-CoV2 transmission in endoscopy: a systematic review of recommendations within gastroenterology to identify best practice.                                                                                                                                                                                                                                              | Endoscopy generates aerosol droplets and fomites, thereby increasing the risk of SARS-CoV2 transmission to healthcare workers and uninfected patients within endoscopy departments. Despite the sharp rise in the incidence of COVID-19, authoritative recommendations to limit the spread of SARS-CoV2 within gastrointestinal endoscopy units are lacking. Therefore, with the primary aim of identifying best practice and scrutinizing its supporting evidence, we conducted a systematic review of literature for articles published between 1 January 2002 and 15 March 2020 in five databases relating to both the current SARS-CoV2 and the previous SARS-CoV outbreaks. Official websites for gastroenterology and endoscopy societies in the 15 most affected countries were also searched. Unfortunately, a paucity of high quality data and heterogeneity of recommendations between countries was observed. Interestingly, not all countries advocated the postponement of non-urgent or elective procedures. Recommendations for patient screening and personal protective equipment were commonly featured in all recommendations but specifics varied. Only 32% (9/28) of all gastroenterology and endoscopy societies issued guidance on endoscopy in the COVID-19 pandemic. In conclusion, stronger evidence to inform current practice and robust guidelines are urgently needed to prevent the transmission of SARS-CoV2 in gastrointestinal endoscopy departments worldwide.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.384512 |
| u5i935we | Response and role of palliative care during the COVID-19 pandemic: a national telephone survey of hospices in Italy                                                                                                                                                                                                                                                                                      | Background Palliative care is an essential component of healthcare in pandemics, contributing to symptom control, psychological support, and supporting triage and complex decision making. Aim To examine preparedness for, and impact of, the COVID-19 pandemic on hospices in Italy to inform the response in other countries. Design Cross-sectional telephone survey, carried out in March 2020. Setting Sixteen Italian hospices, purposively sampled according to COVID-19 risk into high (more than 25 COVID-19 cases per 100,000 inhabitants), medium (15-25 cases per 100,000), and low risk (fewer than 15 cases per 100,000) regions. A brief questionnaire was developed to guide the interviews. Descriptive analysis was undertaken. Results Seven high risk, five medium risk and four low risk hospices provided data. Two high risk hospices had experienced COVID-19 cases among both patients and staff. All hospices had implemented policy changes, and several had rapidly implemented changes in practice including transfer of staff from inpatient to community settings, change in admission criteria, and daily telephone support for families. Concerns included scarcity of personal protective equipment, a lack of hospice-specific guidance on COVID-19, anxiety about needing to care for children and other relatives, and poor integration of palliative care in the acute setting. Conclusion The hospice sector is capable of responding flexibly and rapidly to the COVID-19 pandemic. Governments must urgently recognise the essential contribution of hospice and palliative care to the COVID-19 pandemic, and ensure these services are integrated into the health care system response. Availability of personal protective equipment and setting-specific guidance is essential.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0.381399 |
| fzs99kku | An Interactive Tool to Forecast US Hospital Needs in the Coronavirus 2019 Pandemic                                                                                                                                                                                                                                                                                                                       | Hospital enterprises are currently faced with anticipating the spread of COVID-19 and the effects it will have on visits, admissions, bed needs, and crucial supplies. While many studies have focused on understanding the basic epidemiology of the disease, few open source tools have been made available to aid hospitals in their planning. We developed a web-based application for US states and territories that allows users to choose from a suite of models already employed in characterizing the spread of COVID-19. Users can obtain forecasts for hospital visits and admissions as well as anticipated needs for ICU and non-ICU beds, ventilators, and personal protective equipment supplies. Users can also customize a large set of inputs, view the variability in forecasts over time, and download forecast data. We describe our web application and its models in detail and provide recommendations and caveats for its use. Our application is primarily designed for hospital leaders, healthcare workers, and government official who may lack specialized knowledge in epidemiology and modeling. However, specialists can also use our open source code as a platform for modification and deeper study. As the dynamics of COVID-19 within the US change, our application will also change to meet emerging needs and questions of the healthcare community.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0.378696 |
| q9a374hh | Projected ICU and Mortuary load due to COVID-19 in Sydney                                                                                                                                                                                                                                                                                                                                                | The spread of COVID-19 is expected to put a large strain on many hospital resources, including ICU bed space, and mortuary capacity. In this report we study the possible demands on ICU and mortuary capacity in Sydney, Australia, using an adapted SEIR epidemiological model.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.378558 |
| 0ilj86y9 | Coronavirus: la emergencia geriátrica de 2020. Documento conjunto de la Sección de Cardiología Geriátrica de la Sociedad Española de Cardiología y la Sociedad Española de Geriatría y Gerontología                                                                                                                                                                                                      | Resumen La infección por SARS-CoV-2, denominada COVID-19 (Coronavirus Infectious Disease-19), es una enfermedad desconocida hasta diciembre de 2019 a la que nos enfrentamos en España desde el 31 de enero de 2020 —fecha del primer caso diagnosticado en nuestro país— y que ya ha causado la muerte de 7.340 personas (a 30 de marzo de 2020), sobre todo mayores. Es importante tener en cuenta que, dado que la información evoluciona con extremada rapidez en este campo, lo expuesto en el presente documento puede estar sujeto a modificaciones. La población de mayor edad es especialmente susceptible a la infección por COVID-19, así como a desarrollar criterios de gravedad. Este aumento de morbimortalidad en el paciente mayor se ha asociado tanto con las comorbilidades, especialmente la enfermedad cardiovascular, como con la situación de fragilidad, que conlleva una respuesta inmunológica más pobre. La situación actual, tanto por los países afectados como por el número de casos, constituye una pandemia y supone una emergencia sanitaria de primer nivel. Como España es uno de los países más envejecidos del mundo, la COVID-19 se ha convertido en una emergencia geriátrica. El presente documento se ha elaborado conjuntamente entre la Sección de Cardiología Geriátrica de la Sociedad Española de Cardiología y la Sociedad Española de Geriatría y Gerontología. Abstract SARS-CoV2 infection, also known as COVID-19 (coronavirus infectious disease-19), was first identified in December 2019. In Spain, the first case of this infection was diagnosed on 31 January, 2020 and, by 30 March 2020, has caused 7340 deaths, especially in the elderly. Due to the rapidly evolving situation regarding this disease, the data reported in this article may be subject to modifications. The older population are particularly susceptible to COVID-19 infection and to developing severe disease. The higher morbidity and mortality rates in older people have been associated with comorbidity, especially cardiovascular disease, and frailty, which weakens the immune response. Due to both the number of affected countries and the number of cases, the current situation constitutes an ongoing pandemic and a major health emergency. Because Spain has one of the largest older populations in the world, COVID-19 has emerged as a geriatric emergency. This document has been prepared jointly between the Geriatric Cardiology Section of the Spanish Society of Cardiology and the Spanish Society of Geriatrics and Gerontology.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.378547 |
| ti2k6ptb | New York MH groups launch campaign to help people stay connected                                                                                                                                                                                                                                                                                                                                         | The Coalition for Behavioral Health and the New York Association of Psychiatric Rehabilitation Services (NYAPRS) announced last week the launch of a new campaign to help New Yorkers to reach out, connect and comfort one another as New Yorkers work their way through the COVID‐19 crisis and social distancing, advocates announced in a news release. The “Strive for Five” campaign aims to educate and encourage New Yorkers to use a variety of strategies, such as phone, text, web, social media, and other applications and Zoom technologies to reach out and offer mental health support to at least five people over the next 30 days. As movement is increasingly restricted, virtual check‐ins grow more important each day, advocates stated. “ We encourage New Yorkers to connect with people who may be isolated, lonely or live alone, and to reach out to neighbors, friends, families and co‐workers. First responders, health care workers and other individuals who are courageously keeping our essential services open also need support now, as do caregivers who are hard at work taking care of others,” said NYAPRS and the coalition.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 0.374975 |
| omwv36x8 | The risk of SARS-CoV-2 transmission in the healthcare setting and potential impact of cohorting strategies                                                                                                                                                                                                                                                                                               | The threat of the COVID-19 pandemic of SARS-CoV-2 virus has grown since its start late last year. A customary response to a surge of a communicable disease is to separate healthcare systems into cohorts, such that patients with other conditions can be treated safely without risk of infection. A challenge with COVID-19 is the existence of currently asymptomatic infections, which without aggressive testing and isolation might enter the non-COVID-19 cohort either through unknowingly infected patients or health care workers. Using stochastic simulations we study the impacts of testing and personal protective equipment (PPE) use. In the base case without testing or PPE, healthcare is rapidly overwhelmed, and becomes a net contributor to the force of infection. We find that effective use of PPE by both HCW and patients can prevent this situation, while testing alone is less effective. We also find that even imperfectly effective use of PPE can provide substantial protection and decrease the force of infection. This illustrates the importance of maintaining supplies of PPE throughout healthcare systems                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.368539 |
| xk5igdwo | Coronavirus disease 2019 (covid-19): a guide for UK GPs                                                                                                                                                                                                                                                                                                                                                  | What you need to knowConsider covid-19 infection in anyone with cough, fever, or breathlessness who has had contact with someone with covid-19, or has returned from a high risk area in the 14 days before the onset of symptomsEvery effort should be made to avoid in-person assessment of patients with possible covid-19 in primary careGP surgeries should plan ahead and develop clear protocols for managing possible cases, including isolation procedures, personal protective equipment, seeking specialist advice, and decontaminationIf covid-19 infection is suspected in someone attending the practice, isolate the patient in a room (away from other patients and staff), close the door, and ask the patient to call NHS 111The guidance may change so it is essential to look at the latest guidance online (box 1)The UK recorded its first confirmed case of acute respiratory infection due to coronavirus disease 2019 (covid-19) on 31 January 2020 and responded by quarantining at-risk individuals to contain the spread of infection. Executive agencies Public Health England (PHE)1 and Health Protection Scotland (HPS) have since published guidance to healthcare providers on managing patients suspected to have the disease. Guidance for the public and health professionals varies internationally, depending partly on risk levels and healthcare systems, and is being regularly updated. This article offers a practical guide for GPs and others working in UK primary care on when to suspect covid-19 and how to respond. It is based on current UK guidance at the time of publication. We recommend readers consult the latest guidance (box 1).Box 1 Essential resourcescovid-19: latest case definition, investigation, and initial clinical management of possible cases:https://www.gov.uk/government/publications/wuhan-novel-coronavirus-initial-investigation-of-possible-cases/investigation-and-initial-clinical-management-of-possible-cases-of-wuhan-novel-coronavirus-wn-cov-infectionCoronavirus: latest information and advice including updated list of high risk countries:https://www.gov.uk/guidance/wuhan-novel-coronavirus-information-for-the-publicGuidance on isolation of healthcare workers:https://www.gov.uk/government/publications/novel-coronavirus-2019-ncov-guidance-for-healthcare-providers-with-staff-who-have-travelled-to-china/guidance-for-healthcare-providers-healthcare-workers-who-have-travelled-to-chinaFind your local Health Protection Team in England:https://www.gov.uk/health-protection-teamcovid-19: interim guidance … RETURN TO TEXT                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | 0.36784  |
| 4pvk3fqn | Reorganization of Medical Oncology Departments during COVID-19 Pandemic: a Nationwide Italian Survey                                                                                                                                                                                                                                                                                                     | Abstract The novel severe acute respiratory syndrome coronavirus-2 (SARSCoV-2) pandemic is a global health problem, which started to affect China by the end of year 2019. In Europe, Italy has faced this novel disease entity (named COVID-19) first and severely. COVID-19 represents a significant hurdle for public health services and a potential harm for patients with cancer. The Collegio Italiano dei Primari Oncologi Medici (CIPOMO) is an Italian association of head physicians in oncology departments, which promotes working and research activities in oncology on a national basis. In the midst of the epidemic in Italy, the CIPOMO promoted a national survey aiming to evaluate the impact of COVID-19 on clinical activity of oncologists and the implementation of containment measures of COVID-19 diffusion. Overall, 122 head physicians participated in this survey, with a homogeneous distribution on the national territory. Results show that the following measures for oncologic patients have been promptly implemented through the whole country: use of protective devices, triage of patients accessing the hospital, delay of non-urgent visits, and use of telemedicine. Results of this survey suggest that Italian oncology departments have promptly set a proactive approach to the actual emergency. Oncologists need to preserve the continuum of care of patients, as the benefit of ensuring a well-delivered anti-cancer treatment plan outweighs the risk of COVID-19 infection. International cooperation is an important starting point, as heavily affected nations can serve as an example to find out ways to safely preserve health activity during pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.366742 |
| tq9kjozt | Effect of various decontamination procedures on disposable N95 mask integrity and SARS-CoV-2 infectivity                                                                                                                                                                                                                                                                                                 | The COVID-19 pandemic has created a high demand on personal protective equipment, including disposable N95 masks. Given the need for mask reuse, we tested the feasibility of vaporized hydrogen peroxide (VHP), ultraviolet light (UV), and ethanol decontamination strategies on N95 mask integrity and the ability to remove the infectious potential of SARS-CoV-2. FIT test data showed functional degradation by both ethanol and UV decontamination to different degrees. VHP treated masks showed no significant change in function after two treatments. We also report a single SARS-CoV-2 virucidal experiment using Vero E6 cell infection. We hope our data will guide further research for evidenced-based decisions for disposable N95 mask reuse and help protect caregivers from SARS-CoV-2 and other pathogens.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.36164  |
| ycywtpzj | Piloting Forensic Tele-Mental Health Evaluations of Asylum Seekers                                                                                                                                                                                                                                                                                                                                       | Problem Forensic mental health evaluations can provide critical evidence in the legal cases of individuals seeking asylum and other forms of protected immigration status. While the number of academically affiliated medical human rights programs has increased in recent years, there is still substantial unmet need for pro bono evaluations throughout the United States, especially for individuals in detention. Approach The Mount Sinai Human Rights Program launched its pilot Remote Evaluation Network in September 2019, with the aim to coordinate forensic mental health evaluations by telephone or video call for individuals who are unable to access in-person services. The authors recruited mental health clinicians from across the country, trained them in best practices in conducting forensic evaluations using telehealth platforms, and coordinated pro bono mental health evaluations of individuals seeking immigration relief. Remote forensic services have been a particularly relevant solution in the context of the COVID-19 pandemic. Outcomes The Remote Evaluation Network consists of seventeen active evaluators. From December 2019 to April 2020, the pilot program has coordinated fifteen forensic evaluations of individuals seeking asylum and other forms of protected immigration status in six different states. All clinicians participated in a training module on best practices in conducting forensic evaluations by telephone; respondents to optional pre-and-post-module surveys reported an increase in comfort level with conducting telephonic evaluations after participating in the module. Next Steps We will formally evaluate this pilot program's services by assessing the quality of medico-legal affidavits from telephonic evaluations, tracking legal outcomes and qualitative feedback from attorneys, and investigating the acceptability of telephonic mental health evaluations among legal professionals. Future directions include expansion to new geographies, including individuals affected by the Migrant Protection Protocols or "Remain in Mexico" program.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.361614 |
| eymqlbix | Society of Cardiovascular Computed Tomography Guidance for Use of Cardiac Computed Tomography Amidst the COVID-19 Pandemic                                                                                                                                                                                                                                                                               | Abstract The world is currently suffering through a pandemic outbreak of severe respiratory syndrome coronavirus 2 (SARS-CoV-2) known as Coronavirus Disease 2019 (COVID-19). The United States (US) Centers for Disease Control and Prevention (CDC) currently advises medical facilities to “reschedule non-urgent outpatient visits as necessary” (1). The European Centre for Disease Prevention and Control, the United Kingdom National Health Service and several other international agencies covering Asia, North America and most regions of the world have recommended similar “social distancing” measures(2, 3). The Society of Cardiovascular Computed Tomography (SCCT) offers guidance for cardiac CT (CCT) practitioners to help implement these international recommendations in order to decrease the risk of COVID-19 transmission in their facilities while deciding on the timing of outpatient and inpatient CCT exams. This document also emphasizes SCCT’s commitment to the health and well-being of CCT technologists, imagers, trainees, and research community, as well as the patients served by CCT.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.354201 |
| 70jx0h9v | The COVID-19 pandemic: Important considerations for contact lens practitioners                                                                                                                                                                                                                                                                                                                           | Abstract A novel coronavirus (CoV), the Severe Acute Respiratory Syndrome Coronavirus - 2 (SARS-CoV-2), results in the coronavirus disease 2019 (COVID-19). As information concerning the COVID-19 disease continues to evolve, patients look to their eye care practitioners for accurate eye health guidance. There is currently no evidence to suggest an increased risk of contracting COVID-19 through contact lens (CL) wear compared to spectacle lens wear and no scientific evidence that wearing standard prescription spectacles provides protection against COVID-19 or other viral transmissions. During the pandemic there will potentially be significant changes in access to local eyecare. Thus, it is imperative CL wearers are reminded of the steps they should follow to minimise their risk of complications, to reduce their need to leave isolation and seek care. Management of adverse events should be retained within optometric systems if possible, to minimise the impact on the wider healthcare service, which will be stretched. Optimal CL care behaviours should be the same as those under normal circumstances, which include appropriate hand washing (thoroughly with soap and water) and drying (with paper towels) before both CL application and removal. Daily CL cleaning and correct case care for reusable CL should be followed according to appropriate guidelines, and CL exposure to water must be avoided. Where the availability of local clinical care is restricted, practitioners should consider advising patients to reduce or eliminate sleeping in their CL (where patients have the appropriate knowledge about correct daily care and access to suitable lens-care products) or consider the option of moving patients to daily disposable lenses (where patients have appropriate lens supplies available). Patients should also avoid touching their face, including their eyes, nose and mouth, with unwashed hands and avoid CL wear altogether if unwell (particularly with any cold or flu-like symptoms).                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.353044 |
| npt8i9bc | If not now, when?                                                                                                                                                                                                                                                                                                                                                                                        | The editor of the BJPsych Bulletin reflects on the extraordinary recent events triggered by the COVID-19 pandemic. Mental health professionals are at the front line of managing the pandemic and emergency changes should lead to a much needed refocus on what is really vital. In these unsettling times we ought to review how we manage the crisis, and its aftermath, both personally and professionally.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 0.352296 |
| dgyvbarm | A Nationwide Survey of UK cardiac surgeons view on clinical decision making during the COVID-19 pandemic                                                                                                                                                                                                                                                                                                 | Background: No firm recommendations are currently available to guide decision making for patients requiring cardiac surgery during the COVID-19 pandemic. Systematic appraisal of national expert consensus can be used to generate interim recommendations until data from clinical observations will become available. Hence, we aimed to collect and quantitatively appraise nationwide UK senior surgeons opinion on clinical decision making for patients requiring cardiac surgery during the COVID-19 pandemic. Methods: We mailed a web-based questionnaire to all consultant cardiac surgeons through the Society for Cardiothoracic Surgery in Great Britain and Ireland (SCTS) mailing list on the 17th April 2020 and we pre-determined to close the survey on the 21st April 2020. This survey was primarily designed to gather information on UK surgeons opinion using 12 items. Strong consensus was predefined as an opinion shared by at least 60% of responding consultants. Results: A total of 86 consultant surgeons undertook the survey. All UK cardiac units were represented by at least one consultant. Strong consensus was achieved for the following key questions:1) before hospital admission every patient should receive nasopharyngeal swab, PCR and chest CT; 2) the use of full PPE should to be adopted in every case by the theatre team regardless patients COVID-19 status; 3) the risk of COVID-19 exposure for patients undergoing heart surgery should be considered moderate to high and likely to increase mortality if it occurs; 4) cardiac procedure should be decided based on ad-hoc multidisciplinary team discussion for every patient. The majority believed that both aortic and mitral surgery should be considered in selected cases. The role of CABG surgery during the pandemic was more controversial. Conclusions: In the current unprecedented scenario, the present survey provides information for generating interim recommendations until data from clinical observations will become available.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.350337 |
| ovl8o74r | Expert consensus on rehabilitation practice during outbreaks of the novel coronavirus pneumonia and other infectious respiratory diseases                                                                                                                                                                                                                                                                | The outbreak of the novel coronavirus pneumonia (NCP), an acute infectious respiratory disease in Wuhan, China in December 2019, which was officially named COVID-19 by WHO, has caused enormous burden on society and health services. To ensure safe rehabilitation practice during the epidemic, the Chinese Association of Rehabilitation Medicine formed an expert group to develop a consensus statement for rehabilitation practice. The consensus statement provides guidance on: (1) safeguarding medical staff, patients, caregivers and visitors; and (2) timing and methods of effective rehabilitation interventions for treatment of COVID-19 patients. It also emphasizes that non-COVID-19 patients still require rehabilitation intervention during the epidemic outbreak, bearing in mind the safety of staff and patients and making adaptations according to the individual conditions and the patients&prime; actual environment. This expert consensus statement applies to departments of rehabilitation medicine and related disciplines in general hospitals as well as to specialized rehabilitation facilities, and it provides valuable reference for rehabilitation professionals working with infectious respiratory diseases in the future.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | 0.345099 |
| o62vikyv | Static vs accumulating priorities in healthcare queues under heavy loads                                                                                                                                                                                                                                                                                                                                 | Amid unprecedented times caused by COVID-19, healthcare systems all over the world are strained to the limits of, or even beyond, capacity. A similar event is experienced by some healthcare systems regularly, due to for instance seasonal spikes in the number of patients. We model this as a queueing system in heavy traffic (where the arrival rate is approaching the service rate from below) or in overload (where the arrival rate exceeds the service rate). In both cases we assume that customers (patients) may have different priorities and we consider two popular service disciplines: static priorities and accumulating priorities. It has been shown that the latter allows for patients of all classes to be seen in a timely manner as long as the system is stable. We demonstrate however that if accumulating priorities are used in the heavy traffic or overload regime, then all patients, including those with the highest priority, will experience very long waiting times. If on the other hand static priorities are applied, then one can ensure that the highest-priority patients will be seen in a timely manner even in overloaded systems.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.345068 |
| wy9k0508 | Covid 19 pandemic and gynaecological laparoscopic surgery: knowns and unknowns                                                                                                                                                                                                                                                                                                                           | The worldwide impact of COVID 19 continues to be felt as hospitals in all countries reduce elective and non-urgent cases to allow staffing and resources to be deployed elsewhere. Urgent gynaecological and cancer procedures are continuing, and it is imperative all theatre staff are protected and risks of SARS-CoV-2 viral transmission reduced when operating on asymptomatic, suspected or confirmed COVID 19 patients. In particular, there are concerns relating to the transmission of COVID 19 during gynaecological laparoscopic surgery, arising from the potential generation of SARS-CoV-2 contaminated aerosols from CO(2) leakage and the creation of smoke from the use of energy devices. The aim of this paper is to review all the up to date evidence, including experiences from China and Italy, to guide the safe management of such patients when undergoing gynaecological procedures.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.343905 |
| viwochvv | Modelling strategies to organize healthcare workforce during pandemics: application to COVID-19                                                                                                                                                                                                                                                                                                          | Protection of healthcare workforce is of paramount relevance for the care of infected and non-infected patients in the setting of a pandemic such as coronavirus disease 2019 (COVID-19). Healthcare workers are at increased risk to become infected because of contact to infected patients, infected co-workers and their community outside the hospital. The ideal organisational strategy to protect the healthcare workforce in a situation in which social distancing cannot be maintained at the workplace remains to be determined. In this study, we have mathematically modelled strategies for the employment of hospital workforce with the goal to simulate health and productivity of the workers. Therefore, deterministic models were extended to account for stochastic influences potentially occurring in rather small populations. The models were also designed to determine if desynchronization of medical teams by dichotomizing the workers may protect the workforce. Our studies model workforce productivity depending on the infection rate, the presence of reinfection and the efficiency of home office. As an application example, we apply our theory to the case of coronavirus disease 2019 (COVID-19). The results of the models reveal that a desynchronization strategy in which two medical teams work alternating for 7 days reduces the infection rate of the healthcare workforce. In case of immunity to the infectious agent this affect is mainly relevant at early stages of the pandemic. This effect is independent on infection rates and increases the overall workforce productivity under certain situations.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.341964 |
| gy1trcj3 | Expert recommendations for diabetes with or without COVID-19 management in communities                                                                                                                                                                                                                                                                                                                   | Since December 2019, the outbreak of coronavirus disease 2019 (COVID-19) has occurred in China. Higher risk of COVID-19 infection and worse prognosis were observed among patients with diabetes. There are rigorous challenges existing in terms of diabetes prevention and glycemic control in primary medical care during period of COVID-19. Here, expert recommendations were developed by Chinese diabetologists, healthcare providers, and public health administrators to improve the ability of primary health facilities and provide standardized basic public health and medical services throughout country. The main contents include basic requirements for management, workflow of health management, referral, treatment, and long-term follow-up.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.339881 |
| 5tydrrz1 | Pharmacists at the frontline beating the COVID-19 pandemic                                                                                                                                                                                                                                                                                                                                               | As the lockdowns are being observed all over the globe and the national level pharmacy professionals are performing frontline roles, this editorial highlights the role of pharmacists in the COVID − 19 pandemic. Pharmacists globally are providing services amidst pandemic, including TRIAGE services, seeing patients and reducing the patients’ burden on health care facilities such as hospitals and GP practices. Pharmacists are also working to providing home deliveries, as well as dealing with the increasing number of patients coming through to pharmacies with the other ailments. Pharmacy associations have issued their guidelines and in this editorial, several global examples of pharmacists’ role in the COVID 19 are being discussed. Pakistan is used as a country case study in this editorial. The editorial also elaborates how pharmacists in the UK and Pakistan have teamed up together to compile 10-steps protection guidelines for the pharmacy teams in Pakistan in English and Urdu language. This 10-point guidance educates community pharmacies for safety and standard operation as the number of patients in the country continues to rise. These guidelines are endorsed by the government and private bodies. These can be adopted and adapted by any country; keeping in view their laws and regulations.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.338969 |
| c7msqrd4 | Pediatric laryngoscopy and bronchoscopy during the COVID-19 pandemic: A four-center collaborative protocol to improve safety with perioperative management strategies and creation of a surgical tent with disposable drapes                                                                                                                                                                             | Abstract Aerosolization procedures during the COVID-19 pandemic place all operating room personnel at risk for exposure. We offer detailed perioperative management strategies and present a specific protocol designed to improve safety during pediatric laryngoscopy and bronchoscopy. Several methods of using disposable drapes for various procedures are described, with the goal of constructing a tent around the patient to decrease widespread contamination of dispersed droplets and generated aerosol. The concepts presented herein are translatable to future situations where aerosol generating procedures increase risk for any pathogenic exposure. This protocol is a collaborative effort based on knowledge gleaned from clinical and simulation experience from Children’s Hospital Colorado, Children’s Hospital of Philadelphia, The Hospital for Sick Children in Toronto, and Boston Children’s Hospital.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.328796 |
| d8otzqnt | Pneumask: Modified Full-Face Snorkel Masks as Reusable Personal Protective Equipment for Hospital Personnel                                                                                                                                                                                                                                                                                              | Here we adapt and evaluate a full-face snorkel mask for use as personal protective equipment (PPE) for health care workers, who lack appropriate alternatives during the COVID-19 crisis in the spring of 2020. The design (referred to as Pneumask) consists of a custom snorkel-specific adapter that couples the snorkel-port of the mask to a rated filter (either a medical-grade ventilator inline filter or an industrial filter). This design has been tested for the sealing capability of the mask, filter performance, CO2 buildup and clinical usability. These tests found the Pneumask capable of forming a seal that exceeds the standards required for half-face respirators or N95 respirators. Filter testing indicates a range of options with varying performance depending on the quality of filter selected, but with typical filter performance exceeding or comparable to the N95 standard. CO2 buildup was found to be roughly equivalent to levels found in half-face elastomeric respirators in literature. Clinical usability tests indicate sufficient visibility and, while speaking is somewhat muffled, this can be addressed via amplification (Bluetooth voice relay to cell phone speakers through an app) in noisy environments. We present guidance on the assembly, usage (donning and doffing) and decontamination protocols. The benefit of the Pneumask as PPE is that it is reusable for longer periods than typical disposable N95 respirators, as the snorkel mask can withstand rigorous decontamination protocols (that are standard to regular elastomeric respirators). With the dire worldwide shortage of PPE for medical personnel, our conclusions on the performance and efficacy of Pneumask as an N95-alternative technology are cautiously optimistic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0.327303 |
| vzwyg6ha | THE EMOTIONAL IMPACT OF THE ASRM GUIDELINES ON FERTILITY PATIENTS DURING THE COVID-19 PANDEMIC                                                                                                                                                                                                                                                                                                           | Objective: To survey fertility patients' agreement with ASRM recommendations during the COVID-19 pandemic and the emotional impact on them. Design: An online survey was sent to current fertility patients Setting: New York City academic fertility practice at the epicenter of the COVID-19 pandemic Patient(s): Fertility patients seen within the last year Intervention(s): None Main Outcome Measure(s): Patient agreement with the ASRM recommendations during the COVID-19 pandemic and the emotional impact rated on a Likert scale Result(s): A total of 518 patients completed the survey for a response rate of 17%. Fifty percent of respondents had a cycle canceled due to the COVID-19 pandemic. Of those who had a cycle cancelled, 85% of respondents found it to be moderately to extremely upsetting with 22% rating it to be equivalent to the loss of a child. There was no difference on the emotional impact based on the type of cycle cancelled. Fifty-five percent of patients agreed that diagnostic procedures such as hysterosalpingograms should be cancelled while 36% of patients agreed all fertility cycles should be cancelled. Patients were slightly more likely to agree with the ASRM guidelines if they have an upcoming cycle cancelled (p = 0.041). Of all respondents 82% would have preferred to have the option to start a treatment cycle in consultation with their doctor. Conclusion(s): Given the severity of the COVID-19 pandemic, the physical, financial and emotional impact of this unprecedented threat cannot be underestimated in our fertility patients. Key Word(s): COVID-19, novel coronavirus, ASRM, mental health, infertility                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | 0.326157 |
| tib0g9ok | COVID-19: Financial Stress Test for Academic Medical Centers                                                                                                                                                                                                                                                                                                                                             | The coronavirus pandemic (COVID-19) is having profound effects on the lives and well-being of the world’s population. All levels of the nation’s public health and health care delivery systems are rapidly adjusting to secure the health infrastructure to manage the pandemic in the United States. As the nation’s safety net health care systems, academic medical centers (AMCs) are vital clinical and academic resources in managing the pandemic. COVID-19 may also risk the financial underpinnings of AMCs because their cost structures are high, and they may have incurred large amounts of debt over the last decade as they expanded their clinical operations and facilities. This Invited Commentary reviews existing data on AMC debt levels; summarizes relief provided in the Coronavirus Aid, Relief, and Economic Security Act; and suggests policy options to help mitigate risk.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.325921 |
| l3d1dc93 | A modification to the Maquet Flow-i anaesthesia machine for ICU ventilation                                                                                                                                                                                                                                                                                                                              | The authors present an easily manufactured modification of the Getinge Group Maquet Flow-i anaesthesia machine that gives it potential to be used long-term as an Intensive Care ventilator for emergency circumstances. There are some 7000 such machines in use worldwide, which could assist in increasing ICU ventilated bed capacity in a number of nations. The authors believe this modification has potential as a solution to increasing ventilator numbers for the COVID-19 pandemic, in hospitals where the Flow-i is underutilised for its designed purpose during this emergency. The technical drawing files are downloadable on the GrabCAD website and are Creative Commons (CC-BY 4.0) licensed to allow local manufacture of the modification.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | 0.322158 |
| 50vo8bnc | Adapting palliative radiation therapy for bone metastases during the Covid-19 pandemic: GEMO position paper                                                                                                                                                                                                                                                                                              | Abstract The current health crisis caused by COVID-19 is a challenge for oncology treatment, especially when it comes to radiotherapy. Cancer patients are already known to be very fragile and COVID-19 brings about the risk of severe respiratory complications. In order to treat patients safely while protecting medical teams, the entire health care system must optimize the way it approaches prevention and treatment at a time when social distancing is key to stemming this pandemic. All indications and treatment modalities must be re-discussed. This is particularly the case for radiotherapy of bone metastases for which it is possible to reduce the number of sessions, the frequency of transport and the complexity of treatments. These changes will have to be discussed according to the organization of each radiotherapy department and the health situation, while medical teams must remain vigilant about the risks of complications of bone metastases, particularly spinal metastases. In this short piece, the members of the GEMO (the European Study Group of Bone Metastases) offer a number of recommendations to achieve the above objectives, both in general and in relation to five of the most common situations on radiation therapy for bone metastases.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.320799 |
| jkyltjap | Locally Informed Simulation to Predict Hospital Capacity Needs During the COVID-19 Pandemic                                                                                                                                                                                                                                                                                                              | BACKGROUND: The coronavirus disease 2019 (COVID-19) pandemic challenges hospital leaders to make time-sensitive, critical decisions about clinical operations and resource allocations. OBJECTIVE: To estimate the timing of surges in clinical demand and the best- and worst-case scenarios of local COVID-19–induced strain on hospital capacity, and thus inform clinical operations and staffing demands and identify when hospital capacity would be saturated. DESIGN: Monte Carlo simulation instantiation of a susceptible, infected, removed (SIR) model with a 1-day cycle. SETTING: 3 hospitals in an academic health system. PATIENTS: All people living in the greater Philadelphia region. MEASUREMENTS: The COVID-19 Hospital Impact Model (CHIME) (http://penn-chime.phl.io) SIR model was used to estimate the time from 23 March 2020 until hospital capacity would probably be exceeded, and the intensity of the surge, including for intensive care unit (ICU) beds and ventilators. RESULTS: Using patients with COVID-19 alone, CHIME estimated that it would be 31 to 53 days before demand exceeds existing hospital capacity. In best- and worst-case scenarios of surges in the number of patients with COVID-19, the needed total capacity for hospital beds would reach 3131 to 12 650 across the 3 hospitals, including 338 to 1608 ICU beds and 118 to 599 ventilators. LIMITATIONS: Model parameters were taken directly or derived from published data across heterogeneous populations and practice environments and from the health system's historical data. CHIME does not incorporate more transition states to model infection severity, social networks to model transmission dynamics, or geographic information to account for spatial patterns of human interaction. CONCLUSION: Publicly available and designed for hospital operations leaders, this modeling tool can inform preparations for capacity strain during the early days of a pandemic. PRIMARY FUNDING SOURCE: University of Pennsylvania Health System and the Palliative and Advanced Illness Research Center.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 0.318753 |
| i9ba6vv7 | COVID-19 and ART: the view of the Italian Society of Fertility and Sterility and Reproductive Medicine                                                                                                                                                                                                                                                                                                   | Abstract The COVID-19 pandemic is an unprecedented global situation. As ART specialists we should be cautious, carefully monitoring the situation whilst contributing by sharing novel evidence to counsel our patients, both pregnant women and would-be mothers. Time to egg collection and drop-out rates are parameters critical to scheduling treatments once the curve of infections has peaked and plateaued in each country. In order to reduce both parameters, infertile patients now require even more support from their IVF team: urgent oocyte collection for oncologic patients must be guaranteed, and oocyte retrievals for women of advanced maternal age and/or reduced ovarian reserve cannot be postponed indefinitely. This document represents the position of the Italian Society of Fertility and Sterility and Reproductive Medicine (SIFES-MR) in outlining ART priorities during and after this emergency.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 0.315182 |
| m9hxq85l | ¿Va a cambiar la neurología tras la pandemia de Covid-19 en los próximos cinco años?. Estudio de enfoque mediante informadores clave                                                                                                                                                                                                                                                                     | ] Abstract Introduction: The COVID-19 pandemic will give rise to long-term changes in neurological care, which are not easily predictable. Material and methods: A key informant survey was used to enquire about the changes expected in the specialty over the next 5 years. The survey was completed by heads of neurology departments with broad knowledge of the situation, having been active during the pandemic. Results: Despite a low level of consensus between participants, there was strong (85%) and moderate consensus (70%) about certain subjects, mainly the increase in precautions to be taken, the use of telemedicine and teleconsultations, the reduction of care provided in in-person consultations to avoid the presence of large numbers of people in waiting rooms, the development of remote training solutions, and the changes in monitoring visits during clinical trials. There was consensus that there would be no changes to the indication of complementary testing or neurological examination. Conclusion: The key informant survey identified the foreseeable changes in neurological care after the pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | 0.309291 |
| 8lud7tqx | Advice on Standardized Diagnosis and Treatment for Spinal Diseases during the Coronavirus Disease 2019 Pandemic                                                                                                                                                                                                                                                                                          | Coronavirus disease 2019 (COVID-19) outbreak started in December 2019 that caused difficulties for clinical work. Practical work experience in our spinal outpatient and emergency department during the COVID-19 pandemic is summarized in this article, with combined evidence-based medical evidence to explore a standardized process of diagnosis and treatment for spinal diseases. Outpatient reservation, continuous screening, triage, and isolation, first consultation accountability system, pandemic reporting system, and online revisit were strictly followed. We hope that our experience in prevention and control of COVID-19 can help spine surgeons globally in stopping the spread of COVID-19. Spine surgeons should collaborate with infection control specialists to avoid cross-infection in hospitals and optimize treatment.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | 0.304472 |
| 2178imf2 | Video consultations for covid-19                                                                                                                                                                                                                                                                                                                                                                         | An opportunity in a crisis? The rapid spread of covid-19, and the fact that healthcare facilities could be sources of contagion, has focused attention on new models of care that avoid face-to-face contact between clinician and patient. There has been particular interest in video consultations, which are already being rolled out in many countries as part of national digital health strategies.123 How appropriate are video consultations for dealing with the coronavirus crisis—and what are the challenges of scaling up this model at speed? Randomised trials (most of which were underpowered) have shown that clinical consultations conducted through a video link tend to be associated with high satisfaction among patients and staff; no difference in disease progression; no substantial difference in service use; and lower transaction costs compared with traditional clinic based care.4567 However, almost all this evidence pertains to highly selected samples of hospital outpatients with chronic, stable conditions and is largely irrelevant to the current escalating situation involving patients with an acute and potentially serious illness. Organisational case studies have shown that introducing video consultations is a complex change that disrupts long established processes and routines.891011 Some …                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            | 0.302338 |
| aarthz9w | Individualized System for Augmenting Ventilator Efficacy (iSAVE): A Rapidly deployable system to expand ventilator capacity                                                                                                                                                                                                                                                                              | The COVID-19 pandemic is overwhelming healthcare systems worldwide. A significant portion of COVID-19 patients develop pneumonia and acute respiratory distress syndrome (ARDS), necessitating ventilator support. Some health systems do not have the capacity to accommodate this surge in ventilator demand, leading to shortages and inevitable mortality. Some clinicians have, of necessity, jerry-rigged ventilators to support multiple patients, but these devices lack protected air streams or individualized controls for each patient. Moreover, some have not been tested under conditions of ARDS. We have developed the Individualized System for Augmenting Ventilator Efficacy (iSAVE), a rapidly deployable platform to more safely use a single ventilator to simultaneously support multiple critically-ill patients. The iSAVE enables patient-specific volume and pressure control and incorporates safety features to mitigate cross-contamination between patients and flow changes due to patient interdependencies within the respiratory circuit. Here we demonstrate through simulated and in vivo pig evaluation the capacity of the iSAVE to support a range of respiratory clinical states. By leveraging off-the-shelf components that are readily available to intensive care unit (ICU) caregivers, the iSAVE could potentially be translated for human application to expand the ventilation capacity of hospitals using existing ventilators, minimizing the need to procure additional ventilators.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0.300588 |
| 98jz8tox | The Impact of School Closure for COVID-19 on the US Healthcare Workforce and the Net Mortality Effects                                                                                                                                                                                                                                                                                                   | Background COVID-19 is leading to the implementation of social distancing policies around the world and in the United States, including school closures. The evidence that mandatory school closures reduce cases and ultimately mortality mostly comes from experience with influenza or from models that do not include the impact of school closure on the healthcare labor supply or the role of the healthcare labor force in reducing the per infection mortality from the pathogen. There is considerable uncertainty of the incremental effect of school closures on transmission and lives saved from school closures. The likely, but uncertain, benefits from school closure need to be weighed against uncertain, and seldom quantified, costs of healthcare worker absenteeism associated with additional child care obligations. Methods We analyze data from the US Current Population Survey to measure the potential child care obligations for US healthcare workers that will need to be addressed if school closures are employed as a social distancing measure. We account for the occupation within the healthcare sector, state, and household structure to identify the segments of the healthcare labor force that are most exposed to child care obligations from school closures. We use these estimates to identify the critical level for the importance of healthcare labor supply in increasing a patient's COVID-19 survival probability that would undo the benefits of school closures and ultimately increase cumulative mortality. Findings The US healthcare sector has some of the highest child care obligations in the United States. 29% of healthcare provider households must provide care for children 3-12. Assuming non-working adults or a sibling 13 years old or older can provide child care, leaves 15% of healthcare provider households in need of childcare during a school closure, while 7% of healthcare households are single-parent households. We document the substantial variation within the healthcare system. For example, 35% of medical assistants and 31% of nursing, psychiatric, and home health aide households have child care obligations, while only 24% of emergency medical personnel have childcare obligations. Child care obligations can vary between states by over 10 percentage points. A 15% decline in the healthcare labor force, combined with reasonable parameters for COVID-19 such as a 15% case reduction from school closings and 2% baseline mortality rate implies that a 15% loss in the healthcare labor force must decrease the survival probability per percent healthcare worker lost by 17.6% for a school closure to increase cumulative mortality. This means that the per infection mortality rate cannot increase from 2% to 2.35% when the healthcare workforce declines by 15%; otherwise, school closures will lead to a greater number of deaths than they prevent. For school closures to unambiguously provide a net reduction in COVID-19 mortality with these parameters, the school closures must reduce cases by over 25%. Conclusion School closures come with many tradeoffs. Setting aside economic costs, school closures implemented to reduce COVID-19 spread create unintended childcare obligations, which are particularly large in healthcare occupations. Detailed data are provided to help public health officials make informed decisions about the tradeoffs associated with closing schools. The results suggest that it is unclear if the potential contagion prevention from school closures justifies the potential loss of healthcare workers from the standpoint of reducing cummulative mortality.                                                                                                                                                                            | 0.298703 |
| nk9hhco3 | COVID-19: Conceptos clave para el cirujano                                                                                                                                                                                                                                                                                                                                                               | Resumen Ante la pandemia por SARS-CoV-2 resulta fundamental conocer los aspectos claves de la infección: su origen epidemiológico, presentación, curso clínico, diagnóstico y los tratamientos empleados (aún experimentales en muchos casos). El conocimiento sobre el virus es limitado, pero a medida que progresa la pandemia y se conoce más su fisiopatología, se está publicando nueva evidencia de forma masiva. Los especialistas quirúrgicos se enfrentan a una situación sin precedentes: deben colaborar en plantas médicas o urgencias atendiendo a estos pacientes y además tomar decisiones sobre pacientes quirúrgicos con posible COVID-19. Esta revisión narrativa pretende resumir los aspectos más relevantes y sintetizar los conceptos básicos sobre COVID-19 para los cirujanos. Abstract In view of the current pandemic by SARS-CoV-2 it deems essential to understand the key concepts about the infection: its epidemiological origin, presentation, clinical course, diagnosis and treatment (still experimental in many cases). The knowledge about the virus is still limited, but as the pandemic progresses and the physiopathology of the disease is understood, new evidence is being massively published. Surgical specialists are facing an unprecedented situation: they must collaborate in the ER or medical wards attending these patients, while still needing to make decisions about surgical patients with probable COVID-19. The present narrative review aims to summarize the most relevant aspects and synthetize concepts on COVID-19 for surgeons.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.290694 |
| 1wae4kw7 | Community Pharmacists in Taiwan at the Frontline Against the Novel Coronavirus Pandemic: Gatekeepers for the Rationing of Personal Protective Equipment                                                                                                                                                                                                                                                  | Compared with other countries, Taiwan has had relatively few cases during the COVID-19 pandemic. One of the many measures the government implemented was a system for rationing and distributing surgical masks to the public while prioritizing allocation of masks to health care workers. This essay describes the role of community pharmacists in implementing the system and distributing masks to the public.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.28448  |
| slejus63 | Emergency management of nursing human resources and supplies to respond to coronavirus disease 2019 epidemic                                                                                                                                                                                                                                                                                             | Abstract Objective To introduce the emergency management of nursing human resources and supplies of a large general hospital when facing the outbreak of coronavirus disease 2019(COVID-19). Method The Nursing Department of the hospital fully executed its functional authority to establish a three-level echelon of sustainable support, allocate human recourses dynamically, organize pre-service training, supervise the key working steps, formulate positive incentive methods, and deploy medical supplies scientifically. Result By taking these strategies, the hospital effectively improved the coping capacity of the nursing team and played a positive role in the prevention and treatment of COVID-19. Conclusion The emergency management of nursing human resources and material resources for COVID-19 of the hospital is successful. But several deficiencies were identified as well, which indicated that the hospital needs to establish an efficient emergency management system, and pay attention to the practice of nursing emergency plans to enhance coping capacities in public health emergencies.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 0.280935 |
| lx7zfgoi | Isolation protocol for a COVID-2019 patient requiring emergent surgical intervention: case presentation                                                                                                                                                                                                                                                                                                  | BACKGROUND: The concerns of the highly contagious and morbid nature of Coronavirus Disease-2019 (COVID-2019) have prompted healthcare workers to implement strict droplet and contact isolation precautions. Unfortunately, some patients who may be or presumptively or confirmed as infected with COVID-2019 may also require emergent surgical procedures. As such, given the high-risk for exposure of many healthcare workers involved the complex requirements for appropriate isolation must be adhered to. CASE PRESENTATION: We present our experience with a 77-year-old who required emergency cardiac surgery for a presumed acute aortic syndrome in the setting of a presumed, and eventually confirmed, COVID-2019 infection. We outline the necessary steps to maintain strict isolation precautions to limit potential exposure to the surgical Team. CONCLUSIONS: We hereby provide our algorithm for emergent surgical procedures in critically-ill patients with presumptive or confirmed infection with COVID-2019. The insights from this case report can potentially be templated to other facilities in order to uphold high standards of infection prevention and patient safety in surgery during the current COVID-19 pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | 0.276948 |
| xwqhaaij | The role and response of palliative care and hospice services in epidemics and pandemics: a rapid review to inform practice during the COVID-19 pandemic                                                                                                                                                                                                                                                 | Abstract Cases of COVID-19 are escalating rapidly across the globe, with the mortality risk being especially high among those with existing illness and multimorbidity. This study aimed to synthesise evidence for the role and response of palliative care and hospice teams to viral epi/pandemics, to inform the COVID-19 pandemic response. We conducted a rapid systematic review according to PRISMA guidelines in five databases. Of 3094 papers identified, ten were included in this narrative synthesis. Included studies were from West Africa, Taiwan, Hong Kong, Singapore, the United States and Italy. All had an observational design. Findings were synthesised using a previously proposed framework according to ‘systems’ (policies, training and protocols, communication and coordination, data), ‘staff’ (deployment, skill mix, resilience), ‘space’ (community provision, use of technology) and ‘stuff’ (medicines and equipment, personal protective equipment). We conclude that hospice and palliative services have an essential role in the response to COVID-19 by: 1) responding rapidly and flexibly; 2) ensuring protocols for symptom management are available, and training non-specialists in their use; 3) being involved in triage; 4) considering shifting resources into the community; 5) considering redeploying volunteers to provide psychosocial and bereavement care; 6) facilitating camaraderie among staff and adopt measures to deal with stress; 7) using technology to communicate with patients and carers; 8) adopting standardised data collection systems to inform operational changes and improve care.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 0.275833 |
| lj8nckpg | Pediatricians are ready for COVID‐19 and other emergencies                                                                                                                                                                                                                                                                                                                                               | In January — just as COVID‐19 was on the verge of being a worldwide pandemic — the American Academy of Pediatrics (AAP) and emergency medicine associations released a joint policy statement, “Pediatric Readiness in Emergency Medical Services Systems,” designed to bolster the emergency medical system so that children are well cared for.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       | 0.274159 |
| 4kzv06e6 | Healthcare worker absenteeism, child care costs, and COVID-19 school closures: a simulation analysis                                                                                                                                                                                                                                                                                                     | School closures have been enacted as a measure of mitigation during the ongoing COVID-19 pandemic due to their ability to reduce transmission. It has been shown that school closures could cause absenteeism amongst healthcare workers with dependent children, but there is a need for detailed high-resolution analysis of the relationship between school closures and healthcare worker absenteeism to inform local community preparedness. We provide national- and county-level simulations of school closures and absenteeism across the United States. At the national level, we estimate the projected absenteeism rate to range from 7.5% to 8.6%, and the effectiveness of school closures to range from 172 to 218 fewer hospital beds used per 100,000 people at peak demand. At the county-level, we find substantial variations of projected absenteeism and school closure effects, ranging from 2.0% to 18.6% absenteeism and 88 to 280 fewer hospital beds used per 100,000 people at peak demand. We also find significant associations between levels of absenteeism and COVID-19 complication factors. We observe from our models that an estimated 98.8% of counties would find it less expensive to provide child care to all healthcare workers with children than to bear the costs of healthcare worker absenteeism during school closures, identifying child care subsidization as a potential solution to help maintain healthcare systems during a pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 0.274093 |
| uyb45nye | Teaching Professional Formation in Response to the COVID-19 Pandemic                                                                                                                                                                                                                                                                                                                                     | In response to the COVID-19 pandemic, the Association of American Medical Colleges has called for a temporary suspension of clinical teaching activities for medical students. Planning for the continued involvement of learners in patient care during this pandemic should include teaching learners professional formation. The authors provide an ethical framework to guide such teaching, based on the ethical principle of beneficence and the professional virtues of courage and self-sacrifice from professional ethics in medicine. The authors show that these concepts support the conclusion that learners are ethically obligated to accept reasonable, but not unreasonable, risk. Based on this ethical framework, the authors provide an account of the process of teaching professional formation that medical educators and academic leaders should implement. Medical educators and academic leaders should embrace the opportunity that the COVID-19 pandemic presents for teaching professional formation. Learners should acquire the conceptual vocabulary of professional formation. Learners should recognize that risk of infection from patients is unavoidable. Learners should become aware of established ethical standards for professional responsibility during epidemics from the history of medicine. Learners should master understandable fear. Medical educators and academic leaders should ensure that didactic teaching of professional formation continues when it becomes justified to end learners’ participation in the processes of patient care; topics should include the professionally responsible management of scarce medical resources. The COVID-19 pandemic will not be the last major infectious disease that puts learners at risk. Professional ethics in medicine provides powerful conceptual tools that can be used as an ethical framework to guide medical educators to teach learners, who will bear leadership responsibilities in responses to future pandemics, professional formation.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0.268858 |
| mhwqgjbg | One Academic Health System’s Early (and Ongoing) Experience Responding to COVID-19: Recommendations From the Initial Epicenter of the Pandemic in the United States                                                                                                                                                                                                                                      | On January 19, 2020, the first case of a patient with coronavirus disease 2019 (COVID-19) in the United States was reported in Washington State. On February 29, 2020, a patient infected with severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) passed away in a hospital in Seattle-King County, the first reported COVID-19-related death in the United States. That same day, a skilled nursing and rehabilitation facility in the county reported that several of its residents tested positive for SARS-CoV-2 and that many staff had symptoms compatible with COVID-19. The University of Washington Medicine health system (UW Medicine), which is based in Seattle-King County and provides quaternary care for the region, was one of several health care organizations called upon to address this growing crisis. What ensued was a series of swiftly enacted decisions and activities at UW Medicine, in partnership with local, state, and national public health agencies, to respond to the COVID-19 pandemic. Tapping into the multipronged mission areas of academic medicine, UW Medicine worked to support the community, innovate in science and clinical practice; lead policy and practice guideline development; and adopt changes as the crisis unfolded. In doing so, health system leaders had to balance their commitments to students, residents and fellows, researchers, faculty, staff, and hospital and health center entities, while ensuring that patients continued to receive cutting-edge, high-quality, safe care. In this Invited Commentary, the authors highlight the work and challenges UW Medicine has faced in responding to the global COVID-19 pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 0.265445 |
| h3v0334o | Advancing computerized cognitive training for early Alzheimer's disease in a pandemic and post-pandemic world                                                                                                                                                                                                                                                                                            | The COVID-19 pandemic has transformed mobile health applications and telemedicine from nice to have tools into essential healthcare infrastructure. This need is particularly great for the elderly who, due to their greater risk for infection, may avoid medical facilities or be required to self-isolate. These are also the very groups at highest risk for cognitive decline. For example, during the COVID-19 pandemic artificially intelligent conversational agents were employed by hospitals and government agencies (such as the CDC) to field queries from patients about symptoms and treatments. Digital health tools also proved invaluable to provide neuropsychiatric and psychological self-help to people isolated at home or in retirement centers and nursing homes.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             | 0.262772 |
| agnmfkoq | US Equity Risk Premiums during the COVID-19 Pandemic                                                                                                                                                                                                                                                                                                                                                     | We study equity risk premiums in the United States during the COVID-19 pandemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        | 0.209664 |

### Search documents based on query ###

In [84]:
display(box3)

Box(children=(Label(value='Search topic:'), Dropdown(description='Aspect:', index=3, layout=Layout(width='max-…

transmission risk


|          | title                                                                         | abstract                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
|:---------|:------------------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| nwl95ysz | Risk of COVID-19 importation to the Pacific islands through global air travel | On 30 January 2020, WHO declared coronavirus (COVID-19) a global public health emergency. As of 12 March 2020, 125 048 confirmed COVID-19 cases in 118 countries had been reported. On 12 March 2020, the first case in the Pacific islands was reported in French Polynesia; no other Pacific island country or territory has reported cases. <strong>The purpose of our analysis is to show how travellers may introduce COVID-19 into the Pacific islands and discuss the role robust health systems play in protecting health and reducing <strong style="color: red">transmission</strong> <strong style="color: red">risk</strong>.</strong> <strong>We analyse travel and Global Health Security Index data using a scoring tool to produce quantitative estimates of COVID-19 importation <strong style="color: red">risk</strong>, by departing and arriving country.</strong> Our analysis indicates that, as of 12 March 2020, the highest <strong style="color: red">risk</strong> air routes by which COVID-19 may be imported into the Pacific islands are from east Asian countries (specifically, China, Korea and Japan) to north Pacific airports (likely Guam, Commonwealth of the Northern Mariana Islands or, to a less extent, Palau); or from China, Japan, Singapore, the United States of America or France to south Pacific ports (likely, Fiji, Papua New Guinea, French Polynesia or New Caledonia). Other importation routes include from other east Asian countries to Guam, and from Australia, New Zealand and other European countries to the south Pacific. The tool provides a useful method for assessing COVID-19 importation <strong style="color: red">risk</strong> and may be useful in other settings. |

### Search documents based on pre-defined query ###

In [85]:
display(box4)

Box(children=(Label(value='Search topic:'), Dropdown(description='Aspect:', index=3, layout=Layout(width='max-…

transmission


|          | title                                                                                                                                                                                           | abstract                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
|:---------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| nwl95ysz | Risk of COVID-19 importation to the Pacific islands through global air travel                                                                                                                   | On 30 January 2020, WHO declared coronavirus (COVID-19) a global public health emergency. As of 12 March 2020, 125 048 confirmed COVID-19 cases in 118 countries had been reported. On 12 March 2020, the first case in the Pacific islands was reported in French Polynesia; no other Pacific island country or territory has reported cases. <strong>The purpose of our analysis is to show how travellers may introduce COVID-19 into the Pacific islands and discuss the role robust health systems play in protecting health and reducing <strong style="color: red">transmission</strong> risk.</strong> <strong>We analyse travel and Global Health Security Index data using a scoring tool to produce quantitative estimates of COVID-19 importation risk, by departing and arriving country.</strong> Our analysis indicates that, as of 12 March 2020, the highest risk air routes by which COVID-19 may be imported into the Pacific islands are from east Asian countries (specifically, China, Korea and Japan) to north Pacific airports (likely Guam, Commonwealth of the Northern Mariana Islands or, to a less extent, Palau); or from China, Japan, Singapore, the United States of America or France to south Pacific ports (likely, Fiji, Papua New Guinea, French Polynesia or New Caledonia). Other importation routes include from other east Asian countries to Guam, and from Australia, New Zealand and other European countries to the south Pacific. The tool provides a useful method for assessing COVID-19 importation risk and may be useful in other settings.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| eqo633am | Novel Coronavirus in Cape Town Informal Settlements: Feasibility of Using Informal Dwelling Outlines to Identify High Risk Areas for COVID-19 Transmission From A Social Distancing Perspective | BACKGROUND: The challenges faced by the Global South during the coronavirus disease (COVID-19) pandemic are compounded by the presence of informal settlements, which are typically densely populated and lacking in formalized sanitation infrastructure. Social distancing measures in informal settlements may be difficult to implement due to the density and layout of settlements. <strong>This study measures the distance between dwellings in informal settlements in Cape Town to identify the risk of COVID-19 <strong style="color: red">transmission</strong>.</strong> <strong>OBJECTIVE: The aim of this paper is to determine if social distancing measures are achievable in informal settlements in Cape Town, using two settlements as an example.</strong> <strong>We will first examine the distance between dwellings and their first, second, and third nearest neighbors and then identify clusters of dwellings in which residents would be unable to effectively practice social isolation due to the close proximity of their homes.</strong> METHODS: Dwellings in the settlements of Masiphumelele and Klipfontein Glebe were extracted from a geographic information system data set of outlines of all informal dwellings in Cape Town. The distance to each dwelling’s first, second, and third nearest neighbors was calculated for each settlement. A social distance measure of 2 m was used (buffer of 1 m, as dwellings less than 2 m apart are joined) to identify clusters of dwellings that are unable to effectively practice social distancing in each settlement. RESULTS: The distance to each dwelling’s first 3 nearest neighbors illustrates that the settlement of Masiphumelele is constructed in a denser fashion as compared to the Klipfontein Glebe settlement. This implies that implementing social distancing will likely be more challenging in Masiphumelele than in Klipfontein Glebe. However, using a 2-m social distancing measure, it was demonstrated that large portions of Klipfontein Glebe would also be unable to effectively implement social distancing. CONCLUSIONS: Effectively implementing social distancing may be a challenge in informal settlements due to their density. <strong>This paper uses dwelling outlines for informal settlements in the city of Cape Town to demonstrate that with a 2 m measure, effective social distancing will be challenging.</strong>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| un1y8r5a | Vertical Transmission of Coronavirus Disease 19 (COVID-19) from Infected Pregnant Mothers to Neonates: A Review                                                                                 | Background: Since early December 2019, the Coronavirus Disease 19 (COVID-19) infection has been prevalent in China and eventually spread to other countries. There are a few published cases of COVID-19 occurring during pregnancy and due the possibility of mother-fetal vertical <strong style="color: red">transmission</strong>, there is a concern that the fetuses may be at risk of congenital COVID-19. <strong>Methods: We reviewed the risk of vertical <strong style="color: red">transmission</strong> of COVID-19 to the fetus of infected mothers by using data of published articles or official websites up to March 4, 2020.</strong> Results: A total of 31 infected pregnant mothers with COVID-19 were reported. No COVID-19 infection was detected in their neonates or placentas. Two mothers died from COVID-19-related respiratory complications after delivery. Conclusions: Currently, based on limited data, there is no evidence for intrauterine <strong style="color: red">transmission</strong> of COVID-19 from infected pregnant women to their fetuses. Mothers may be at increased risk for more severe respiratory complications.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| w7nfbnbb | Review article: gastrointestinal features in COVID‐19 and the possibility of faecal transmission                                                                                                | BACKGROUND: There is little published evidence on the gastrointestinal features of COVID‐19. <strong>AIMS: To report on the gastrointestinal manifestations and pathological findings of patients with COVID‐19, and to discuss the possibility of faecal <strong style="color: red">transmission</strong>.</strong> METHODS: We have reviewed gastrointestinal features of, and faecal test results in, COVID‐19 from case reports and retrospective clinical studies relating to the digestive system published since the outbreak. RESULTS: With an incidence of 3% (1/41)‐79% (159/201), gastrointestinal symptoms of COVID‐19 included anorexia 39.9% (55/138)‐50.2% (101/201), diarrhoea 2% (2/99)‐49.5% (146/295), vomiting 3.6% (5/138)‐66.7% (4/6), nausea 1% (1/99)‐29.4% (59/201), abdominal pain 2.2% (3/138)‐6.0% (12/201) and gastrointestinal bleeding 4% (2/52)‐13.7% (10/73). Diarrhoea was the most common gastrointestinal symptom in children and adults, with a mean duration of 4.1 ± 2.5 days, and was observed before and after diagnosis. Vomiting was more prominent in children. About 3.6% (5/138)‐15.9% (32/201) of adult and 6.5% (2/31)‐66.7% (4/6) of children patients presented vomiting. Adult and children patients can present with digestive symptoms in the absence of respiratory symptoms. The incidence of digestive manifestations was higher in the later than in the early stage of the epidemic, but no differences in digestive symptoms among different regions were found. Among the group of patients with a higher proportion of severe cases, the proportion of gastrointestinal symptoms in severe patients was higher than that in nonsevere patients (anorexia 66.7% vs 30.4%; abdominal pain 8.3% vs 0%); while in the group of patients with a lower severe rate, the proportion with gastrointestinal symptoms was similar in severe and nonsevere cases (nausea and vomiting 6.9% vs 4.6%; diarrhoea 5.8% vs 3.5%). Angiotensin converting enzyme 2 and virus nucleocapsid protein were detected in gastrointestinal epithelial cells, and infectious virus particles were isolated from faeces. Faecal PCR testing was as accurate as respiratory specimen PCR detection. In 36% (5/14)‐53% (39/73) faecal PCR became positive, 2‐5 days later than sputum PCR positive. Faecal excretion persisted after sputum excretion in 23% (17/73)‐82% (54/66) patients for 1‐11 days. CONCLUSIONS: Gastrointestinal symptoms are common in patients with COVID‐19, and had an increased prevalence in the later stage of the recent epidemic in China. SARS‐CoV‐2 enters gastrointestinal epithelial cells, and the faeces of COVID‐19 patients are potentially infectious.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| l5q5wc06 | Epidemiology, virology, and clinical features of severe acute respiratory syndrome -coronavirus-2 (SARS-CoV-2; Coronavirus Disease-19)                                                          | A cluster of severe pneumonia of unknown etiology in Wuhan City, Hubei province in China emerged in December 2019. A novel coronavirus named severe acute respiratory syndrome coronavirus-2 (SARS-CoV-2) was isolated from lower respiratory tract sample as the causative agent. The current outbreak of infections with SARS-CoV-2 is termed Coronavirus Disease 2019 (COVID-19) by the World Health Organization (WHO). COVID-19 rapidly spread into at least 114 countries and killed more than 4,000 people by March 11 2020. WHO officially declared COVID-19 a pandemic on March 11, 2020. There have been 2 novel coronavirus outbreaks in the past 2 decades. The outbreak of severe acute respiratory syndrome (SARS) in 2002–2003 caused by SARS-CoV had a case fatality rate of around 10% (8,098 confirmed cases and 774 deaths), while Middle East respiratory syndrome (MERS) caused by MERS-CoV killed 861 people out of a total 2,502 confirmed cases between 2012 and 2019. <strong>The purpose of this review is to summarize known-to-date information about SARS-CoV-2, <strong style="color: red">transmission</strong> of SARS-CoV-2, and clinical features.</strong>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| ldf7uso2 | Role of changes in SARS-CoV-2 spike protein in the interaction with the human ACE2 receptor: An in silico analysis                                                                              | Many human viral diseases are a consequence of a zoonotic event. Some of the diseases caused by these zoonotic events have affected millions of people around the world, some of which have resulted in high rates of morbidity/mortality in humans. Changes in the viral proteins that function as ligands of the host receptor may promote the spillover between species. The most recent of these zoonotic events that have caused an ongoing epidemic of high magnitude is the Covid-19 epidemics caused by SARS-CoV-2. <strong>The aim of this study was to determine the mutation(s) in the sequence of the spike protein of the SARS-CoV-2 that might be favoring human to human <strong style="color: red">transmission</strong>.</strong> An in silico approach was performed, and changes were detected in the S1 subunit of the receptor-binding domain of spike. The observed changes have significant effect on SARS-CoV-2 spike/ACE2 interaction and produce a reduction in the binding energy, compared to the one of the Bat-CoV to this receptor. The data presented in this study suggest a higher affinity of the SARS-Cov-2 spike protein to the human ACE2 receptor, compared to the one of Bat-CoV spike and ACE2. This could be the cause of the rapid viral spread of SARS-CoV-2 in humans.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| lfndq85x | Zoonotic origins of human coronaviruses                                                                                                                                                         | Mutation and adaptation have driven the co-evolution of coronaviruses (CoVs) and their hosts, including human beings, for thousands of years. Before 2003, two human CoVs (HCoVs) were known to cause mild illness, such as common cold. The outbreaks of severe acute respiratory syndrome (SARS) and the Middle East respiratory syndrome (MERS) have flipped the coin to reveal how devastating and life-threatening an HCoV infection could be. The emergence of SARS-CoV-2 in central China at the end of 2019 has thrusted CoVs into the spotlight again and surprised us with its high transmissibility but reduced pathogenicity compared to its sister SARS-CoV. HCoV infection is a zoonosis and understanding the zoonotic origins of HCoVs would serve us well. Most HCoVs originated from bats where they are non-pathogenic. The intermediate reservoir hosts of some HCoVs are also known. Identifying the animal hosts has direct implications in the prevention of human diseases. Investigating CoV-host interactions in animals might also derive important insight on CoV pathogenesis in humans. <strong>In this review, we present an overview of the existing knowledge about the seven HCoVs, with a focus on the history of their discovery as well as their zoonotic origins and interspecies <strong style="color: red">transmission</strong>.</strong> Importantly, we compare and contrast the different HCoVs from a perspective of virus evolution and genome recombination. The current CoV disease 2019 (COVID-19) epidemic is discussed in this context. In addition, the requirements for successful host switches and the implications of virus evolution on disease severity are also highlighted.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| v8i97mbx | Brief Review on COVID-19: The 2020 Pandemic Caused by SARS-CoV-2                                                                                                                                | Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) is the virus responsible for the coronavirus disease of 2019 (COVID-19). First identified in Wuhan (Hubei, China) in December of 2019, it has since been declared a pandemic by the World Health Organization in March of 2020. <strong>In this study, we will provide a brief review of viral origin, identification, symptoms, <strong style="color: red">transmission</strong>, diagnosis, and potential treatment strategies for the newly identified SARS-CoV-2 strain.</strong>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 2ftw85xw | Facing the COVID-19 outbreak: What should we know and what could we do?                                                                                                                         | A novel coronavirus, severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), emerged in Wuhan, Hubei Province in China in December 2019 and caused a serious type of pneumonia called coronavirus disease 2019 or COVID-19. This epidemic quickly spread across China and extended to more than 20 other countries. <strong>This commentary discusses the reasons for the fast spread of SARS-CoV-2 in three aspects: the infectious sources, including the biological nature of the virus; the susceptible population; and the <strong style="color: red">transmission</strong> routes.</strong> The current situations and suggestions regarding the control of the disease are summarized. This article is protected by copyright. All rights reserved.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| frkk6w0a | The transmission and diagnosis of 2019 novel coronavirus infection disease (COVID-19): A Chinese perspective                                                                                    | 2019 novel coronavirus (SARS-CoV-2), which originated in Wuhan, China, has attracted the world's attention over the last month. The Chinese government has taken emergency measures to control the outbreak and has undertaken initial steps in the diagnosis and treatment of 2019 novel coronavirus infection disease (COVID-19). However, SARS-CoV-2 possesses powerful pathogenicity as well as transmissibility and still holds many mysteries that are yet to be solved, such as whether the virus can be transmitted by asymptomatic patients or by mothers to their infants. <strong>Our research presents selected available cases of COVID-19 in China to better understand the <strong style="color: red">transmission</strong> and diagnosis regarding this infectious disease.</strong> This article is protected by copyright. All rights reserved.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| iff8cuum | Clinical characteristics of 24 asymptomatic infections with COVID-19 screened among close contacts in Nanjing, China                                                                            | Previous studies have showed clinical characteristics of patients with the 2019 novel coronavirus disease (COVID-19) and the evidence of person-to-person <strong style="color: red">transmission</strong>. Limited data are available for asymptomatic infections. <strong>This study aims to present the clinical characteristics of 24 cases with asymptomatic infection screened from close contacts and to show the <strong style="color: red">transmission</strong> potential of asymptomatic COVID-19 virus carriers.</strong> Epidemiological investigations were conducted among all close contacts of COVID-19 patients (or suspected patients) in Nanjing, Jiangsu Province, China, from Jan 28 to Feb 9, 2020, both in clinic and in community. Asymptomatic carriers were laboratory-confirmed positive for the COVID-19 virus by testing the nucleic acid of the pharyngeal swab samples. Their clinical records, laboratory assessments, and chest CT scans were reviewed. As a result, none of the 24 asymptomatic cases presented any obvious symptoms while nucleic acid screening. Five cases (20.8%) developed symptoms (fever, cough, fatigue, etc.) during hospitalization. Twelve (50.0%) cases showed typical CT images of ground-glass chest and 5 (20.8%) presented stripe shadowing in the lungs. The remaining 7 (29.2%) cases showed normal CT image and had no symptoms during hospitalization. These 7 cases were younger (median age: 14.0 years; P=0.012) than the rest. None of the 24 cases developed severe COVID-19 pneumonia or died. The median communicable period, defined as the interval from the first day of positive nucleic acid tests to the first day of continuous negative tests, was 9.5 days (up to 21 days among the 24 asymptomatic cases). Through epidemiological investigation, we observed a typical asymptomatic <strong style="color: red">transmission</strong> to the cohabiting family members, which even caused severe COVID-19 pneumonia. Overall, the asymptomatic carriers identified from close contacts were prone to be mildly ill during hospitalization. However, the communicable period could be up to three weeks and the communicated patients could develop severe illness. These results highlighted the importance of close contact tracing and longitudinally surveillance via virus nucleic acid tests. Further isolation recommendation and continuous nucleic acid tests may also be recommended to the patients discharged.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| mvm1t0y7 | Active Monitoring of Persons Exposed to Patients with Confirmed COVID-19 — United States, January–February 2020                                                                                 | In December 2019, an outbreak of coronavirus disease 2019 (COVID-19), caused by the virus SARS-CoV-2, began in Wuhan, China (1). The disease spread widely in China, and, as of February 26, 2020, COVID-19 cases had been identified in 36 other countries and territories, including the United States. Person-to-person <strong style="color: red">transmission</strong> has been widely documented, and a limited number of countries have reported sustained person-to-person spread. * On January 20, state and local health departments in the United States, in collaboration with teams deployed from CDC, began identifying and monitoring all persons considered to have had close contact† with patients with confirmed COVID-19 (2). <strong>The aims of these efforts were to ensure rapid evaluation and care of patients, limit further <strong style="color: red">transmission</strong>, and better understand risk factors for <strong style="color: red">transmission</strong>.</strong>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| r9mv4anf | [Investigation and analysis on characteristics of a cluster of COVID-19 associated with exposure in a department store in Tianjin]                                                              | <strong>Objective: To describe the epidemiological characteristics of a cluster of COVID-19 cases reported in Baodi district of Tianjin as of 18 February, 2020, which might be associated with the exposure in a local department store, and provide suggestions for prevention and control strategy development.</strong> <strong>Methods: The basic characteristics, time and area distributions, clinical manifestations, epidemiological history and <strong style="color: red">transmission</strong> mode of the COVID-19 cases associated with the department store exposure were analyzed.</strong> Results: A total of 40 COVID-19 cases were associated with the department store exposure, accounting for 75.47% of the total confirmed cases (53 cases) reported in Baodi district. The cases were mainly at the age of 60 years or older (35.00%) and farmers (40.00%). The main clinical manifestations included fever (95.00%), cough (35.00%), and diarrhea (15.00%). The proportion of confirmed severe cases was 32.50%. The incidence curve showed that the incidence peak occurred on 31 January, 2020. Among the 40 cases, 6(15.00%) were department store employees, 19(47.50%) were customers and 15(37.50%) were close contacts (secondary cases). The first case occurred on 21 January, 2020, this case was a department store employee who had a purchasing history at whole sale markets in other provinces and cities before the onset, and 3 employees were still on duty after symptom onsets. The median of the incubation period of customer cases was 6 days, and the median of the interval between onset and medical treatment of customer cases was 7 days. Conclusion: This was a cluster epidemic of COVID-19, which might be associated with the exposure in the department store. By now, the current prevention and control measures have achieved satisfied effects.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| voofruqm | [Epidemiological analysis on a family cluster of COVID-19]                                                                                                                                      | <strong>Objective: To understand the possible <strong style="color: red">transmission</strong> route of a family cluster of COVID-19 in Zhengzhou and the potential infectivity of COVID-19 in incubation period, and provide scientific evidence for the timely control of infectious source and curb the spread of the epidemic.</strong> Methods: Epidemiological investigation was conducted for a family cluster of COVID-19 (8 cases) with descriptive epidemiological method, and respiratory tract samples of the cases were collected for the nucleic acid detection of 2019-nCoV by RT-PCR. Results: Two primary cases, which occurred on 31 January and 1 February, 2020, respectively, had a common exposure history in Wuhan. The other six family members had onsets on 30 January, 31 January, 1 February (three cases) and 3 February, 2020. Conclusions: In this family cluster of COVID-19, six family members were infected through common family exposure to the 2 primary cases. Five secondary cases had onsets earlier than or on the same day as the primary cases, indicating that COVID-19 is contagious in incubation period, and the home isolation in the early phase of the epidemic might lead to the risk of family cluster of COVID-19.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| y1d2y9c0 | Coronavirus disease 2019 (COVID-19) in pregnant women: A report based on 116 cases                                                                                                              | Abstract Background The coronavirus disease 2019 (COVID-19), caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), is a global public health emergency. Data on the effect of COVID-19 in pregnancy are limited to small case series. <strong>Objectives To evaluate the clinical characteristics and outcomes in pregnancy and the vertical <strong style="color: red">transmission</strong> potential of SARS-CoV-2 infection.</strong> Study Desigh Clinical records were retrospectively reviewed for 116 pregnant women with COVID-19 pneumonia from 25 hospitals in China between January 20 and March 24, 2020. Evidence of vertical <strong style="color: red">transmission</strong> was assessed by testing for SARS-CoV-2 in amniotic fluid, cord blood, and neonatal pharyngeal swab samples. Results The median gestational age on admission was 38+0 (IQR 36+0-39+1) weeks. The most common symptoms were fever (50.9%, 59/116) and cough (28.4%, 33/116); 23.3% (27/116) patients presented without symptoms. Abnormal radiologic findings were found in 96.3% (104/108) of cases. There were eight cases (6.9%, 8/116) of severe pneumonia but no maternal deaths. One of eight patients (1/8) that presented in the first- and early-second-trimester had a missed spontaneous abortion. Twenty-one of 99 patients (21.2%, 21/99) that had delivered had preterm birth, including six with preterm premature ruptured of membranes. The rate of spontaneous preterm birth before 37 weeks was 6.1% (6/99). There was one case of severe neonatal asphyxia that resulted in neonatal death. Eighty-six of the 100 neonates that had testing for SARS-CoV-2 had negative results, of these ten neonates had paired amniotic fluid and cord blood samples that were tested negative for SARS-CoV-2. Conclusions SARS-CoV-2 infection during pregnancy is not associated with an increased risk of spontaneous abortion and spontaneous preterm birth. There is no evidence of vertical <strong style="color: red">transmission</strong> of SARS-CoV-2 infection when the infection manifests during the third-trimester of pregnancy.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| y18nru1d | COVID-19 in cardiac arrest and infection risk to rescuers: a systematic review                                                                                                                  | Abstract Background There may be a risk of COVID-19 <strong style="color: red">transmission</strong> to rescuers delivering treatment for cardiac arrest. <strong>The aim of this review was to identify the potential risk of <strong style="color: red">transmission</strong> associated with key interventions (chest compressions, defibrillation, cardiopulmonary resuscitation) to inform international treatment recommendations.</strong> <strong>Methods We undertook a systematic review comprising three questions: 1) aerosol generation associated with key interventions; 2) risk of airborne infection <strong style="color: red">transmission</strong> associated with key interventions; and 3) the effect of different personal protective equipment strategies.</strong> We searched MEDLINE, Embase, Cochrane Central Register of Controlled Trials, and the World Health Organisation COVID-19 database on 24th March 2020. Eligibility criteria were developed individually for each question. We assessed risk of bias for individual studies, and used the GRADE process to assess evidence certainty by outcome. Results We included eleven studies: two cohort studies, one case control study, five case reports, and three manikin randomised controlled trials. We did not find any direct evidence that chest compressions or defibrillation either are or are not associated with aerosol generation or <strong style="color: red">transmission</strong> of infection. Data from manikin studies indicates that donning of personal protective equipment delays treatment delivery. Studies provided only indirect evidence, with no study describing patients with COVID-19. Evidence certainty was low or very low for all outcomes. Conclusion It is uncertain whether chest compressions or defibrillation cause aerosol generation or <strong style="color: red">transmission</strong> of COVID-19 to rescuers. There is very limited evidence and a rapid need for further studies. Review registration: PROSPERO CRD42020175594                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| j7vuoer6 | Recomendaciones para la atención oftalmológica durante el estado de alarma por la pandemia de enfermedad por coronavirus COVID-19                                                               | <strong>ABSTRACT Objective: Minimize exposure to the SARS-CoV-2, reduce the chances of cross-<strong style="color: red">transmission</strong> between patients and healthcare personnel, and prevent the development of postoperative complications from the management of patients with eye diseases during the 2019 coronavirus disease pandemic (COVID -19).</strong> <strong>Methods: COVID-19 literature review and consensus establishment between different Spanish ophthalmology societies in order to provide guidelines and recommendations of maximum resources primarily conditioned by the state of alert, confinement and social distancing that occurs in Spain since March 16, 2020.</strong> Results: The recommendations will promote the adoption of action and protection measures for eye care in outpatient clinics, surgical areas and hospitalization, for unconfirmed (asymptomatic and symptomatic) and confirmed COVID-19 patients. Measures must be adapted to the circumstances and availability of Personal Protective Equipment (PPE) in each of the centers and Autonomous Communities, which will be updated according to the pandemic phases and the measures adopted by the Spanish Government. Conclusions: During the COVID-19 pandemic, attention to the potential health risks to the population caused by coronavirus should prevail over the possible progression of the common eye diseases. Ophthalmologists and other eye care professionals must assume a possible progression of these diseases due to the impossibility of adequate patient follow-up.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| tv13furv | New understanding of the damage of SARS-CoV-2 infection outside the respiratory system                                                                                                          | Abstract Since early December 2019, a number of pneumonia cases associated with unknown coronavirus infection were identified in Wuhan, China, and many additional cases were identified in other regions of China and in other countries within 3 months. Currently, more than 80,000 cases have been diagnosed in China, including more than 3,000 deaths. The epidemic is spreading to the rest of the world, posing a grave challenge to prevention and control. On February 12, 2020, the International Committee on Taxonomy of Viruses and the World Health Organization officially named the novel coronavirus and associated pneumonia as severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) and coronavirus disease 2019 (COVID-19), respectively. According to the recent research on SARS-CoV-2, the virus mainly infects the respiratory system but may cause damage to other systems. <strong>In this paper, we will systematically review the pathogenic features, <strong style="color: red">transmission</strong> routes, and infection mechanisms of SARS-CoV-2, as well as any adverse effects on the digestive system, urogenital system, central nervous system, and circulatory system, in order to provide a theoretical and clinical basis for the diagnosis, classification, treatment, and prognosis assessment of SARS-CoV-2 infection.</strong>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| wskmzg7d | Personal Protective Equipment: Current Best Practices for Orthopaedic Teams                                                                                                                     | Abstract The COVID-19 pandemic caused by the SARS-CoV-2 virus is challenging healthcare providers across the world. Current best practices for personal protective equipment (PPE) during this time are rapidly evolving and fluid due to the novel and acute nature of the pandemic and the dearth of high-level evidence. Routine infection control practices augmented by airborne precautions are paramount when treating the COVID-19 positive patient. Best practices for PPE use in patients who have unknown COVID-19 status are a highly charged and emotional issue. The variables to be considered include protection of patients and healthcare providers, accuracy and availability of testing, and responsible use of PPE resources. <strong>This article also explores the concerns of surgeons regarding possible <strong style="color: red">transmission</strong> to their own family members as a result of caring for COVID-19 patients.</strong>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| bklpufud | Novel Approach to Reduce Transmission of COVID-19 During Tracheostomy                                                                                                                           | ABSTRACT Severe infection involving the novel coronavirus 2019 (COVID-19) has been associated with acute respiratory distress syndrome that subsequently requires patients to be intubated and dependent on mechanical ventilation. In the setting of the recent pandemic, there is a greater need to perform tracheostomy for these patients. With the high transmissibility of the virus, there has been an increasing concern for the development of techniques to perform surgical intervention while mitigating the risk for infecting hospital staff. As more data emerge pertaining to viral shedding in various bodily fluids, it has become more important to give special attention to precautions. <strong>In this article, we submit a novel approach for better protection and thus reduced <strong style="color: red">transmission</strong> for tracheostomy in a COVID-19 positive patient.</strong> Importantly, this technique is functional, easy to set up, and can be used for additional operations that involve risk of aerosolization or droplet exposure to operating room staff.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| wzwa3f8f | Coronavirus disease 2019 in pregnancy                                                                                                                                                           | <strong>Abstract Objectives This study aims to compare clinical course and outcomes between pregnant and reproductive-aged non-pregnant women with COVID-19 and assess the vertical <strong style="color: red">transmission</strong> potential of COVID-19 in pregnancy.</strong> Methods Medical records of pregnant and reproductive-aged non-pregnant women hospitalized with COVID-19 from January 15 to March 15, 2020, were retrospectively reviewed. The severity of disease, virus clearance time, and length of hospital stay were measured as the primary interest and the vertical <strong style="color: red">transmission</strong> potential of COVID-19 was also assessed. Results Eighty-two patients (28 pregnant women, 54 reproductive-aged non-pregnant women) with laboratory confirmed COVID-19 were enrolled in this study. Univariate regression indicated no association between pregnancy and the severity of disease (OR 0.73, 95% CI 0.08-5.15; p =0.76), virus clearance time (HR 1.16, 95% CI 0.65-2.01; p =0.62), and length of hospital stay (HR 1.10, 95% CI 0.66-1.84; p =0.71). There were 22 pregnant women delivered 23 live births either by cesarean section (17, 60.7%) or vaginal delivery (5, 17.9%) and no neonate was infected with SARS-CoV-2. Conclusions Pregnant women have comparable clinical course and outcomes compared with reproductive-aged non-pregnant women when infected with SARS-CoV-2. No evidence supported vertical <strong style="color: red">transmission</strong> of COVID-19 in the late stage of pregnancy including vaginal delivery.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 0rlcc1wt | Gastrointestinal, Hepatobiliary, and Pancreatic Manifestations of COVID-19                                                                                                                      | Abstract There is an increasing number of confirmed cases and deaths caused by the Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) contributing to the Coronavirus disease 2019 (COVID-19) pandemic. At this point, the need for further disease characterization is critical. COVID-19 is well established as a respiratory tract pathogen; however, recent studies have shown an increasing number of patients reporting gastrointestinal manifestations such as diarrhea, nausea, vomiting, and abdominal pain. The time from onset of gastrointestinal symptoms to hospital presentation is often delayed compared to that of respiratory symptoms. It has been noted that SARS-CoV-2 RNA can be detected in fecal matter for an extended period of time, even after respiratory samples have tested negative and patients are asymptomatic. <strong>In this article, SARS-CoV-2 and its disease COVID-19 will be reviewed with consideration of the latest literature about gastrointestinal symptomatology, the mechanisms by which the virus may inflict damage, and the possibility of viral replication contributing to a fecal-oral route of <strong style="color: red">transmission</strong>.</strong>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| bctb0ewu | Effect of changing case definitions for COVID-19 on the epidemic curve and transmission parameters in mainland China: a modelling study                                                         | Summary Background When a new infectious disease emerges, appropriate case definitions are important for clinical diagnosis and for public health surveillance. Tracking case numbers over time is important to establish the speed of spread and the effectiveness of interventions. <strong>We aimed to assess whether changes in case definitions affected inferences on the <strong style="color: red">transmission</strong> dynamics of coronavirus disease 2019 (COVID-19) in China.</strong> <strong>Methods We examined changes in the case definition for COVID-19 in mainland China during the first epidemic wave.</strong> We used exponential growth models to estimate how changes in the case definitions affected the number of cases reported each day. We then inferred how the epidemic curve would have appeared if the same case definition had been used throughout the epidemic. Findings From Jan 15 to March 3, 2020, seven versions of the case definition for COVID-19 were issued by the National Health Commission in China. We estimated that when the case definitions were changed, the proportion of infections being detected as cases increased by 7·1 times (95% credible interval [CrI] 4·8–10·9) from version 1 to 2, 2·8 times (1·9–4·2) from version 2 to 4, and 4·2 times (2·6–7·3) from version 4 to 5. If the fifth version of the case definition had been applied throughout the outbreak with sufficient testing capacity, we estimated that by Feb 20, 2020, there would have been 232 000 (95% CrI 161 000–359 000) confirmed cases in China as opposed to the 55 508 confirmed cases reported. Interpretation The case definition was initially narrow and was gradually broadened to allow detection of more cases as knowledge increased, particularly milder cases and those without epidemiological links to Wuhan, China, or other known cases. These changes should be taken into account when making inferences on epidemic growth rates and doubling times, and therefore on the reproductive number, to avoid bias. Funding Health and Medical Research Fund, Hong Kong.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 39mfts0g | Impact assessment of non-pharmaceutical interventions against coronavirus disease 2019 and influenza in Hong Kong: an observational study                                                       | Summary Background A range of public health measures have been implemented to suppress local <strong style="color: red">transmission</strong> of coronavirus disease 2019 (COVID-19) in Hong Kong. <strong>We examined the effect of these interventions and behavioural changes of the public on the incidence of COVID-19, as well as on influenza virus infections, which might share some aspects of <strong style="color: red">transmission</strong> dynamics with COVID-19.</strong> Methods We analysed data on laboratory-confirmed COVID-19 cases, influenza surveillance data in outpatients of all ages, and influenza hospitalisations in children. We estimated the daily effective reproduction number (R t) for COVID-19 and influenza A H1N1 to estimate changes in transmissibility over time. Attitudes towards COVID-19 and changes in population behaviours were reviewed through three telephone surveys done on Jan 20–23, Feb 11–14, and March 10–13, 2020. Findings COVID-19 transmissibility measured by R t has remained at approximately 1 for 8 weeks in Hong Kong. Influenza <strong style="color: red">transmission</strong> declined substantially after the implementation of social distancing measures and changes in population behaviours in late January, with a 44% (95% CI 34–53%) reduction in transmissibility in the community, from an estimated R t of 1·28 (95% CI 1·26–1·30) before the start of the school closures to 0·72 (0·70–0·74) during the closure weeks. Similarly, a 33% (24–43%) reduction in transmissibility was seen based on paediatric hospitalisation rates, from an R t of 1·10 (1·06–1·12) before the start of the school closures to 0·73 (0·68–0·77) after school closures. Among respondents to the surveys, 74·5%, 97·5%, and 98·8% reported wearing masks when going out, and 61·3%, 90·2%, and 85·1% reported avoiding crowded places in surveys 1 (n=1008), 2 (n=1000), and 3 (n=1005), respectively. Interpretation Our study shows that non-pharmaceutical interventions (including border restrictions, quarantine and isolation, distancing, and changes in population behaviour) were associated with reduced <strong style="color: red">transmission</strong> of COVID-19 in Hong Kong, and are also likely to have substantially reduced influenza <strong style="color: red">transmission</strong> in early February, 2020. Funding Health and Medical Research Fund, Hong Kong.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| hp8pswt7 | What are the Underlying Transmission Patterns of COVID-19 Outbreak? – An Age-specific Social Contact Characterization                                                                           | Abstract Background COVID-19 has spread to 6 continents. Now is opportune to gain a deeper understanding of what may have happened. The findings can help inform mitigation strategies in the disease-affected countries. <strong>Methods In this work, we examine an essential factor that characterizes the disease <strong style="color: red">transmission</strong> patterns: the interactions among people.</strong> We develop a computational model to reveal the interactions in terms of the social contact patterns among the population of different age-groups. We divide a city's population into seven age-groups: 0-6 years old (children); 7-14 (primary and junior high school students); 15-17 (high school students); 18-22 (university students); 23-44 (young/middle-aged people); 45-64 years old (middle-aged/elderly people); and 65 or above (elderly people). We consider four representative settings of social contacts that may cause the disease spread: (1) individual households; (2) schools, including primary/high schools as well as colleges and universities; (3) various physical workplaces; and (4) public places and communities where people can gather, such as stadiums, markets, squares, and organized tours. A contact matrix is computed to describe the contact intensity between different age-groups for each of the four settings. By integrating the four contact matrices with the next-generation matrix, we quantitatively characterize the underlying <strong style="color: red">transmission</strong> patterns of COVID-19 among different populations. Findings We focus our study on 6 representative cities in China: Wuhan, the epicenter of COVID-19, together with Beijing, Tianjin, Hangzhou, Suzhou, and Shenzhen, which are five major cities from three key economic zones. The results show that the social contact-based analysis can readily explain the underlying disease <strong style="color: red">transmission</strong> patterns as well as the associated risks (including both confirmed and unconfirmed cases). In Wuhan, the age-groups involving relatively intensive contacts in households and public/communities are dispersedly distributed. This can explain why the <strong style="color: red">transmission</strong> of COVID-19 in the early stage mainly took place in public places and families in Wuhan. We estimate that Feb. 11, 2020 was the date with the highest <strong style="color: red">transmission</strong> risk in Wuhan, which is consistent with the actual peak period of the reported case number (Feb. 4-14). Moreover, the surge in the number of new cases reported on Feb. 12-13 in Wuhan can readily be captured using our model, showing its ability in forecasting the potential/unconfirmed cases. We further estimate the disease <strong style="color: red">transmission</strong> risks associated with different work resumption plans in these cities after the outbreak. The estimation results are consistent with the actual situations in the cities with relatively lenient control policies, such as Beijing, and those with strict control policies, such as Shenzhen. Interpretation With an in-depth characterization of age-specific social contact-based <strong style="color: red">transmission</strong>, the retrospective and prospective situations of the disease outbreak, including the past and future <strong style="color: red">transmission</strong> risks, the effectiveness of different interventions, and the disease <strong style="color: red">transmission</strong> risks of restoring normal social activities, are computationally analyzed and reasonably explained. The conclusions drawn from the study not only provide a comprehensive explanation of the underlying COVID-19 <strong style="color: red">transmission</strong> patterns in China, but more importantly, offer the social contact-based risk analysis methods that can readily be applied to guide intervention planning and operational responses in other countries, so that the impact of COVID-19 pandemic can be strategically mitigated. Funding General Research Fund of the Hong Kong Research Grants Council; Key Project Grants of the National Science Foundation of China. |
| gmi1ewc2 | Early dynamics of transmission and control of COVID-19: a mathematical modelling study                                                                                                          | Background: An outbreak of the novel coronavirus SARS-CoV-2 has led to 46,997 confirmed cases as of 13th February 2020. Understanding the early <strong style="color: red">transmission</strong> dynamics of the infection and evaluating the effectiveness of control measures is crucial for assessing the potential for sustained <strong style="color: red">transmission</strong> to occur in new areas. <strong>Methods: We combined a stochastic <strong style="color: red">transmission</strong> model with data on cases of novel coronavirus disease (COVID-19) in Wuhan and international cases that originated in Wuhan to estimate how <strong style="color: red">transmission</strong> had varied over time during January and February 2020.</strong> Based on these estimates, we then calculated the probability that newly introduced cases might generate outbreaks in other areas. Findings: We estimated that the median daily reproduction number, Rt , declined from 2.35 (95% CI: 1.15-4.77) one week before travel restrictions were introduced on 23rd January to 1.05 (95% CI: 0.413-2.39) one week after. Based on our estimates of Rt,we calculated that in locations with similar <strong style="color: red">transmission</strong> potential as Wuhan in early January, once there are at least four independently introduced cases, there is a more than 50% chance the infection will establish within that population. Interpretation: Our results show that COVID-19 <strong style="color: red">transmission</strong> likely declined in Wuhan during late January 2020, coinciding with the introduction of control measures. As more cases arrive in international locations with similar <strong style="color: red">transmission</strong> potential to Wuhan pre-control, it is likely many chains of <strong style="color: red">transmission</strong> will fail to establish initially, but may still cause new outbreaks eventually.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| rj9k9awy | Evidence and characteristics of human-to-human transmission of SARS-CoV-2                                                                                                                       | Background: On December 31, 2019, an outbreak of COVID-19 in humans was reported in Wuhan, and then spread fast to other provinces, China. <strong>We analyzed data from field investigations and genetic sequencing to describe the evidence and characteristics of human-to-human <strong style="color: red">transmission</strong> in Guangdong Province.</strong> Methods: A confirmed COVID-19 case was defined if a suspected case was verified with positive of SARS-CoV-2 in throat swabs, nasal swabs, bronchoalveolar lavage fluid (BALF), or endotracheal aspirates by real-time reverse transcriptase polymerase chain reaction assay (RT-PCR) or genetic sequencing. Field investigations were conducted for each confirmed case. Clinical and demographic data of confirmed cases were collected from medical records. Exposure and travel history were obtained by interview. Results: A total of 1,151 confirmed cases were identified as of February 10, 2020 in Guangdong Province, China. Of them, 697 (60.1%) cases were from 234 cluster infections. Two hundred and fourteen (18.6%) were secondary cases, in which 144 cases were from family cluster infections. With the epidemic continuing, although familial cluster events were dominated, community cluster events increased with a nosocomial event. The whole genomes within the same family cluster infections were identical, and presented a few unique single nucleotide variants (SNVs) compared with SARS-CoV-2 identified on December 2019 in Wuhan. Conclusions: We observed evident human-to-human <strong style="color: red">transmission</strong>s of SARS-CoV-2 in Guangdong, China. Although most of them were from family cluster infections, community and nosocomial infections were increasing. Our findings indicate that human-to-human <strong style="color: red">transmission</strong> risks are transferring from family to community in Guangdong Province.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| kfsuiz5t | Early epidemiological assessment of the transmission potential and virulence of coronavirus disease 2019 (COVID-19) in Wuhan City: China, January-February, 2020                                | Background: Since the first cluster of cases was identified in Wuhan City, China, in December, 2019, coronavirus disease 2019 (COVID-19) has rapidly spread across China, causing multiple introductions in 109 countries/territories/areas as of March 10th. Despite the scarcity of publicly available data, scientists around the world have made strides in estimating the magnitude of the epidemic, the basic reproduction number, and <strong style="color: red">transmission</strong> patterns. Recently more evidence suggests that a substantial fraction of the infected individuals with the novel coronavirus show little if any symptoms, which suggest the need to reassess the <strong style="color: red">transmission</strong> potential of this emerging disease. <strong>In this study, we derive estimates of the transmissibility and virulence of COVID-19 in Wuhan City, China, by reconstructing the underlying <strong style="color: red">transmission</strong> dynamics using multiple data sources.</strong> Methods: We employ statistical methods and publicly available epidemiological datasets to jointly derive estimates of transmissibility and severity associated with the novel coronavirus. For this purpose, the daily series of laboratory-confirmed COVID-19 cases and deaths in Wuhan City and epidemiological data of Japanese evacuees from Wuhan City on board government-chartered flights were integrated into our analysis. Results: Our posterior estimates of basic reproduction number (R) in Wuhan City, China in 2019-2020 reached values as high as 5.20 (95%CrI: 5.04-5.47) and the enhanced public health intervention after January 23rd in 2020 was associated with a declined R at 0.58 (95%CrI: 0.51-0.64), with the total number of infections (i.e. cumulative infections) estimated at 1905526 (95%CrI: 1350283-2655936) in Wuhan City, raising the proportion of infected individuals to 19.1% (95%CrI: 13.5-26.6%). We also found that most recent crude infection fatality ratio (IFR) and time-delay adjusted IFR is estimated to be 0.04% (95% CrI: 0.03-0.06%) and 0.12% (95%CrI: 0.08-0.17%), which is several orders of magnitude smaller than the crude CFR estimated at 4.19% Conclusions: We have estimated key epidemiological parameters of the transmissibility and virulence of COVID-19 in Wuhan, China during January-February, 2020 using an ecological modelling approach. The power of our approach lies in the ability to infer epidemiological parameters with quantified uncertainty from partial observations collected by surveillance systems.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| asyrijyg | Study on SARS-COV-2 transmission and the effects of control measures in China                                                                                                                   | <strong>Objective To reconstruct the <strong style="color: red">transmission</strong> trajectory of SARS-COV-2 and analyze the effects of control measures in China.</strong> Methods Python 3.7.1 was used to write a SEIR class to model the epidemic procedure and a back propagation class to estimate the initial true infected number. The epidemic area in China was divided into three parts, Wuhan city, Hubei province (except Wuhan) and China (except Hubei) based on the different <strong style="color: red">transmission</strong> pattern. A limitation factor for the medical resource was imposed to model the infected but not quarantined. Credible data source from Baidu Qianxi were used to assess the number of infected cases migrated from Wuhan to other areas. Results Basic reproduction number, R0, was 3.6 in the very early stage. The true infected number was 4508 in our model in Wuhan before January 22, 2020. By January 22 2020, it was estimated that 1764 infected cases migrated from Wuhan to other cities in Hubei province. Effective reproductive number, R, gradually decreased from 3.6 (Wuhan, stage 1), 3.4 (Hubei except Wuhan, stage 1) and 3.3 (China except Hubei, stage 1) to 0.67 (Wuhan, stage 4), 0.83 (Hubei except Wuhan, stage 2) and 0.63 (China except Hubei, stage 2), respectively. Especially after January 23, 2020 when Wuhan City was closed, the infected number showed a turning point in Wuhan. By early April, there would be 42073, 21342 and 13384 infected cases in Wuhan, Hubei (except Wuhan) and China (except Hubei) respectively, and there would be 2179, 633 and 107 death in Wuhan, Hubei (except Wuhan) and China (except Hubei) respectively. Conclusion A series of control measures in China have effectively prevented the spread of COVID-19, and the epidemic will end in early April.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 2w3bx6p8 | Estimating number of global importations of COVID-19 from Wuhan, risk of transmission outside mainland China and COVID-19 introduction index between countries outside mainland China           | Background The emergence of a novel coronavirus (SARS-CoV-2) in Wuhan, China in early December 2019 has caused widespread <strong style="color: red">transmission</strong> within the country, with over 1,000 deaths reported to date. Other countries have since reported coronavirus disease 2019 (COVID-19) importation from China, with some experiencing local <strong style="color: red">transmission</strong> and even case importation from countries outside China. <strong>We aim to estimate the number of cases imported from Wuhan to each country or territory outside mainland China, and with these estimates assess the risk of onward local <strong style="color: red">transmission</strong> and the relative potential of case importation between countries outside China.</strong> Methods We used the reported number of cases imported from Wuhan and flight data to generate an uncertainty distribution for the estimated number of imported cases from Wuhan to each location outside mainland China. This uncertainty was propagated to quantify the local outbreak risk using a branching process model. A COVID-19 introduction index was derived for each pair of donor and recipient countries, accounting for the local outbreak risk in the donor country and the between-country connectivity. Results We identified 13 countries or territories outside mainland China that may have under-detected COVID-19 importation from Wuhan, such as Thailand and Indonesia. In addition, 16 countries had a local outbreak risk estimate exceeding 50%, including four outside Asia. The COVID-19 introduction index highlights potential locations outside mainland China from which cases may be imported to each recipient country. Conclusions As SARS-CoV-2 continues to spread globally, more epicentres may emerge outside China. Hence, it is important for countries to remain alert for the possibilities of viral introduction from other countries outside China, even before local <strong style="color: red">transmission</strong> in a source country becomes known.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| ofoqk100 | Clinical Characteristics of 24 Asymptomatic Infections with COVID-19 Screened among Close Contacts in Nanjing, China                                                                            | Background: Previous studies have showed clinical characteristics of patients with the 2019 novel coronavirus disease (COVID-19) and the evidence of person-to-person <strong style="color: red">transmission</strong>. Limited data are available for asymptomatic infections. <strong>This study aims to present the clinical characteristics of 24 cases with asymptomatic infection screened from close contacts and to show the <strong style="color: red">transmission</strong> potential of asymptomatic COVID-19 virus carriers.</strong> Methods: Epidemiological investigations were conducted among all close contacts of COVID-19 patients (or suspected patients) in Nanjing, Jiangsu Province, China, from Jan 28 to Feb 9, 2020, both in clinic and in community. Asymptomatic carriers were laboratory-confirmed positive for the COVID-19 virus by testing the nucleic acid of the pharyngeal swab samples. Their clinical records, laboratory assessments, and chest CT scans were reviewed. Findings: None of the 24 asymptomatic cases presented any obvious symptoms before nucleic acid screening. Five cases (20.8%) developed symptoms (fever, cough, fatigue and etc.) during hospitalization. Twelve (50.0%) cases showed typical CT images of ground-glass chest and five (20.8%) presented stripe shadowing in the lungs. The remaining seven (29.2%) cases showed normal CT image and had no symptoms during hospitalization. These seven cases were younger (median age: 14.0 years; P = 0.012) than the rest. None of the 24 cases developed severe COVID-19 pneumonia or died. The median communicable period, defined as the interval from the first day of positive nucleic acid tests to the first day of continuous negative tests, was 9.5 days (up to 21 days among the 24 asymptomatic cases). Through epidemiological investigation, we observed a typical asymptomatic <strong style="color: red">transmission</strong> to the cohabiting family members, which even caused severe COVID-19 pneumonia. Interpretation: The asymptomatic carriers identified from close contacts were prone to be mildly ill during hospitalization. However, the communicable period could be up to three weeks and the communicated patients could develop severe illness. These results highlighted the importance of close contact tracing and longitudinally surveillance via virus nucleic acid tests. Further isolation recommendation and continuous nucleic acid tests may also be recommended to the patients discharged.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 3e1ji2mw | Risk estimation and prediction by modeling the transmission of the novel coronavirus (COVID-19) in mainland China excluding Hubei province                                                      | Background: In December 2019, an outbreak of novel coronavirus disease (COVID-19) emerged in Wuhan, China and has swiftly spread to other parts of China and a number of oversea countries. <strong>Our aim is to evaluate the effectiveness of the evolution of interventions and self-protection measures, estimate the risk of partial lifting control measures and predict the epidemic trend of the virus in mainland China excluding Hubei province based on the published data and a novel mathematical model.</strong> <strong>Methods: A novel COVID-19 <strong style="color: red">transmission</strong> dynamic model incorporating the intervention measures implemented in China is proposed.</strong> We parameterize the model by using the Markov Chain Monte Carlo (MCMC) method and estimate the control reproduction number Rc, as well as the effective daily reproduction ratio Re(t), of the disease <strong style="color: red">transmission</strong> in mainland China excluding Hubei province. Results: The estimation outcomes indicate that the control reproduction number is 3.36 (95% CI 3.20-3.64) and Re(t) has dropped below 1 since January 31st, 2020, which implies that the containment strategies implemented by the Chinese government in mainland China excluding Hubei province are indeed effective and magnificently suppressed COVID-19 <strong style="color: red">transmission</strong>. Moreover, our results show that relieving personal protection too early may lead to the spread of disease for a longer time and more people would be infected, and may even cause epidemic or outbreak again. By calculating the effective reproduction ratio, we proved that the contact rate should be kept at least less than 30% of the normal level by April, 2020. Conclusions: To ensure the epidemic ending rapidly, it is necessary to maintain the current integrated restrict interventions and self-protection measures, including travel restriction, quarantine of entry, contact tracing followed by quarantine and isolation and reduction of contact, like wearing masks, etc. People should be fully aware of the real-time epidemic situation and keep sufficient personal protection until April. If all the above conditions are met, the outbreak is expected to be ended by April in mainland China apart from Hubei province.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 66ulqu11 | Transmission interval estimates suggest pre-symptomatic spread of COVID-19                                                                                                                      | Background: As the COVID-19 epidemic is spreading, incoming data allows us to quantify values of key variables that determine the <strong style="color: red">transmission</strong> and the effort required to control the epidemic. <strong>We determine the incubation period and serial interval distribution for <strong style="color: red">transmission</strong> clusters in Singapore and in Tianjin.</strong> We infer the basic reproduction number and identify the extent of pre-symptomatic <strong style="color: red">transmission</strong>. Methods: We collected outbreak information from Singapore and Tianjin, China, reported from Jan.19-Feb.26 and Jan.21-Feb.27, respectively. We estimated incubation periods and serial intervals in both populations. Results: The mean incubation period was 7.1 (6.13, 8.25) days for Singapore and 9 (7.92, 10.2)days for Tianjin. Both datasets had shorter incubation periods for earlier-occurring cases. The mean serial interval was 4.56 (2.69, 6.42) days for Singapore and 4.22 (3.43, 5.01) for Tianjin. We inferred that early in the outbreaks, infection was transmitted on average 2.55 and 2.89days before symptom onset (Singapore, Tianjin). The estimated basic reproduction number for Singapore was 1.97 (1.45, 2.48) secondary cases per infective; for Tianjin it was 1.87 (1.65,2.09) secondary cases per infective. Conclusions: Estimated serial intervals are shorter than incubation periods in both Singapore and Tianjin, suggesting that pre-symptomatic <strong style="color: red">transmission</strong> is occurring. Shorter serial intervals lead to lower estimates of R0, which suggest that half of all secondary infections should be prevented to control spread.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| 50zrnau8 | The time scale of asymptomatic transmission affects estimates of epidemic potential in the COVID-19 outbreak                                                                                    | <strong>We assess the impact of asymptomatic <strong style="color: red">transmission</strong> on epidemic potential of novel respiratory pathogens (like COVID-19) -- as measured both by the basic reproduction number (i.e., the expected number of secondary cases generated by an average primary case in a fully susceptible population) and the fraction of new secondary cases attributable to asymptomatic individuals.</strong> We show that the impact of asymptomatic <strong style="color: red">transmission</strong> depends on generation intervals (i.e., time between when an individual is infected and when that individual infects another person). If the generation-interval distribution of asymptomatic <strong style="color: red">transmission</strong> differs from that of symptomatic <strong style="color: red">transmission</strong>, then estimates of the basic reproduction number which do not explicitly account for asymptomatic cases may be systematically biased. Specifically, if asymptomatic cases have a shorter generation interval than symptomatic cases, R_0 will be over-estimated, and if they have a longer generation interval, R_0 will be under-estimated. We also show that as the length of asymptomatic generation intervals increase, estimates of the realized proportion of asymptomatic <strong style="color: red">transmission</strong> during the exponential phase of the epidemic decrease. Our analysis provides a rationale for assessing the duration of asymptomatic cases of COVID-19 in addition to their prevalence in the population.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 0xzsa21a | Prediction of the COVID-19 outbreak based on a realistic stochastic model                                                                                                                       | The current outbreak of coronavirus disease 2019 (COVID-19) has become a global crisis due to its quick and wide spread over the world. A good understanding of the dynamic of the disease would greatly enhance the control and prevention of COVID-19. However, to the best of our knowledge, the unique features of the outbreak have limited the applications of all existing models. <strong>In this paper, a novel stochastic model is proposed which aims to account for the unique <strong style="color: red">transmission</strong> dynamics of COVID-19 and capture the effects of intervention measures implemented in Mainland China.</strong> We find that, (1) instead of aberration, there is a remarkable amount of asymptomatic individuals, (2) an individual with symptoms is approximately twice more likely to pass the disease to others than that of an asymptomatic patient, (3) the <strong style="color: red">transmission</strong> rate has reduced significantly since the implementation of control measures in Mainland China, (4) it is expected that the epidemic outbreak would be contained by early March in the the selected provinces and cities.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| j181i5pr | Strongly heterogeneous transmission of COVID-19 in mainland China: local and regional variation                                                                                                 | Background The outbreak of novel coronavirus disease 2019 (COVID-19) started in the city of Wuhan, China, with a period of rapid initial spread. <strong style="color: red">transmission</strong> on a regional and then national scale was promoted by intense travel during the holiday period of the Chinese New Year. <strong>We studied the variation in <strong style="color: red">transmission</strong> of COVID-19, locally in Wuhan, as well as on a larger spatial scale, among different cities and even among provinces in mainland China.</strong> Methods In addition to reported numbers of new cases, we have been able to assemble detailed contact data for some of the initial clusters of COVID-19. This enabled estimation of the serial interval for clinical cases, as well as reproduction numbers for small and large regions. Findings We estimated the average serial interval was 4.8 days. For early <strong style="color: red">transmission</strong> in Wuhan, any infectious case produced as many as four new cases, <strong style="color: red">transmission</strong> outside Wuhan was less intense, with reproduction numbers below two. During the rapid growth phase of the outbreak the region of Wuhan city acted as a hot spot, generating new cases upon contact, while locally, in other provinces, <strong style="color: red">transmission</strong> was low. Interpretation COVID-19 is capable of spreading very rapidly. The sizes of outbreak in provinces of mainland China mainly depended on the numbers of cases imported from Wuhan as the local reproduction numbers were low. The COVID-19 epidemic should be controllable with appropriate interventions. Funding No specific funding.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| ytsqfqvh | Impact assessment of non-pharmaceutical interventions against COVID-19 and influenza in Hong Kong: an observational study                                                                       | Background: A range of public health measures have been implemented to delay and reduce local <strong style="color: red">transmission</strong> of COVID-19 in Hong Kong, and there have been major changes in behaviours of the general public. <strong>We examined the effect of these interventions and behavioral changes on the incidence of COVID-19 as well as on influenza virus infections which may share some aspects of <strong style="color: red">transmission</strong> dynamics with COVID-19.</strong> Methods: We reviewed policy interventions and measured changes in population behaviours through two telephone surveys, on January 20-23 and February 11-14. <strong>We analysed data on laboratory-confirmed COVID-19 cases, influenza surveillance data in outpatients of all ages, and influenza hospitalisations in children.</strong> We estimated the daily effective reproduction number (R_t), for COVID-19 and influenza A(H1N1). Findings: COVID-19 transmissibility has remained at or below 1, indicating successful containment to date. Influenza <strong style="color: red">transmission</strong> declined substantially after the implementation of social distancing measures and changes in population behaviours in late January, with a 44% (95% confidence interval, CI: 34% to 53%) reduction in transmissibility in the community, and a 33% (95% CI: 24% to 43%) reduction in transmissibility based on paediatric hospitalization rates. In the two surveys we estimated that 74.5% and 97.5% of the general adult population wore masks when going out, and 61.3% and 90.2% avoided going to crowded places, respectively. Implications: Containment measures, social distancing measures and changes in population behaviour have successfully prevented spread of COVID-19. The social distancing measures and behavioural changes led to a substantial reduction in influenza <strong style="color: red">transmission</strong> in early February 2020. However, it may be challenging to avoid fatigue and sustain these measures and population behaviours as COVID-19 continues to spread globally. Funding: Health and Medical Research Fund, Hong Kong                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| xfjexm5b | Impact of self-imposed prevention measures and short-term government intervention on mitigating and delaying a COVID-19 epidemic                                                                | Background: With new cases of COVID-19 surging around the world, some countries may have to prepare for moving beyond the containment phase. Prediction of the effectiveness of non-case-based interventions for mitigating, delaying or preventing the epidemic is urgent, especially for countries affected by the increased seasonal influenza activity. <strong>Methods: We developed a <strong style="color: red">transmission</strong> model to evaluate the impact of self-imposed prevention measures (handwashing, mask-wearing, and social distancing) due to COVID-19 awareness and of short-term government-imposed social distancing on the peak number of diagnoses, attack rate and time until the peak number of diagnoses.</strong> Findings: For fast awareness spread in the population, self-imposed measures can significantly reduce the attack rate, diminish and postpone the peak number of diagnoses. A large epidemic can be prevented if the efficacy of these measures exceeds 50%. For slow awareness spread, self-imposed measures reduce the peak number of diagnoses and attack rate but do not affect the timing of the peak. Early implementation of short-term government interventions can only delay the peak. Interpretation: Handwashing, mask-wearing and social distancing as a reaction to information dissemination about COVID-19 can be effective strategies to mitigate and delay the epidemic. We stress the importance of a rapid spread of awareness on the use of self-imposed prevention measures in the population. Early-initiated short-term government-imposed social distancing can buy time for healthcare systems to prepare for an increasing COVID-19 burden. Keywords: SARS-CoV-2, COVID-19, mathematical model, prevention measures, disease awareness, epidemic control, social distancing, handwashing, mask-wearing                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| eflwztji | Temporal dynamics in viral shedding and transmissibility of COVID-19                                                                                                                            | <strong>We report temporal patterns of viral shedding in 94 laboratory-confirmed COVID-19 patients and modelled COVID-19 infectiousness profile from a separate sample of 77 infector-infectee <strong style="color: red">transmission</strong> pairs.</strong> We observed the highest viral load in throat swabs at the time of symptom onset, and inferred that infectiousness peaked on or before symptom onset. We estimated that 44% of <strong style="color: red">transmission</strong> could occur before first symptoms of the index. Disease control measures should be adjusted to account for probable substantial pre-symptomatic <strong style="color: red">transmission</strong>.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| mrsjhjh4 | Estimating Preventable COVID19 Infections Related to Elective Outpatient Surgery in Washington State: A Quantitative Model                                                                      | Background: As the number of suspected and confirmed COVID19 cases in the US continues to rise, the US surgeon general, Centers for Disease Control and Prevention, and several specialty societies have issued recommendations to consider canceling elective surgeries. However, these recommendations have also faced controversy and opposition. <strong>Objective: The goal of this study is to provide a quantitative analysis and model for preventable COVID19 infections from elective outpatient or ambulatory surgery cases, which can also be adapted to analyze COVID19 <strong style="color: red">transmission</strong> in other healthcare settings.</strong> Furthermore, given the controversy over the appropriate handling of elective surgical cases during this pandemic, we hope that our results may have a positive impact on health policy and public health. <strong>Methods: Using previously published information on elective ambulatory or outpatient surgical procedures and publicly available data on COVID19 infections in the US and on the Diamond Princess cruise ship, we calculated a <strong style="color: red">transmission</strong> rate and generated a mathematical model to predict a lower bound for the number of healthcare-acquired COVID19 infections that could be prevented by canceling or postponing elective outpatient surgeries in Washington state.</strong> Results: Our model predicts that over the course of 30 days, at least 2445 preventable patient infections and at least 1557 preventable healthcare worker (HCW) infections would occur in WA state alone if elective outpatient procedures were to continue as usual. The majority of these infections are caused by <strong style="color: red">transmission</strong> from HCW who became infected at work. Conclusion: Given the large numbers of COVID19 infections that could be prevented by canceling elective outpatient surgeries, our findings support the recommendations of the US Surgeon General, CDC, American College of Surgeons (ACS), American Society of Anesthesiologists (ASA), and Anesthesia Patient Safety Foundation (APSF) to consider rescheduling or postponing elective surgeries until the COVID19 pandemic is under better control in the US.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 901m6zw0 | Spatial Visualization of Cluster-Specific COVID-19 Transmission Network in South Korea During the Early Epidemic Phase                                                                          | Coronavirus disease 2019 (COVID-19) has been rapidly spreading throughout China and other countries including South Korea. As of March 12, 2020, a total number of 7,869 cases and 66 deaths had been documented in South Korea. Although the first confirmed case in South Korea was identified on January 20, 2020, the number of confirmed cases showed a rapid growth on February 19, 2020 with a total number of 1,261 cases with 12 deaths based on the Korea Centers for Disease Control and Prevention (KCDC). <strong>Using the data of confirmed cases of COVID-19 in South Korea that are publicly available from the KCDC, this paper aims to create spatial visualizations of COVID-19 <strong style="color: red">transmission</strong> between January 20, 2020 and February 19, 2020.</strong> Compared to various epidemiological measures such as the basic reproduction number, spatial visualizations of the cluster-specific <strong style="color: red">transmission</strong> networks may be more useful to characterize super-spreaders and the spread of the virus especially in the early epidemic phase.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| ry9wpcxo | Global transmission network of SARS-CoV-2: from outbreak to pandemic                                                                                                                            | Background: The COVID-19 pandemic caused by the severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) is straining health systems around the world. Although the Chinese government implemented a number of severe restrictions on people's movement in an attempt to contain its local and international spread, the virus had already reached many areas of the world in part due to its potent transmissibility and the fact that a substantial fraction of infected individuals develop little or no symptoms at all. Following its emergence, the virus started to generate sustained <strong style="color: red">transmission</strong> in neighboring countries in Asia, Western Europe, Australia, Canada and the United States, and finally in South America and Africa. As the virus continues its global spread, a clear and evidence-based understanding of properties and dynamics of the global <strong style="color: red">transmission</strong> network of SARS-CoV-2 is essential to design and put in place efficient and globally coordinated interventions. Methods: We employ molecular surveillance data of SARS-CoV-2 epidemics for inference and comprehensive analysis of its global <strong style="color: red">transmission</strong> network before the pandemic declaration. <strong>Our goal was to characterize the spatial-temporal <strong style="color: red">transmission</strong> pathways that led to the establishment of the pandemic.</strong> We exploited a network-based approach specifically tailored to emerging outbreak settings. Specifically, it traces the accumulation of mutations in viral genomic variants via mutation trees, which are then used to infer <strong style="color: red">transmission</strong> networks, revealing an up-to-date picture of the spread of SARS-CoV-2 between and within countries and geographic regions. Results and Conclusions: The analysis suggest multiple introductions of SARS-CoV-2 into the majority of world regions by means of heterogeneous <strong style="color: red">transmission</strong> pathways. The <strong style="color: red">transmission</strong> network is scale-free, with a few genomic variants responsible for the majority of possible <strong style="color: red">transmission</strong>s. The network structure is in line with the available temporal information represented by sample collection times and suggest the expected sampling time difference of few days between potential <strong style="color: red">transmission</strong> pairs. The inferred network structural properties, <strong style="color: red">transmission</strong> clusters and pathways and virus introduction routes emphasize the extent of the global epidemiological linkage and demonstrate the importance of internationally coordinated public health measures.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| e4pr78n0 | Mathematical modeling of COVID-19 transmission and mitigation strategies in the population of Ontario, Canada                                                                                   | <strong>Background: We evaluated how non-pharmaceutical interventions could be used to control the COVID-19 pandemic and reduce the burden on the healthcare system.</strong> <strong>Methods: Using an age-structured compartmental model of COVID-19 <strong style="color: red">transmission</strong> in the population of Ontario, Canada, we compared a base case with limited testing, isolation, and quarantine to scenarios with: enhanced case finding; restrictive social distancing measures; or a combination of enhanced case finding and less restrictive social distancing.</strong> Interventions were either implemented for fixed durations or dynamically cycled on and off, based on projected ICU bed occupancy. We present median and credible intervals (CrI) from 100 replicates per scenario using a two-year time horizon. Results: We estimated that 56% (95% CrI: 42-63%) of the Ontario population would be infected over the course of the epidemic in the base case. At the epidemic peak, we projected 107,000 (95% CrI: 60,760-149,000) cases in hospital and 55,500 (95% CrI: 32,700-75,200) cases in ICU. For fixed duration scenarios, all interventions were projected to delay and reduce the height of the epidemic peak relative to the base case, with restrictive social distancing estimated to have the greatest effect. Longer duration interventions were more effective. Dynamic interventions were projected to reduce the proportion of the population infected at the end of the two-year period. Dynamic social distancing interventions could reduce the median number of cases in ICU below current estimates of Ontario's ICU capacity. Interpretation: Without significant social distancing or a combination of moderate social distancing with enhanced case finding, we project that ICU resources would be overwhelmed. Dynamic social distancing could maintain health system capacity and also allow periodic psychological and economic respite for populations.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 52zjm9jt | Forecasting the Worldwide Spread of COVID-19 based on Logistic Model and SEIR Model                                                                                                             | Background: As the outbreak of coronavirus disease 2019 (COVID-19), a sudden case increase in late February 2020 in global attracted deep concern. Italy, South Korea, Iran, France, Germany, Spain, the U.S. and Japan are probable the most severe countries. Collecting epidemiological data and predicting epidemic trends are important to develop and measure public intervention strategies. Epidemic predictions results yield by different mathematical models are out of line, therefore, we sought to compare different models and their prediction results, so as to generate objective conclusions. Methods: We used the number of cases reported from January 23 to March 20, 2020 to estimate possible spread size and peak time of COVID-19, especially in 8 high risk countries. Logistic growth model, basic SEIR model and adjusted SEIR model were adopted for predicting. Considering different model inputs may infer different model outputs, we implemented three model predictions with three scenarios of epidemic development. Results: When contrasting all 8 countries short-term prediction results and peak predictions, the difference between the models was relatively large. The logistic growth model estimated a smaller epidemic size than the basic SERI model, however, once we added parameters which considered the effects of public health interventions and control measures, the adjusted SERI model results demonstrated a considerably rapid decelerate of the epidemic development. Our results demonstrated contact rate, quarantine scale, quarantine initiate time and length are important factors to control the epidemic size and length. Conclusions: We demonstrated a comparative assessment of the predictions of COVID-19 outbreak of 8 high risk countries using multiple methods. <strong>By forecasting epidemic size and peak time as well as simulating the effects of public health interventions, the intent of this paper is to help understand the <strong style="color: red">transmission</strong> dynamics of COVID-19 and recommend operation suggestions to slow down the epidemic.</strong> It is suggested that quickly detecting cases, enough quarantine implementation and public self-protection behaviors are critical to slow down the epidemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| cbc98t7x | A Genomic Survey of SARS-CoV-2 Reveals Multiple Introductions into Northern California without a Predominant Lineage                                                                            | The COVID-19 pandemic caused by the novel coronavirus SARS-CoV-2 has spread globally, resulting in >300,000 reported cases worldwide as of March 21st, 2020. <strong>Here we investigate the genetic diversity and genomic epidemiology of SARS-CoV-2 in Northern California using samples from returning travelers, cruise ship passengers, and cases of community <strong style="color: red">transmission</strong> with unclear infection sources.</strong> Virus genomes were sampled from 29 patients diagnosed with COVID-19 infection from Feb 3rd through Mar 15th. Phylogenetic analyses revealed at least 8 different SARS-CoV-2 lineages, suggesting multiple independent introductions of the virus into the state. Virus genomes from passengers on two consecutive excursions of the Grand Princess cruise ship clustered with those from an established epidemic in Washington State, including the WA1 genome representing the first reported case in the United States on January 19th. We also detected evidence for presumptive <strong style="color: red">transmission</strong> of SARS-CoV-2 lineages from one community to another. These findings suggest that cryptic <strong style="color: red">transmission</strong> of SARS-CoV-2 in Northern California to date is characterized by multiple <strong style="color: red">transmission</strong> chains that originate via distinct introductions from international and interstate travel, rather than widespread community <strong style="color: red">transmission</strong> of a single predominant lineage. Rapid testing and contact tracing, social distancing, and travel restrictions are measures that will help to slow SARS-CoV-2 spread in California and other regions of the USA.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| xdjgjeb9 | A mathematical model of COVID-19 transmission between frontliners and the general public                                                                                                        | The number of COVID-19 cases is continuously increasing in different countries (as of March 2020) including the Philippines. It is estimated that the basic reproductive number of COVID-19 is around 1.5 to 4. The basic reproductive number characterizes the average number of persons that a primary case can directly infect in a population full of susceptible individuals. However, there can be superspreaders that can infect more than this estimated basic reproductive number. <strong>In this study, we formulate a conceptual mathematical model on the <strong style="color: red">transmission</strong> dynamics of COVID-19 between the frontliners and the general public.</strong> We assume that the general public has a reproductive number between 1.5 to 4, and frontliners (e.g. healthcare workers, customer service and retail personnel, food service crews, and transport or delivery workers) have a higher reproduction number. Our simulations show that both the frontliners and the general public should be protected or resilient against the disease. Protecting only the frontliners will not result in flattening the epidemic curve. Protecting only the general public may flatten the epidemic curve but the infection risk faced by the frontliners is still high, which may eventually affect their work. Our simple model does not consider all factors involved in COVID-19 <strong style="color: red">transmission</strong> in a community, but the insights from our model results remind us of the importance of community effort in controlling the <strong style="color: red">transmission</strong> of the disease. All in all, the take-home message is that everyone in the community, whether a frontliner or not, should be protected or should implement preventive measures to avoid being infected.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 6njag0dq | COVID-19: Projecting the impact in Rohingya refugee camps and beyond                                                                                                                            | Background: COVID-19 could have even more dire consequences among refugees living in camps than in general populations. Bangladesh has confirmed COVID-19 cases and hosts almost 1 million Rohingya refugees from Myanmar with 600,000 concentrated in Kutupalong-Balukhali Expansion Site. Projections of the potential COVID-19 infection burden, epidemic speed, and healthcare needs in such settings are critical to inform preparedness planning. <strong>Methods: To explore the potential impact of the introduction of SARS-CoV-2 in Kutupalong-Balukhali Expansion Site, we used a stochastic disease <strong style="color: red">transmission</strong> model with parameters derived from emerging literature.</strong> We considered three scenarios with different assumptions about the <strong style="color: red">transmission</strong> potential of SARS-CoV-2. We estimated the number and daily rate of infections, hospitalizations, deaths, and healthcare needs expected under each scenario. Findings: A large-scale outbreak is highly likely after a single introduction of the virus into the camp across scenarios with 65-95% of simulations leading to at least 1,000 infections. Estimated infections range from 119-504 between low and high <strong style="color: red">transmission</strong> scenarios in the first 30 days; and between 424,798 and 543,647 in 12 months. Hospitalization needs exceeded the existing hospitalization capacity of 340 beds after 58-139 days between the low and high <strong style="color: red">transmission</strong> scenarios. Estimated deaths range from 1,647-2,109 between low and high <strong style="color: red">transmission</strong> scenarios. Interpretation: A COVID-19 epidemic in a refugee settlement may have profound consequences, requiring unrealistically large increases in healthcare capacity and infrastructure. Detailed and realistic planning for the worst in Kutupalong-Balukhali and all refugee camps worldwide must begin now. Plans should consider novel and radical strategies to reduce infectious contacts and fill health worker gaps while recognizing that refugees may not have access to national health systems. As global resources to fight this pandemic become scarce, displaced persons must not be forgotten.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| f936bioj | A phased lift of control: a practical strategy to achieve herd immunity against Covid-19 at the country level                                                                                   | Most countries are affected by the Covid-19 pandemic and experience rapidly increasing numbers of cases and deaths. Many have implemented nationwide stringent control to avoid overburdening the health care system. This paralyzes economic and social activities until the availability of a vaccine, which may take years. We propose an alternative exit strategy to develop herd immunity in a predictable and controllable way: a phased lift of control. This means that successive parts of the country (e.g. provinces) stop stringent control, and Covid-19-related IC admissions are distributed over the country as the whole. Importantly, vulnerable individuals need to be shielded until herd immunity has developed in their area. <strong>We explore the characteristics and duration of this strategy using a novel individual-based model for geographically stratified <strong style="color: red">transmission</strong> of Covid-19 in a country.</strong> The model predicts that individuals will have to experience stringent control for about 14 months on average, but this duration may be significantly shortened by future developments (more IC beds, better drugs). Clearly, the strategy will have a profound impact on individuals and society, and should therefore be considered carefully by various other disciplines (e.g. health systems, ethics, economics) before actual implementation.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| 3ueg2i6w | Transmission Dynamics of COVID-19 and Impact on Public Health Policy                                                                                                                            | <strong>In this work we construct a mathematical model for the <strong style="color: red">transmission</strong> and spread of coronavirus disease 2019 or COVID-19.</strong> Our model features delay terms to account for (a) the time lapse or latency period between contracting the disease and displaying symptoms, and (b) the time lag in testing patients for the virus due to the limited numbers of testing facilities currently available. We find that the delay introduces a significant disparity between the actual and reported time-trajectories of cases in a particular region. Specifically, the reported case histories lag the actual histories by a few days. Hence, to minimize the spread of the disease, lockdowns and similarly drastic social isolation measures need to be imposed some time before the reported figures are approaching their peak values. We then account for the social reality that lockdowns can only be of a limited duration in view of practical considerations. We find that the most effective interval for imposing such a limited-time lockdown is one where the midpoint of the lockdown period coincides with the actual peak of the spread of the disease in the absence of the lockdown. We further show that the true effectivity of imposing a lockdown may be misrepresented and grossly underestimated by the reported case trajectories in the days following the action.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| drj3al9t | Intervention Serology and Interaction Substitution: Modeling the Role of 'Shield Immunity' in Reducing COVID-19 Epidemic Spread                                                                 | The COVID-19 pandemic has precipitated a global crisis, with more than 690,000 confirmed cases and more than 33,000 confirmed deaths globally as of March 30, 2020 [1-4]. At present two central public health control strategies have emerged: mitigation and suppression (e.g, [5]). Both strategies focus on reducing new infections by reducing interactions (and both raise questions of sustainability and long-term tactics). Complementary to those approaches, here we develop and analyze an epidemiological intervention model that leverages serological tests [6, 7] to identify and deploy recovered individuals as focal points for sustaining safer interactions via interaction substitution, i.e., to develop what we term 'shield immunity' at the population scale. Recovered individuals, in the present context, represent those who have developed protective, antibodies to SARS-CoV-2 and are no longer shedding virus [8]. <strong>The objective of a shield immunity strategy is to help sustain the interactions necessary for the functioning of essential goods and services (including but not limited to tending to the elderly [9], hospital care, schools, and food supply) while decreasing the probability of <strong style="color: red">transmission</strong> during such essential interactions.</strong> We show that a shield immunity approach may significantly reduce the length and reduce the overall burden of an outbreak, and can work synergistically with social distancing. The present model highlights the value of serological testing as part of intervention strategies, in addition to its well recognized roles in estimating prevalence [10, 11] and in the potential development of plasma-based therapies [12-15].                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| r1a2zwj4 | Sustainable social distancing through facemask use and testing during the Covid-19 pandemic                                                                                                     | <strong>We investigate how individual protective behaviors, different levels of testing, and isolation influence the <strong style="color: red">transmission</strong> and control of the COVID-19 pandemic.</strong> Based on an SEIR-type model incorporating asymptomatic but infectious individuals (40%), we show that the pandemic may be readily controllable through a combination of testing, treatment if necessary, and self-isolation after testing positive (TTI) of symptomatic individuals together with social protection (e.g., facemask use, handwashing). When the basic reproduction number, R0, is 2.4, 65% effective social protection alone (35% of the unprotected <strong style="color: red">transmission</strong>) brings the R below 1. Alternatively, 20% effective social protection brings the reproduction number below 1.0 so long as 75% of the symptomatic population is covered by TTI within 12 hours of symptom onset. Even with 20% effective social protection, TTI of 1 in 4 symptomatic individuals can substantially 'flatten the curve' cutting the peak daily incidence in half.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| qhbzs9tb | Interaction between malarial transmission and BCG vaccination with COVID-19 incidence in the world map: A changing landscape human immune system?                                               | Background: COVID-19 (Corona virus Disease-2019) is a new public health emergency and is a pandemic currently. Incidence and mortality of COVID-19 vary in different geographical areas. <strong>In this study we aimed to analyse the relationship between malaria <strong style="color: red">transmission</strong> and BCG vaccination with COVID-19 incidence in the world map.</strong> Materials and methods: We collected malaria cases data (World Health Organisation (WHO), 2018), worldwide COVID-19 cases and mortality data (European Centre for Disease Prevention and Control) and data on BCG vaccination. COVID-19 incidence and mortality was compared. Findings: Data on 5316978938 persons from 166 countries were analysed. Malaria incidence rate was negatively correlated with COVID-19 incidence rate (correlation coefficient = -0.513, p<0.001). Malaria free countries had significantly higher number of COVID-19 cases compared to malaria endemic countries. In Europe and Americas, countries, which have higher BCG vaccination coverage, had significantly less mortality per thousand population compared to those with low BCG coverage (median 0.0002 (0-0.0005) vs 0.0029 (0.0002-0.0177), p=0.017). The case fatality ratio of COVID-19 was related nonlinearly to the malaria incidence. Conclusions: The results suggest the changing human immune system as we progress to eliminate parasitic diseases with time. Chloroquine exposure in malaria endemic zones might have a protective effect.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| vytmwmoj | Control Strategies to Curtail Transmission of COVID-19                                                                                                                                          | Recently, the World Health Organization has declared the outbreak of a severe acute respiratory syndrome coronavirius as a pandemic, and declared it as Public Health Emergency of International Concern. More than 6,83,536 positive cases and 32,139 deaths caused by coronavirus 2019 affecting 199 countries and territories. This pandemic can transform into an extremely destructive form if we still do not take it seriously. <strong>In this present study, we propose a generalized SEIR model of COVID-2019 to study the behavior of its <strong style="color: red">transmission</strong> under different control strategies.</strong>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 9bi6pobg | The impact of early social distancing at COVID-19 Outbreak in the largest Metropolitan Area of Brazil.                                                                                          | <strong>We evaluated the impact of early social distancing on the COVID-19 <strong style="color: red">transmission</strong> in the Sao Paulo metropolitan area.</strong> Using an age-stratified SEIR model, we determined the time-dependent reproductive number, and forecasted the ICU beds necessary to tackle this epidemic. Within 60 days, these measures might prevent 89,450 deaths.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| vfr4pvmz | INFEKTA: A General Agent-based Model for Transmission of Infectious Diseases: Studying the COVID-19 Propagation in Bogota - Colombia                                                            | The <strong style="color: red">transmission</strong> dynamics of the coronavirus - COVID-19- have challenged humankind at almost every level. Currently, research groups around the globe are trying to figure out such <strong style="color: red">transmission</strong> dynamics using different scientific and technological approaches. One of those is by using mathematical and computational models like the compartmental model or the agent-based models. <strong>In this paper, a general agent-based model, called INFEKTA, that combines the <strong style="color: red">transmission</strong> dynamics of an infectious disease with agents (individuals) that can move on a complex network of accessible places defined over a Euclidean space representing a real town or city is proposed.</strong> The applicability of INFEKTA is shown by modeling the <strong style="color: red">transmission</strong> dynamics of the COVID-19 in Bogotá city, the capital of Colombia.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 7styuuzo | Characterizing key attributes of the epidemiology of COVID-19 in China: Model-based estimations                                                                                                 | Background: A novel coronavirus strain, severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), emerged in China in late 2019. The resulting disease, Coronavirus Disease 2019 (COVID-2019), soon became a pandemic. <strong>This study aims to characterize key attributes of the epidemiology of this infection in China.</strong> <strong>Methods: An age-stratified mathematical model was constructed to describe the <strong style="color: red">transmission</strong> dynamics and estimate the age-specific differences in the biological susceptibility to the infection, age-assortativeness in <strong style="color: red">transmission</strong> mixing, case fatality rate (CFR), and transition in rate of infectious contacts (and reproduction number R0) following introduction of mass interventions.</strong> Results: The model estimated the infectious contact rate in early epidemic at 0.59 contacts per day (95% uncertainty interval (UI)=0.48-0.71). Relative to those 60-69 years of age, susceptibility to the infection was only 0.06 in those ≤19 years, 0.34 in 20-29 years, 0.57 in 30-39 years, 0.69 in 40-49 years, 0.79 in 50-59 years, 0.94 in 70-79 years, and 0.88 in ≥80 years. The assortativeness in <strong style="color: red">transmission</strong> mixing by age was very limited at 0.004 (95% UI=0.002-0.008). Final CFR was 5.1% (95% UI=4.8-5.4%). R0 rapidly declined from 2.1 (95% UI=1.8-2.4) to 0.06 (95% UI=0.05-0.07) following onset of interventions. Conclusion: Age appears to be a principal factor in explaining the patterns of COVID-19 <strong style="color: red">transmission</strong> dynamics in China. The biological susceptibility to the infection seems limited among children, intermediate among young to mid-age adults, but high among those >50 years of age. There was no evidence for differential contact mixing by age, consistent with most <strong style="color: red">transmission</strong> occurring in households rather than in schools or workplaces.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| b8a78ym6 | Estimating cost-benefit of quarantine length for Covid-19 mitigation                                                                                                                            | Background: The world has been put in an unprecedented situation by the Covid-19 pandemic. Creating models to describe and quantify alternative mitigation strategies becomes increasingly urgent. <strong>Methods: In this study, we propose an agent-based model of disease <strong style="color: red">transmission</strong> in a society divided into closely connected families, workplaces, and social groups.</strong> This allows us to discuss mitigation strategies, including targeted quarantine measures. Results: We find that workplace and more diffuse social contacts are roughly equally important to disease spread, and that an effective lockdown must target both. We examine the cost-benefit of replacing a lockdown with tracing and quarantine contacts of the infected. When reopening society, testing and quarantining is a strategy that is much cheaper in terms of lost workdays than a long lockdown of workplaces. Conclusions: A targeted quarantine strategy is quite efficient with only 5 days of quarantine, and its relative effect increases when supplemented with other measures that reduce disease <strong style="color: red">transmission</strong>.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| be7hez61 | Pregnancy outcomes, Newborn complications and Maternal-Fetal Transmission of SARS-CoV-2 in women with COVID-19: A systematic review                                                             | <strong>Abstract Objective: The aim of this systematic review was to examine published and preprint reports for maternal and fetal outcomes in pregnant women with COVID-19 and also assess the incidence of maternal-fetal <strong style="color: red">transmission</strong> of SARS CO-V-2 infection.</strong> <strong>Design : Systematic review Data sources:We searched PUMBED.</strong> Medline, Embase, MedRxiv and bioRxiv databases upto 31st March 2020 utilizing combinations of word variants for &#34 coronavirus &#34 or &#34 COVID-19 &#34 or &#34 severe acute respiratory syndrome &#34 or &#34 SARS-COV-2 &#34 and &#34 pregnancy &#34 . We also included data from preprint articles. <strong>Study selection : Original case reports and case series on pregnant women with a confirmed diagnosis of SARS-CoV-2 infection.</strong> Data extraction : We included 23 studies [China (20), USA (01), Republic of Korea (01) and Honduras, Central America (01) reporting the information on 172 pregnant women and 162 neonates. The primary outcome measures were maternal health characteristics and adverse pregnancy outcomes, neonatal outcomes and SARS-CoV-2 infection in neonates was extracted. Treatments given to pregnant women with COVID-19 were also recorded. Results: Out of 172 women affected by COVID-19 in pregnancy, 160 women had delivered 162 newborns (2 set of twins, 12 ongoing pregnancies). In pregnant women with COVID-19, the most common symptoms were fever (54%), cough (35%), myalgia (17%), dyspnea (12%) and diarrhea (4%). Pneumonia was diagnosed by CT scan imaging in 100 % of COVID-19 pregnant women. Pregnancy complications included delivery by cesarean section (89%), preterm labor (21%), fetal distress (9%) and premature rupture of membranes (8%). The most common co-morbidities associated with pregnant women with COVID-19 were diabetes (11%), hypertensive disorders (9%), placental disorders (5%), co-infections (6%), scarred uterus (5%), hypothyroidism (5%) and anemia (4%). Amongst the neonates of COVID-19 mothers, preterm birth (23%), respiratory distress syndrome (14%), pneumonia (14%) low birth weight (11%), small for gestational age (3%) were reported. There was one still birth and one neonatal death reported. Vertical <strong style="color: red">transmission</strong> rate of SARS-CoV-2 is estimated to be 11%. Conclusion In pregnant women with COVID-19, diabetes and hypertensive disorders are common co-morbidities and there is a risk of preterm delivery. Amongst the neonates born to mothers with COVID-19, respiratory distress syndrome and pneumonia are common occurrence. There is an evidence of vertical <strong style="color: red">transmission</strong> of SARS-CoV-2 infection in women with COVID-19.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| 6xgiep8q | Breastfeeding of Infants Born to Mothers with COVID-19: A Rapid Review                                                                                                                          | Abstract Background: Existing recommendations on whether mothers with COVID-19 should continue breastfeeding are still conflicting. <strong>We aimed to conduct a rapid review of mother-to-child <strong style="color: red">transmission</strong> of COVID-19 during breastfeeding.</strong> Methods: We systematically searched Medline, Embase, Web of Science, Cochrane library, China Biology Medicine disc, China National Knowledge Infrastructure, Wanfang, and preprint articles up to March 2020. We included studies relevant to <strong style="color: red">transmission</strong> through milk and respiratory droplets during breastfeeding of mothers with COVID-19, SARS, MERS and influenza. Two reviewers independently screened studies for eligibility, extracted data, assessed risk of bias and used GRADE to assess certainty of evidence. Results: A total of 4481 records were identified in our literature search. Six studies (five case reports and one case series) involving 58 mothers (16 mothers with COVID-19, 42 mothers with influenza) and their infants proved eligible. Five case reports showed that the viral nucleic acid tests for all thirteen collected samples of breast milk from mothers with COVID-19 were negative. A case series of 42 influenza infected postpartum mothers taking precautions (hand hygiene and wearing masks) before breastfeeding showed that no neonates were infected with influenza during one-month of follow-up. Conclusions: The current evidence indicates that SARS-CoV-2 viral nucleic acid has not been detected in breast milk. The benefits of breastfeeding may outweigh the risk of SARS-CoV-2 infection in infants. Mothers with COVID-19 should take appropriate precautions to reduce the risk of <strong style="color: red">transmission</strong> via droplets and close contact during breastfeeding. Keywords: Breastfeeding; COVID-19; infant; mother-to-child <strong style="color: red">transmission</strong>; rapid Review.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| xh29euj4 | Refined compartmental models, asymptomatic carriers and COVID-19                                                                                                                                | <strong>The goal of this article is to analyze some compartmental models specially designed to model the spread of a disease whose <strong style="color: red">transmission</strong> has the same features as COVID-19.</strong> <strong>The major contributions of this article are: (1) Rigorously find sufficient conditions for the outbreak to only have one peak, i.e. for no second wave of infection to form; (2) Investigate the formation of other waves of infection when such conditions are not met; (3) Use numerical simulations to analyze the different roles asymptomatic carriers can have.</strong> We also argue that dividing the population into non-interacting groups leads to an effective reduction of the <strong style="color: red">transmission</strong> rates. As in any compartmental model, the goal of this article is to provide qualitative understanding rather than exact quantitative predictions.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| 6kiv3qi5 | The effect of inter-city travel restrictions on geographical spread of COVID-19: Evidence from Wuhan, China                                                                                     | Background: To contain the spread of COVID-19, a cordon sanitaire was put in place in Wuhan prior to the Lunar New Year, on 23 January 2020, restricting travel to other parts of China. <strong>We assess the efficacy of the cordon sanitaire to delay the introduction and onset of local <strong style="color: red">transmission</strong> of COVID-19 in other major cities in mainland China.</strong> Methods: We estimated the number of infected travellers from Wuhan to other major cities in mainland China from November 2019 to March 2020 using previously estimated COVID-19 prevalence in Wuhan and publicly available mobility data. We focused on Beijing, Chongqing, Hangzhou, and Shenzhen as four representative major cities to identify the potential independent contribution of the cordon sanitaire and holiday travel. To do this, we simulated outbreaks generated by infected arrivals in these destination cities using stochastic branching processes. We also modelled the effect of the cordon sanitaire in combination with reduced transmissibility scenarios representing the effect of local non-pharmaceutical interventions. Findings: In the four cities, given the potentially high prevalence of COVID-19 in Wuhan between Dec 2019 and early Jan 2020, local <strong style="color: red">transmission</strong> may have been seeded as early as 2 - 8 January 2020. By the time the cordon sanitaire was imposed, simulated case counts were likely in the hundreds. The cordon sanitaire alone did not substantially affect the epidemic progression in these cities, although it may have had some effect in smaller cities. Interpretation: Our results indicate that the cordon sanitaire may not have prevented COVID-19 spread in major Chinese cities; local non-pharmaceutical interventions were likely more important for this.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| bbghqy1a | Evidence for probable aerosol transmission of SARS-CoV-2 in a poorly ventilated restaurant                                                                                                      | Background: The role of aerosols in the <strong style="color: red">transmission</strong> of SARS-CoV-2 remains debated. <strong>We analysed an outbreak involving three non-associated families in Restaurant X in Guangzhou, China, and assessed the possibility of aerosol <strong style="color: red">transmission</strong> of SARS-CoV-2 and characterize the associated environmental conditions.</strong> Methods: We collected epidemiological data, obtained a video record and a patron seating-arrangement from the restaurant, and measured the dispersion of a warm tracer gas as a surrogate for exhaled droplets from the suspected index patient. Computer simulations were performed to simulate the spread of fine exhaled droplets. We compared the in-room location of subsequently infected cases and spread of the simulated virus-laden aerosol tracer. The ventilation rate was measured using the tracer decay method. Results: Three families (A, B, C), 10 members of which were subsequently found to have been infected with SARS-CoV-2 at this time, or previously, ate lunch at Restaurant X on Chinese New Year's Eve (January 24, 2020) at three neighboring tables. Subsequently, three members of family B and two members of family C became infected with SARS-CoV-2, whereas none of the waiters or 68 patrons at the remaining 15 tables became infected. During this occasion, the ventilation rate was 0.75-1.04 L/s per person. No close contact or fomite contact was observed, aside from back-to-back sitting by some patrons. Our results show that the infection distribution is consistent with a spread pattern representative of exhaled virus-laden aerosols. Conclusions: Aerosol <strong style="color: red">transmission</strong> of SARS-CoV-2 due to poor ventilation may explain the community spread of COVID-19.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| we9kzreh | Persistent viral shedding of SARS-CoV-2 in faeces - a rapid review                                                                                                                              | <strong>Background and aims In addition to respiratory symptoms, patients with COVID-19 can present with gastrointestinal complaints suggesting a possible faeco-oral <strong style="color: red">transmission</strong> route.</strong> <strong>The primary aim of this review is to establish the incidence and timing of positive faecal samples for the SARS-CoV-2 virus in patients with COVID-19.</strong> Methods A systematic literature review was performed to identify studies describing COVID-19 patients tested for the virus in their stool. Data were extracted concerning the nature of the test, number and timing of positive samples, incidence of positive faecal tests after negative nasopharyngeal swabs and any evidence of viable faecal virus or faeco-oral <strong style="color: red">transmission</strong> of the virus. Results There were 26 relevant articles identified. Combining these results demonstrated that 53.9% of those tested for faecal RNA in these studies were positive. Duration of faecal viral shedding ranged from 1 to 33 days after nasopharyngeal swab turned negative with one result remaining positive after 47 days of onset of symptoms. There was insufficient evidence to draw firm conclusions about the proportion of cases potentially transmitted through infection via faecally shed virus. Conclusions There is a relatively high rate of positive tests and persistence of the SARS-CoV-2 virus in faecal samples of selected patients with COVID-19. Further research is needed to demonstrate how much these positive tests correlate with viable virus and <strong style="color: red">transmission</strong> through the faeco-oral route. This may have important implications for duration of isolation, precautions recommended in individuals undertaking a period of isolation, protective equipment for health professionals and interventional procedures involving the gastrointestinal tract.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| hywf0f49 | Exit strategies: optimising feasible surveillance for detection, elimination and ongoing prevention of COVID-19 community transmission                                                          | Background Following successful implementation of strong containment measures by the community, Australia is now close to the point of eliminating detectable community <strong style="color: red">transmission</strong> of COVID-19. <strong>We aimed to develop an efficient, rapid and scalable surveillance strategy for detecting all remaining COVID-19 community <strong style="color: red">transmission</strong> through exhaustive identification of every active <strong style="color: red">transmission</strong> chain.</strong> We also identified measures to enable early detection and effective management of any reintroduction of <strong style="color: red">transmission</strong> once containment measures are lifted to ensure strong containment measures do not need to be reinstated. <strong>Methods We compared efficiency and sensitivity to detect community <strong style="color: red">transmission</strong> chains through testing of: hospital cases; primary care fever and cough patients; or asymptomatic community members, using surveillance evaluation methods and mathematical modelling, varying testing capacities and prevalence of COVID-19 and non-COVID-19 fever and cough, and the reproduction number.</strong> System requirements for increasing testing to allow exhaustive identification of all <strong style="color: red">transmission</strong> chains, and then enable complete follow-up of all cases and contacts within each chain, were assessed per million population. Findings Assuming 20% of cases are asymptomatic and all symptomatic COVID-19 cases present to primary care, with high <strong style="color: red">transmission</strong> (R=2.2) there are a median of 13 unrecognised community cases (5 infectious) when a <strong style="color: red">transmission</strong> chain is identified through hospital surveillance versus 3 unrecognised cases (1 infectious) through primary care surveillance. 3 unrecognised community upstream community cases themselves are estimated to generate a further 22-33 contacts requiring follow-up. The unrecognised community cases rise to 5 if only 50% of symptomatic cases present to primary care. Screening for asymptomatic disease in the community cannot exhaustively identify all <strong style="color: red">transmission</strong> under any of the scenarios assessed. The additional capacity required to screen all fever and cough primary care patients would be approximately 2,000 tests/million population per week using 1/16 pooling of samples. Interpretation Screening all syndromic fever and cough primary care presentations, in combination with exhaustive and meticulous case and contact identification and management, enables appropriate early detection and elimination of community <strong style="color: red">transmission</strong> of COVID-19. If testing capacity is limited, interventions such as pooling allow increased case detection, even given reduced test sensitivity. Wider identification and testing of all upstream contacts, (i.e. potential sources of infection for identified cases, and their related <strong style="color: red">transmission</strong> chains) is critical, and to be done exhaustively requires more resources than downstream contact tracing. The most important factor in determining the performance of such a surveillance system is community participation in screening and follow up, and as such, appropriate community engagement, messaging and support to encourage presentation and compliance is essential. We provide operational guidance on implementing such a system.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 82o75tet | Analysis of Effectiveness of Quarantine Measures in Controlling COVID-19                                                                                                                        | COVID-19 has created an interesting discourse among the people of the world particularly regarding preventive measures of infectious diseases. <strong>In this paper, the authors forecast the spread of the Corona virus outbreak and study how the reduction of <strong style="color: red">transmission</strong> rates influences its decline.</strong> The paper makes use of the SIR (Susceptible Infected Recovered) Model which is a deterministic model used in the field of epidemiology-based on differential equations derived from sections of the population. The Basic Reproduction Number (Ro) represents the criticality of the epidemic in numeric terms. Forecasting an epidemic provides insights about the geographic spreading of the disease and the case incidences required to better inform intervention strategists about situations that may occur during the outbreak. Through this research paper, the authors wish to provide an insight into the impact of control measures on the pandemic. By drawing a comparison of three countries and their quarantine measures, observations on the decline of the outbreak are made. Authors intend to guide the intervention strategies of under-resourced countries like India and aid in the overall containment of the outbreak.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| q5xc4m3j | Modelling the potential impact of social distancing on the COVID-19 epidemic in South Africa                                                                                                    | The novel coronavirus (COVID-19) pandemic continues to be a global health problem whose impact has been significantly felt in South Africa. Social distancing has been touted as the best form of response in managing a rapid increase in the number of infected cases. <strong>In this paper, we present a deterministic model to model the impact of social distancing on the <strong style="color: red">transmission</strong> dynamics of COVID-19 in South Africa.</strong> The model is fitted to the currently available data on the cumulative number of infected cases and a scenario analysis on different levels of social distancing are presented. The results show a continued rise in the number of cases in the lock down period with the current levels of social distancing albeit at a lower rate. The model shows that the number of cases will rise to above 4000 cases by the end of the lockdown. The model also looks at the impact of relaxing the social distancing measures after the initial announcement of the lock down measures. A relaxation of the social distancing by 2% can result in a 23% rise in the number of cumulative cases while on the other hand increasing the levels of social distancing by 2% would reduce the number of cumulative cases by about 18%. These results have implications on the management and policy direction in the early phases of the epidemic.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| e0wzkhbz | Taking Account of Asymptomatic Infections in Modeling the Transmission Potential of the COVID-19 Outbreak on the Diamond Princess Cruise Ship                                                   | <strong>We take the case of the Diamond Princess (DP) cruise ship as an experimental model for studying the <strong style="color: red">transmission</strong> potential of COVID-19 in a closed environment.</strong> We investigate the changes in R0 for COVID-19 on the DP from January 21 to February 19, 2020 with a chain-binomial model at different times under two scenarios: no quarantine assuming a random mixing condition, and quarantine of passengers in cabins -- passengers may get infected either by an infectious case in a shared cabin or by asymptomatic crew who continued to work. Our modeling approach takes account of the asymptomatic ratio of crew members during the quarantine of passengers, which has not been explored in the literature. Assuming an asymptomatic ratio 0.505 and the proportion of infections in cabins 0.2, R0 at the beginning of the epidemic was 3.27 (95% CI (3.02, 3.54)) and 3.78 (95% CI (3.49, 4.09)) respectively for serial intervals of 5 and 6 days, and increased for passengers in contact with asymptomatic crew during quarantine. We find evidence to support a CDC report that "a high proportion of asymptomatic infections could partially explain the high attack rate among cruise ship passengers and crew. " This emphasizes the importance of improved reporting and quarantine of asymptomatic cases, and raises questions on quarantine procedures in closed environments such as military vessels, cruise ships, dormitories, prisons, and other enclosed living complexes with high population densities.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 8tyrnajb | Modeling the Effective Control Strategy for Transmission Dynamics of Global Pandemic COVID-19                                                                                                   | The novel coronavirus disease (namely COVID-19) has taken attention because of its deadliness across the globe, causing a massive death as well as critical situation around the world. It is an infectious disease which is caused by newly discovered coronavirus. Our study demonstrates with a nonlinear model of this devastating COVID-19 which narrates <strong style="color: red">transmission</strong> from human-to-human in the society. Pontryagin Maximum principle has also been applied in order to obtain optimal control strategies where the maintenance of social distancing is the major control. <strong>The target of this study is to find out the most fruitful control measures of averting coronavirus infection and eventually, curtailed of the COVID-19 <strong style="color: red">transmission</strong> among people.</strong> The model is investigated analytically by using most familiar necessary conditions of Pontryagin maximum principle. Furthermore, numerical simulations have been performed to illustrate the analytical results. The analysis reveals that implementation of educational campaign, social distancing and developing human immune system are the major factors which can be able to plunge the scenario of becoming infected.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| djuomhww | Risks of Novel Coronavirus Disease (COVID-19) in Pregnancy; a Narrative Review                                                                                                                  | INTRODUCTION: The outbreak of the new Coronavirus in China in December 2019 and subsequently in various countries around the world has raised concerns about the possibility of vertical <strong style="color: red">transmission</strong> of the virus from mother to fetus. <strong>The present study aimed to review published literature in this regard.</strong> <strong>METHODS: In this narrative review, were searched for all articles published in various databases including PubMed, Scopus, Embase, Science Direct, and Web of Science using MeSH-compliant keywords including COVID-19, Pregnancy, Vertical <strong style="color: red">transmission</strong>, Coronavirus 2019, SARS-CoV-2 and 2019-nCoV from December 2019 to March 18, 2020 and reviewed them.</strong> All type of articles published about COVID-19 and vertical <strong style="color: red">transmission</strong> in pregnancy were included. RESULTS: A review of 13 final articles published in this area revealed that COVID-19 can cause fetal distress, miscarriage, respiratory distress and preterm delivery in pregnant women but does not infect newborns. There has been no report of vertical <strong style="color: red">transmission</strong> in pregnancy, and it has been found that clinical symptoms of COVID-19 in pregnant women are not different from those of non-pregnant women. CONCLUSION: Overall, due to lack of appropriate data about the effect of COVID-19 on pregnancy, it is necessary to monitor suspected pregnant women before and after delivery. For confirmed cases both the mother and the newborn child should be followed up comprehensively.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| 7gl5lptp | Epidemiological analysis on a family cluster of COVID-19                                                                                                                                        | <strong>Objective To understand the possible <strong style="color: red">transmission</strong> route of a family cluster of COVID-19 in Zhengzhou and the potential infectivity of COVID-19 in incubation period, and provide scientific evidence for the timely control of infectious source and curb the spread of the epidemic.</strong> Methods Epidemiological investigation was conducted for a family cluster of COVID-19 (8 cases) with descriptive epidemiological method, and respiratory tract samples of the cases were collected for the nucleic acid detection of 2019-nCoV by RT-PCR. Results Two primary cases, which occurred on 31 January and 1 February, 2020, respectively, had a common exposure history in Wuhan. The other six family members had onsets on 30 January, 31 January, 1 February (three cases) and 3 February, 2020. Conclusions In this family cluster of COVID-19, six family members were infected through common family exposure to the 2 primary cases. Five secondary cases had onsets earlier than or on the same day as the primary cases, indicating that COVID-19 is contagious in incubation period, and the home isolation in the early phase of the epidemic might lead to the risk of family cluster of COVID-19.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| lfpco9hp | The Outbreak Evaluation of COVID-19 in Wuhan District of China                                                                                                                                  | There were 27 novel coronavirus pneumonia cases found in Wuhan, China in December 2019, named as 2019-nCoV temporarily and COVID-19 formally by WHO on 11 February, 2020. In December 2019 and January 2020, COVID-19 has spread in large scale among the population, which brought terrible disaster to the life and property of the Chinese people. <strong>In this paper, we will first analyze the feature and pattern of the virus <strong style="color: red">transmission</strong>, and discuss the key impact factors and uncontrollable factors of epidemic <strong style="color: red">transmission</strong> based on public data.</strong> Then the virus <strong style="color: red">transmission</strong> can be modelled and used for the inflexion and extinction period of epidemic development so as to provide theoretical support for the Chinese government in the decision-making of epidemic prevention and recovery of economic production. Further, this paper demonstrates the effectiveness of the prevention methods taken by the Chinese government such as multi-level administrative region isolation. It is of great importance and practical significance for the world to deal with public health emergencies.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| myu0e9vf | Information Mining for COVID-19 Research From a Large Volume of Scientific Literature                                                                                                           | The year 2020 has seen an unprecedented COVID-19 pandemic due to the outbreak of a novel strain of coronavirus in 180 countries. In a desperate effort to discover new drugs and vaccines for COVID-19, many scientists are working around the clock. Their valuable time and effort may benefit from computer-based mining of a large volume of health science literature that is a treasure trove of information. <strong>In this paper, we have developed a graph-based model using abstracts of 10,683 scientific articles to find key information on three topics: <strong style="color: red">transmission</strong>, drug types, and genome research related to coronavirus.</strong> A subgraph is built for each of the three topics to extract more topic-focused information. Within each subgraph, we use a betweenness centrality measurement to rank order the importance of keywords related to drugs, diseases, pathogens, hosts of pathogens, and biomolecules. The results reveal intriguing information about antiviral drugs (Chloroquine, Amantadine, Dexamethasone), pathogen-hosts (pigs, bats, macaque, cynomolgus), viral pathogens (zika, dengue, malaria, and several viruses in the coronaviridae virus family), and proteins and therapeutic mechanisms (oligonucleotide, interferon, glycoprotein) in connection with the core topic of coronavirus. The categorical summary of these keywords and topics may be a useful reference to expedite and recommend new and alternative directions for COVID-19 research.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   |
| ibwo59yh | Assessment of 21 Days Lockdown Effect in Some States and Overall India: A Predictive Mathematical Study on COVID-19 Outbreak                                                                    | As of April, 6th, 2020, the total number of COVID-19 reported cases and deaths are 4778 and 136. This is an alarming situation as with a huge population within few days India will enter in stage-3 of COVID-19 <strong style="color: red">transmission</strong>. In the absence of neither an effective treatment or vaccine and with an incomplete understanding of the epidemiological cycle, predictive mathematical models can help exploring of both COVID-19 <strong style="color: red">transmission</strong> and control. <strong>In this present study, we consider a new mathematical model on COVID-19 <strong style="color: red">transmission</strong> that incorporate lock-down effect and variability in <strong style="color: red">transmission</strong> between symptomatic and asymptomatic populations with former being a fast spreader of the disease.</strong> Using daily COVID-19 notified cases from three states (Maharashtra, Delhi, and Telangana) and overall India, we assess the effect of current 21 days lock-down in terms of reduction cases and deaths. Lock-down effect is studied with different lock-down success rate. Our result suggest that 21 days lock-down will have no impact in Maharashtra and overall India. Furthermore, the presence of a higher percentage of COVID-19 super-spreaders will further deteriorate the situation in Maharashtra. However, for Tamil Nadu and Delhi there is some ray of hope as our prediction shows that lock-down will reduce a significant number of cases and deaths. in these two locations. Further extension of lock-down may place Delhi and Tamil Nadu in a comfort zone. Comparing estimated parameter samples for the mentioned four locations, we find a correlation between effect of lockdown and percentage of symptomatic infected in a region. Our result suggests that a higher percentage of symptomatic infected in a region leads to a large number of reduction in notified cases and deaths due to different lock-down scenario. Finally, we suggest a policy for the Indian Govt to control COVID-19 outbreak.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| g521vbth | Scenario analysis of non-pharmaceutical interventions on global COVID-19 transmissions                                                                                                          | <strong>This paper introduces a dynamic panel SIR (DP-SIR) model to investigate the impact of non-pharmaceutical interventions (NPIs) on the COVID-19 <strong style="color: red">transmission</strong> dynamics with panel data from 9 countries across the globe.</strong> By constructing scenarios with different combinations of NPIs, our empirical findings suggest that countries may avoid the lockdown policy with imposing school closure, mask wearing and centralized quarantine to reach similar outcomes on controlling the COVID-19 infection. Our results also suggest that, as of April 4th, 2020, certain countries such as the U.S. and Singapore may require additional measures of NPIs in order to control disease <strong style="color: red">transmission</strong>s more effectively, while other countries may cautiously consider to gradually lift some NPIs to mitigate the costs to the overall economy.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| ir43zqea | Real-Time Differential Epidemic Analysis and Prediction for COVID-19 Pandemic                                                                                                                   | <strong>In this paper, we propose a new real-time differential virus <strong style="color: red">transmission</strong> model, which can give more accurate and robust short-term predictions of COVID-19 transmitted infectious disease with benefits of near-term projection.</strong> Different from the existing Susceptible-Exposed-Infected-Removed (SEIR) based virus <strong style="color: red">transmission</strong> models, which fits well for pandemic modeling with sufficient historical data, the new model, which is also SEIR based, uses short history data to find the trend of the changing disease dynamics for the infected, the dead and the recovered so that it can naturally accommodate the adaptive real-time changes of disease mitigation, business activity and social behavior of populations. As the parameters of the improved SEIR models are trained by short history window data for accurate trend prediction, our differential epidemic model, essentially are window-based time-varying SEIR model. Since SEIR model still is a physics-based disease <strong style="color: red">transmission</strong> model, its near-term (like one month) projection can still be very instrumental for policy makers to guide their decision for disease mitigation and business activity policy change in a real-time. This is especially useful if the pandemic lasts more than one year with different phases across the world like 1918 flu pandemic. Numerical results on the recent COVID-19 data from China, Italy and US, California and New York states have been analyzed. Based on the projection as of April 13, 2020 from the proposed model, the time for peak medical resource usage for US as a whole will be around the beginning of May. Also US will reach the peak in terms of daily new infected cases and death cases around middle of April. The total cumulative infected cases will reach to peak of 1.09 million people around June 12, 2020 and the estimated total death cases will reach to 90K in the end.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| sb0mjhzm | Controlling the Transmission Dynamics of COVID-19                                                                                                                                               | The outbreak of COVID-19 caused by SARS-CoV-2 in Wuhan and other cities in China in 2019 has become a global pandemic as declared by World Health Organization (WHO) in the first quarter of 2020 . The delay in diagnosis, limited hospital resources and other treatment resources leads to rapid spread of COVID-19. <strong>In this article, we consider an optimal control COVID-19 <strong style="color: red">transmission</strong> model and assess the impact of some control measures that can lead to the reduction of exposed and infectious individuals in the population.</strong> We investigate three control strategies for this deadly infectious disease using personal protection, treatment with early diagnosis, treatment with delay diagnosis and spraying of virus in the environment as time-dependent control functions in our dynamical model to curb the disease spread.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 9z3skg4n | Impact of intervention on the spread of COVID-19 in India: A model based study                                                                                                                  | The outbreak of corona virus disease 2019 (COVID-19), caused by the virus severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) has already created emergency situations in almost every country of the world. The disease spreads all over the world within a very short period of time after its first identification in Wuhan, China in December, 2019. In India, the outbreaks starts on $2^{nd}$ March, 2020 and after that the cases are increasing exponentially. Very high population density, the unavailability of specific medicines or vaccines, insufficient evidences regarding the <strong style="color: red">transmission</strong> mechanism of the disease also make it difficult to fight against the disease properly in India. Mathematical models have been used to predict the disease dynamics and also to assess the efficiency of the intervention strategies in reducing the disease burden. <strong>In this work, we propose a mathematical model to describe the disease <strong style="color: red">transmission</strong> mechanism between the individuals.</strong> We consider the initial phase of the outbreak situation in India and our proposed model is fitted to the daily cumulative new reported cases during the period $2^{nd}$ March, 2020 to $24^{th}$ March, 2020. We estimate the basic reproduction number $(R_0)$, effective reproduction number (R(t)) and epidemic doubling time from the incidence data for the above-mentioned period. We further assess the effect of preventive measures such as spread of awareness, lock-down, proper hand sanitization, etc. in reducing the new cases. Two intervention scenarios are considered depending on the variability of the intervention strength over the period of implementation. Our study suggests that higher intervention effort is required to control the disease outbreak within a shorter period of time in India. Moreover, our analysis reveals that the strength of the intervention should be strengthened over the time to eradicate the disease effectively.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| d91t5o01 | Connecting the Dots of COVID-19 Transmissions in India                                                                                                                                          | Witnessing its first case in late January 2020 India has seen a sharp rise in the number of positive cases of COVID-19. 34 States/UT (s) of the country have been found to be affected by the pandemic to date. <strong>We in this work, study the progress of COVID-19 pandemic in India.</strong> <strong>We aim to create <strong style="color: red">transmission</strong> network visualization (s) of COVID-19 in India and perform analysis upon them.</strong> Using the <strong style="color: red">transmission</strong> networks obtained we attempt to find the possible Super Spreader Individual (s) and Super Spreader Events (SSE) responsible for the outbreak in their respective regions. We discuss the potentials of network analysis in mitigating the further spread of the disease. This is one of the initial studies of the outbreak in India and the first attempt to study the pandemic in the country from a <strong style="color: red">transmission</strong> network perspective.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| y3joi01a | Artificial Intelligence Forecasting of Covid-19 in China                                                                                                                                        | <strong>BACKGROUND An alternative to epidemiological models for <strong style="color: red">transmission</strong> dynamics of Covid-19 in China, we propose the artificial intelligence (AI)-inspired methods for real-time forecasting of Covid-19 to estimate the size, lengths and ending time of Covid-19 across China.</strong> METHODS We developed a modified stacked auto-encoder for modeling the <strong style="color: red">transmission</strong> dynamics of the epidemics. We applied this model to real-time forecasting the confirmed cases of Covid-19 across China. The data were collected from January 11 to February 27, 2020 by WHO. We used the latent variables in the auto-encoder and clustering algorithms to group the provinces/cities for investigating the <strong style="color: red">transmission</strong> structure. RESULTS We forecasted curves of cumulative confirmed cases of Covid-19 across China from Jan 20, 2020 to April 20, 2020. Using the multiple-step forecasting, the estimated average errors of 6-step, 7-step, 8-step, 9-step and 10-step forecasting were 1.64%, 2.27%, 2.14%, 2.08%, 0.73%, respectively. We predicted that the time points of the provinces/cities entering the plateau of the forecasted <strong style="color: red">transmission</strong> dynamic curves varied, ranging from Jan 21 to April 19, 2020. The 34 provinces/cities were grouped into 9 clusters. CONCLUSIONS The accuracy of the AI-based methods for forecasting the trajectory of Covid-19 was high. We predicted that the epidemics of Covid-19 will be over by the middle of April. If the data are reliable and there are no second <strong style="color: red">transmission</strong>s, we can accurately forecast the <strong style="color: red">transmission</strong> dynamics of the Covid-19 across the provinces/cities in China. The AI-inspired methods are a powerful tool for helping public health planning and policymaking.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| feen4e8w | Modeling the Control of COVID-19: Impact of Policy Interventions and Meteorological Factors                                                                                                     | <strong>In this paper, we propose a dynamical model to describe the <strong style="color: red">transmission</strong> of COVID-19, which is spreading in China and many other countries.</strong> To avoid a larger outbreak in the worldwide, Chinese government carried out a series of strong strategies to prevent the situation from deteriorating. Home quarantine is the most important one to prevent the spread of COVID-19. In order to estimate the effect of population quarantine, we divide the population into seven categories for simulation. Based on a Least-Squares procedure and officially published data, the estimation of parameters for the proposed model is given. Numerical simulations show that the proposed model can describe the <strong style="color: red">transmission</strong> of COVID-19 accurately, the corresponding prediction of the trend of the disease is given. The home quarantine strategy plays an important role in controlling the disease spread and speeding up the decline of COVID-19. The control reproduction number of most provinces in China are analyzed and discussed adequately. We should pay attention to that, though the epidemic is in decline in China, the disease still has high risk of human-to-human <strong style="color: red">transmission</strong> continuously. Once the control strategy is removed, COVID-19 may become a normal epidemic disease just like flu. Further control for the disease is still necessary, we focus on the relationship between the spread rate of the virus and the meteorological conditions. A comprehensive meteorological index is introduced to represent the impact of meteorological factors on both high and low migration groups. As the progress on the new vaccine, we design detail vaccination strategies for COVID-19 in different control phases and show the effectiveness of efficient vaccination. Once the vaccine comes into use, the numerical simulation provide a promptly prospective research.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 32gw2ug3 | Passengers' destinations from China: low risk of Novel Coronavirus (2019-nCoV) transmission into Africa and South America                                                                       | Novel Coronavirus (2019-nCoV [SARS-COV-2]) was detected in humans during the last week of December 2019 at Wuhan city in China, and caused 24 554 cases in 27 countries and territories as of 5 February 2020. <strong>The objective of this study was to estimate the risk of <strong style="color: red">transmission</strong> of 2019-nCoV through human passenger air flight from four major cities of China (Wuhan, Beijing, Shanghai and Guangzhou) to the passengers' destination countries.</strong> We extracted the weekly simulated passengers' end destination data for the period of 1–31 January 2020 from FLIRT, an online air travel dataset that uses information from 800 airlines to show the direct flight and passengers' end destination. We estimated a risk index of 2019-nCoV <strong style="color: red">transmission</strong> based on the number of travellers to destination countries, weighted by the number of confirmed cases of the departed city reported by the World Health Organization (WHO). We ranked each country based on the risk index in four quantiles (4(th) quantile being the highest risk and 1(st) quantile being the lowest risk). During the period, 388 287 passengers were destined for 1297 airports in 168 countries or territories across the world. The risk index of 2019-nCoV among the countries had a very high correlation with the WHO-reported confirmed cases (0.97). According to our risk score classification, of the countries that reported at least one Coronavirus-infected pneumonia (COVID-19) case as of 5 February 2020, 24 countries were in the 4(th) quantile of the risk index, two in the 3(rd) quantile, one in the 2(nd) quantile and none in the 1(st) quantile. Outside China, countries with a higher risk of 2019-nCoV <strong style="color: red">transmission</strong> are Thailand, Cambodia, Malaysia, Canada and the USA, all of which reported at least one case. In pan-Europe, UK, France, Russia, Germany and Italy; in North America, USA and Canada; in Oceania, Australia had high risk, all of them reported at least one case. In Africa and South America, the risk of <strong style="color: red">transmission</strong> is very low with Ethiopia, South Africa, Egypt, Mauritius and Brazil showing a similar risk of <strong style="color: red">transmission</strong> compared to the risk of any of the countries where at least one case is detected. The risk of <strong style="color: red">transmission</strong> on 31 January 2020 was very high in neighbouring Asian countries, followed by Europe (UK, France, Russia and Germany), Oceania (Australia) and North America (USA and Canada). Increased public health response including early case recognition, isolation of identified case, contract tracing and targeted airport screening, public awareness and vigilance of health workers will help mitigate the force of further spread to naïve countries.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 |
| uvavdk6n | Early dynamics of transmission and control of COVID-19: a mathematical modelling study                                                                                                          | Summary Background An outbreak of severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2) has led to 95 333 confirmed cases as of March 5, 2020. Understanding the early <strong style="color: red">transmission</strong> dynamics of the infection and evaluating the effectiveness of control measures is crucial for assessing the potential for sustained <strong style="color: red">transmission</strong> to occur in new areas. Combining a mathematical model of severe SARS-CoV-2 <strong style="color: red">transmission</strong> with four datasets from within and outside Wuhan, we estimated how <strong style="color: red">transmission</strong> in Wuhan varied between December, 2019, and February, 2020. We used these estimates to assess the potential for sustained human-to-human <strong style="color: red">transmission</strong> to occur in locations outside Wuhan if cases were introduced. <strong>Methods We combined a stochastic <strong style="color: red">transmission</strong> model with data on cases of coronavirus disease 2019 (COVID-19) in Wuhan and international cases that originated in Wuhan to estimate how <strong style="color: red">transmission</strong> had varied over time during January, 2020, and February, 2020.</strong> Based on these estimates, we then calculated the probability that newly introduced cases might generate outbreaks in other areas. To estimate the early dynamics of <strong style="color: red">transmission</strong> in Wuhan, we fitted a stochastic <strong style="color: red">transmission</strong> dynamic model to multiple publicly available datasets on cases in Wuhan and internationally exported cases from Wuhan. The four datasets we fitted to were: daily number of new internationally exported cases (or lack thereof), by date of onset, as of Jan 26, 2020; daily number of new cases in Wuhan with no market exposure, by date of onset, between Dec 1, 2019, and Jan 1, 2020; daily number of new cases in China, by date of onset, between Dec 29, 2019, and Jan 23, 2020; and proportion of infected passengers on evacuation flights between Jan 29, 2020, and Feb 4, 2020. We used an additional two datasets for comparison with model outputs: daily number of new exported cases from Wuhan (or lack thereof) in countries with high connectivity to Wuhan (ie, top 20 most at-risk countries), by date of confirmation, as of Feb 10, 2020; and data on new confirmed cases reported in Wuhan between Jan 16, 2020, and Feb 11, 2020. Findings We estimated that the median daily reproduction number (R t) in Wuhan declined from 2·35 (95% CI 1·15–4·77) 1 week before travel restrictions were introduced on Jan 23, 2020, to 1·05 (0·41–2·39) 1 week after. Based on our estimates of R t, assuming SARS-like variation, we calculated that in locations with similar <strong style="color: red">transmission</strong> potential to Wuhan in early January, once there are at least four independently introduced cases, there is a more than 50% chance the infection will establish within that population. Interpretation Our results show that COVID-19 <strong style="color: red">transmission</strong> probably declined in Wuhan during late January, 2020, coinciding with the introduction of travel control measures. As more cases arrive in international locations with similar <strong style="color: red">transmission</strong> potential to Wuhan before these control measures, it is likely many chains of <strong style="color: red">transmission</strong> will fail to establish initially, but might lead to new outbreaks eventually. Funding Wellcome Trust, Health Data Research UK, Bill & Melinda Gates Foundation, and National Institute for Health Research.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| kwq2y3il | Coronavirus Disease 2019: Coronaviruses and Blood Safety                                                                                                                                        | Abstract With the outbreak of unknown pneumonia in Wuhan, China, in December 2019, a new coronavirus, Severe Acute Respiratory Syndrome Coronavirus 2 (SARS-CoV-2), aroused the attention of the entire world. The current outbreak of infections with SARS-CoV-2 is termed Coronavirus Disease 2019 (COVID-19). The World Health Organization declared COVID-19 in China as a Public Health Emergency of International Concern. Two other coronavirus infections—SARS in 2002-2003 and Middle East Respiratory Syndrome (MERS) in 2012—both caused severe respiratory syndrome in humans. All 3 of these emerging infectious diseases leading to a global spread are caused by β-coronaviruses. Although coronaviruses usually infect the upper or lower respiratory tract, viral shedding in plasma or serum is common. Therefore, there is still a theoretical risk of <strong style="color: red">transmission</strong> of coronaviruses through the transfusion of labile blood products. Because more and more asymptomatic infections are being found among COVID-19 cases, considerations of blood safety and coronaviruses have arisen especially in endemic areas. <strong>In this review, we detail current evidence and understanding of the <strong style="color: red">transmission</strong> of SARS-CoV, MERS–CoV, and SARS-CoV-2 through blood products as of February 10, 2020, and also discuss pathogen inactivation methods on coronaviruses.</strong>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          |
| xuczplaf | The epidemiology and pathogenesis of coronavirus disease (COVID-19) outbreak                                                                                                                    | Abstract Coronavirus disease (COVID-19) is caused by SARS-COV2 and represents the causative agent of a potentially fatal disease that is of great global public health concern. Based on the large number of infected people that were exposed to the wet animal market in Wuhan City, China, it is suggested that this is likely the zoonotic origin of COVID-19. Person-to-person <strong style="color: red">transmission</strong> of COVID-19 infection led to the isolation of patients that were subsequently administered a variety of treatments. Extensive measures to reduce person-to-person <strong style="color: red">transmission</strong> of COVID-19 have been implemented to control the current outbreak. Special attention and efforts to protect or reduce <strong style="color: red">transmission</strong> should be applied in susceptible populations including children, health care providers, and elderly people. <strong>In this review, we highlights the symptoms, epidemiology, <strong style="color: red">transmission</strong>, pathogenesis, phylogenetic analysis and future directions to control the spread of this fatal disease.</strong>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 9mvmwchh | Discussion on Etiology and Pathogenesis of Corona Virus Disease 2019 from “Cold-dampness and Insidious Dryness”                                                                                 | <strong>Abstract Based on the characteristics of the epidemic situation and the authors’ understanding of the related ancient books and documents, this paper explores the etiology and pathogenesis of Corona Virus Disease 2019 (COVID-19) from 5 aspects: abnormal climate in “warm winter”, unique geographical location, pathogenesis evolution of cold and dampness mixed with insidious dryness, <strong style="color: red">transmission</strong> and change of “triple energizer” of toxic pathogens, and game between healthy Qi and toxic pathogens.</strong> <strong>Combined with the special treatment of traditional Chinese medicine (TCM), the purpose is to make a modest contribution to curbing the epidemic situation with TCM.</strong>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| drto66xt | Clinical characteristics and intrauterine vertical transmission potential of COVID-19 infection in nine pregnant women: a retrospective review of medical records                               | Summary Background Previous studies on the pneumonia outbreak caused by the 2019 novel coronavirus disease (COVID-19) were based on information from the general population. Limited data are available for pregnant women with COVID-19 pneumonia. <strong>This study aimed to evaluate the clinical characteristics of COVID-19 in pregnancy and the intrauterine vertical <strong style="color: red">transmission</strong> potential of COVID-19 infection.</strong> Methods Clinical records, laboratory results, and chest CT scans were retrospectively reviewed for nine pregnant women with laboratory-confirmed COVID-19 pneumonia (ie, with maternal throat swab samples that were positive for severe acute respiratory syndrome coronavirus 2 [SARS-CoV-2]) who were admitted to Zhongnan Hospital of Wuhan University, Wuhan, China, from Jan 20 to Jan 31, 2020. Evidence of intrauterine vertical <strong style="color: red">transmission</strong> was assessed by testing for the presence of SARS-CoV-2 in amniotic fluid, cord blood, and neonatal throat swab samples. Breastmilk samples were also collected and tested from patients after the first lactation. Findings All nine patients had a caesarean section in their third trimester. Seven patients presented with a fever. Other symptoms, including cough (in four of nine patients), myalgia (in three), sore throat (in two), and malaise (in two), were also observed. Fetal distress was monitored in two cases. Five of nine patients had lymphopenia (<1·0 × 10⁹ cells per L). Three patients had increased aminotransferase concentrations. None of the patients developed severe COVID-19 pneumonia or died, as of Feb 4, 2020. Nine livebirths were recorded. No neonatal asphyxia was observed in newborn babies. All nine livebirths had a 1-min Apgar score of 8–9 and a 5-min Apgar score of 9–10. Amniotic fluid, cord blood, neonatal throat swab, and breastmilk samples from six patients were tested for SARS-CoV-2, and all samples tested negative for the virus. Interpretation The clinical characteristics of COVID-19 pneumonia in pregnant women were similar to those reported for non-pregnant adult patients who developed COVID-19 pneumonia. Findings from this small group of cases suggest that there is currently no evidence for intrauterine infection caused by vertical <strong style="color: red">transmission</strong> in women who develop COVID-19 pneumonia in late pregnancy. Funding Hubei Science and Technology Plan, Wuhan University Medical Development Plan.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| si7csqr2 | Interrupting COVID-19 transmission by implementing enhanced traffic control bundling: Implications for global prevention and control efforts                                                    | Abstract <strong>We argue that enhanced Traffic Control Bundling (eTCB) can interrupt the community-hospital-community <strong style="color: red">transmission</strong> cycle, thereby limiting COVID-19’s impact.</strong> Enhanced TCB is an expansion of the traditional TCB that proved highly effective during Taiwan’s 2003 SARS outbreak. TCB’s success derived from ensuring that Health Care Workers (HCWs) and patients were protected from fomite, contact and droplet <strong style="color: red">transmission</strong> within hospitals. Although TCB proved successful during SARS, achieving a similar level of success with the COVID-19 outbreak requires adapting TCB to the unique manifestations of this new disease. These manifestations include asymptomatic infection, a hyper-affinity to ACE2 receptors resulting in high transmissibility, false negatives, and an incubation period of up to 22 days. Enhanced TCB incorporates the necessary adaptations. In particular, eTCB includes expanding the TCB transition zone to incorporate a new sector – the quarantine ward. This ward houses patients exhibiting atypical manifestations or awaiting definitive diagnosis. A second adaptation involves enhancing the checkpoint hand disinfection and gowning up with Personal Protective Equipment deployed in traditional TCB. Under eTCB, checkpoint hand disinfection and donning of face masks are now required of all visitors who seek to enter hospitals. These enhancements ensure that <strong style="color: red">transmission</strong>s by droplets, fomites and contact are disrupted both within hospitals and between hospitals and the broader community. Evidencing eTCB effectiveness is Taiwan’s success to date in containing and controlling the community-hospital-community <strong style="color: red">transmission</strong> cycle.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |

### Visualize aspect documents ###

In [86]:
display(box5)

Box(children=(Label(value='Visualize the documents in aspect:'), Dropdown(description='Aspect:', index=4, layo…

### Visualize abstract ###

In [45]:
display(box6)

Box(children=(Label(value='Visualize the abstract of:'), Dropdown(description='UID:', layout=Layout(width='max…